<font size="5">

# Multi-Grid measurements at LET, 2022
    
  

> __Author:__ A. Backis
<br/>__Institute:__ University of Glasgow (UoG), European Spallation Source (ESS)
    
> __Author:__ R. Wahlén
<br/>__Institute:__ Lund University (LU), European Spallation Source (ESS)

>__Date:__ 7/4-2022
    
_Abstract:_
This notebook contains the data-analysis tools used for the measurements at the LET instrument at ISIS. It describes how the analysis was performed, and summarizes the results.

# Contents
    
* [1. Introduction](#INTRODUCTION)
    * [1.1 Packages](#PACKAGES)
    * [1.2 Parameters](#PARAMETERS)
    * [1.3 Cluster and save](#CLUSTER_AND_SAVE)
    * [1.4 Helper functions](#HELPER_FUNCTIONS)
* [2. Utgård](#UTGARD)
    * [2.1 Example](#UTGARD_EXAMPLE)
    * [2.2 Without shielding](#UTGARD_WITHOUT_SHIELDING)
    * [2.3 With shielding](#UTGARD_WITH_SHIELDING)
* [3. STF](#STF)
    * [3.1 AmBe source](#STF_AMBE_SOURCE)
    * [3.2 Co-60 source](#STF_CO_60_SOURCE)
* [4. ISIS](#ISIS)
    * [4.1 Before beamtime](#ISIS_BEFORE_BEAMTIME)
        * [4.1.1 AmBe source](#ISIS_BEFORE_BEAMTIME_AMBE_SOURCE)
        * [4.1.2 Cave background](#ISIS_BEFORE_BEAMTIME_CAVE_BACKGROUND)
        * [4.1.3 Tank background](#ISIS_BEFORE_BEAMTIME_TANK_BACKGROUND)
        * [4.1.4 Position reconstruction](#ISIS_BEFORE_BEAMTIME_POSITION_RECONSTRUCTION)
    * [4.2 Beamtime](#ISIS_BEAMTIME)
        * [4.2.1 C$_{60}$](#ISIS_BEAMTIME_C60)
            * [4.2.1.1 White beam](#ISIS_BEAMTIME_C60_WHITE_BEAM)
            * [4.2.1.2 Monochromatic](#ISIS_BEAMTIME_C60_MONOCHROMATIC)
        * [4.2.2 V](#ISIS_BEAMTIME_V)
            * [4.2.2.1 White beam](#ISIS_BEAMTIME_V_WHITE_BEAM)
            * [4.2.2.2 Monochromatic](#ISIS_BEAMTIME_V_MONOCHROMATIC)
        * [4.2.3 SiO$_2$](#ISIS_BEAMTIME_SiO2)
        * [4.2.4 V & SiO$_2$](#ISIS_BEAMTIME_V_AND_SiO2)
    * [4.3 After beamtime](#ISIS_AFTER_BEAMTIME)
        * [4.3.1 Outside detector testing area](#OUTSIDE_DETECTOR_TESTING_AREA)
        * [4.3.2 AmBe source](#ISIS_AFTER_BEAMTIME_AMBE_SOURCE)
        * [4.3.3 Cave background](#ISIS_AFter_BEAMtimE_CAVE_BACKGROUND)
* [5. Summary](#SUMMARY)

# 1. Introduction<a class="anchor" id="INTRODUCTION"></a>

Notebook showing the analysis of the LET measurements.

## 1.1 Packages<a class="anchor" id="PACKAGES"></a>

Import required packages.

In [ ]:
# Autoload packages after conducting an external change
%load_ext autoreload
%autoreload 2

# Activate matplotlib in interactive notebook mode
%matplotlib widget

# General packages
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy.signal as sci_sig
import h5py
import tqdm
import matplotlib
import time
import datetime
import imageio
import shutil

# Matplotlib packages
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
from matplotlib.lines import Line2D
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Ensure custom packages can be reached
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Local packages
import file_handling.let.he3_read as he3_read
import file_handling.let.bm_read as bm_read
import file_handling.mg_ref.mg_ref_read as mg_read
import file_handling.mg_ref.mg_ref_manage as mg_manage
import file_handling.ess_he3_tube.read as ess_he3_tube_read
import calculations.distance_calibration as dclb
import calculations.energy as e_calc
import calculations.lineshape as ls
import calculations.normalization as norm
import calculations.absolute_time as at
import plotting.mg_ref_basic_plot as mg_basic_plot
import plotting.he3_basic_plot as he3_basic_plot
import plotting.ess_he3_tube_basic_plot as ess_he3_basic_plot
import plotting.bm_basic_plot as bm_basic_plot
import plotting.lineshape as lineshape
import plotting.common_plot as cmnplt
import plotting.helper_functions as hf

## 1.2 Parameters<a class="anchor" id="PARAMETERS"></a>

Define global parameters.

In [ ]:
# Instrument definition
MODERATOR_TO_SAMPLE_IN_M = 25
# Energies
Eis_in_meV = np.array([2.425, 3.675, 6.325, 13.425, 44.925])
# Multi-Grid detector position in LET tank relative to sample position
MG_OFFSET = {'x': -1.612, 'y': -0.862, 'z': 2.779}
MG_THETA = 57.181 * np.pi/180
# Delimiters for the separate peaks recieved in RRM mode
TOF_RANGES = [np.array([42746, 52754])-5500,
              np.array([32608, 39406])-3000,
              np.array([24809, 30478])-1900,
              np.array([16946, 22275])-1300,
              np.array([8000, 12000])]
TOF_RANGES_BM = [np.array([42746, 48754])-10500,
                 np.array([32608, 37406])-6500,
                 np.array([24809, 28978])-5400,
                 np.array([16946, 22275])-4800,
                 np.array([8000, 12000]) - 2000]

In [ ]:
# Declare lists with file names related to the separate measurements
mg_file_names_V = np.array([
                            'MG-Ref_ISIS_LETTank_VSampleI_MonoChrom_R78239_220504_184238',
                            'MG-Ref_ISIS_LETTank_VSampleI_MonoChromo_R78239_220505_004304',
                            'MG-Ref_ISIS_LETTank_VSampleI_MonoChromo_R78239_220505_081810',
                            'MG-Ref_ISIS_LETTank_VSampleI_MonoChrom_GasFlowMAX_R78239_220505_102636'
                            ])
mg_file_names_SiO2 = np.array([
                               'MG-Ref_ISIS_Th10_LETTank_SiO2SampleI_MonoChrom_GasFlowMAX_R78260_220505_170125',
                               'MG-Ref_ISIS_Th10_LETTank_SiO2SampleI_MonoChrom_GasFlowMAX_R78260_220505_185715',
                               'MG-Ref_ISIS_Th10_LETTank_SiO2SampleI_MonoChrom_GasFlowMAX_R78260_220505_201533',
                               'MG-Ref_ISIS_Th10_LETTank_SiO2SampleI_MonoChrom_GasFlowMAX_R78260_220506_013007',
                               ])
he3_file_names_V = [
                    'LET00078256',
                    'LET00078255',
                    'LET00078254',
                    'LET00078253',
                    'LET00078252',
                    'LET00078251',
                    'LET00078250',
                    'LET00078249',
                    'LET00078248',
                    'LET00078247',
                    'LET00078246',
                    'LET00078245',
                    'LET00078244',
                    'LET00078243',
                    'LET00078242',
                    'LET00078241',
                    'LET00078240',
                    'LET00078239'
                     ]
he3_file_names_SiO2 = [
                       'LET00078260',
                       'LET00078261',
                       'LET00078262',
                       'LET00078263',
                       'LET00078264',
                       'LET00078265',
                       'LET00078266',
                       'LET00078267',
                       'LET00078268',
                       'LET00078269',
                       'LET00078270',
                       'LET00078271',
                       'LET00078272',
                       'LET00078273',
                       'LET00078274'
                        ]

In [ ]:
# Declare lists with mg areas for the different measurements
mg_areas_V = 0.025*0.025*6*np.array([33, 33, 33, 33])
mg_areas_SiO2 = 0.025*0.025*6*np.array([33, 33, 33, 33])

In [ ]:
# Paths to folders containing data
nb_path = os.getcwd()
MG_RAW_FOLDER = nb_path + '/../data/mg/ref/raw/'
MG_PROCESSED_FOLDER = nb_path + '/../data/mg/ref/processed/'
HE3_FOLDER = nb_path + '/../data/he3/'
ESS_HE3_FOLDER = nb_path + '/../data/ess_he3/'

In [ ]:
# Filters for Multi-Grid detector
mg_filter_standard = {'wm': [1, 1, True],                   # Wire multiplicity
                      'gm': [1, 5, True],                   # Grid multiplicity
                      'wadc': [550, np.inf, True],          # Wire charge
                      'gadc': [550, np.inf, True],          # Grid charge
                      'tof': [0, np.inf, False],            # Time-of-flight (TDC channels)
                      'time': [0, np.inf, False],           # Time (TDC channels)
                      'bus': [9, 9, True],                 # Bus
                      'layer': [0, 19, False],              # Layer, front=0 to back=19
                      'row': [0, 11, False],                # Row, right to left (seen from neutrons)
                      'gch': [81, 118, False]}              # Grid channel, bottom=96 to top=132

mg_no_filter = {'wm': [1, 1, False],                   # Wire multiplicity
                'gm': [1, 5, False],                   # Grid multiplicity
                'wadc': [550, np.inf, False],          # Wire charge
                'gadc': [550, np.inf, False],          # Grid charge
                'tof': [0, np.inf, False],            # Time-of-flight (TDC channels)
                'time': [0, np.inf, False],           # Time (TDC channels)
                'bus': [9, 9, True],                 # Bus
                'layer': [0, 19, False],              # Layer, front=0 to back=19
                'row': [0, 11, False],                # Row, right to left (seen from neutrons)
                'gch': [81, 118, False]}              # Grid channel, bottom=96 to top=132

mg_filter_standard_no_prompt = {'wm': [1, 1, True],                   # Wire multiplicity
                                'gm': [1, 5, True],                   # Grid multiplicity
                                'wadc': [550, np.inf, True],          # Wire charge
                                'gadc': [550, np.inf, True],          # Grid charge
                                'tof': [0.002/(62.5e-9), np.inf, True],            # Time-of-flight (TDC channels)
                                'time': [0, np.inf, False],           # Time (TDC channels)
                                'bus': [9, 9, True],                 # Bus
                                'layer': [0, 19, False],              # Layer, front=0 to back=19
                                'row': [0, 11, False],                # Row, right to left (seen from neutrons)
                                'gch': [81, 118, False]}              # Grid channel, bottom=96 to top=132

In [ ]:
# Declare list of pixels with incorrect positions
pixels_with_incorrect_positions = np.array([125, 638, 1662, 3454, 3966, 4991, 5503, 6783,
                                            7551, 8318, 9087, 10110, 15742, 17279, 19582,
                                            19839, 20350, 20861, 20094, 19071, 16511, 16767,
                                            16254, 15998, 15487, 14975, 14719, 13695, 12416,
                                            9855, 8575, 8063, 7296, 7039, 6526, 5247, 4223, 2687, 1407])
# Filter for Helium-3 tubes: Multi-Grid sized region at correct polar angle
pixels = []
start_pixel = 39180+1
height = 58
diff_tubes = 256
number_tubes = 32
number_rows = 6
for i in np.arange(0, number_rows, 1):
    start = start_pixel+diff_tubes*i
    stop = start_pixel+diff_tubes*i + height
    for j in np.arange(start, stop):
        if j not in pixels_with_incorrect_positions:
            pixels.append(j)
region_of_interest = np.array(pixels)
# Define region edges of region of interest
region_edges = np.zeros(5, dtype='int')
region_edges[0] = region_of_interest[0]
region_edges[1] = region_of_interest[height-1]
region_edges[2] = region_of_interest[number_rows*height-1]
region_edges[3] = region_of_interest[number_rows*height-height]
region_edges[4] = region_of_interest[0]
# Filter full array without odd behaving tubes
pixels_semi_full = []
for i in np.arange(0, 384, 1):
    start = diff_tubes*i
    stop = diff_tubes*i + diff_tubes
    if i not in [48, 60, 61, 62, 63, 92]:
        for j in np.arange(start, stop):
            if j not in pixels_with_incorrect_positions:
                pixels_semi_full.append(j)
region_semi_full = np.array(pixels_semi_full)
# Make filter with tubes adjacent to the Multi-Grid
pixels_around_MG = []
tube_ids = np.concatenate((np.arange(0, 7, 1), np.arange(23, 32, 1)))
for tube_id in tube_ids:
    start = 256*tube_id
    stop = 256*tube_id + 58
    for j in np.arange(start, stop):
        if j not in pixels_with_incorrect_positions:
            pixels_around_MG.append(j)
region_around_MG = np.array(pixels_around_MG)
# Make filter with everything except pixels with incorrect posistions
pixels_full = []
for i in np.arange(0, 384, 1):
    start = diff_tubes*i
    stop = diff_tubes*i + diff_tubes
    for j in np.arange(start, stop):
        if j not in pixels_with_incorrect_positions:
            pixels_full.append(j)
region_full = np.array(pixels_full)
# Make filter with everything except shadow region pixels
pixels_shadow = []
for i in np.arange(8, 22, 1):
    start = diff_tubes*i
    stop = diff_tubes*i + 70
    for j in np.arange(start, stop):
        pixels_shadow.append(j)
pixels_full_except_shadow = []
for i in np.arange(0, 384, 1):
    start = diff_tubes*i
    stop = diff_tubes*i + diff_tubes
    for j in np.arange(start, stop):
        if (j not in pixels_with_incorrect_positions) & (j not in pixels_shadow):
            pixels_full_except_shadow.append(j)
region_full_except_shadow = np.array(pixels_full_except_shadow)

Cross-check that Multi-Grid area and Helium-3 array segment is of comparable size.

In [ ]:
# Extract areas
mg_area = 0.025*0.025*6*37
he3_area = norm.he3_get_area(len(region_of_interest))
# Print values
print('Multi-Grid area (mg_area): %.4f m^2' % mg_area)
print('Helium-3 area (he3_area): %.4f m^2' % he3_area)
print('Area ratio (mg_area/he3_area) %.4f' % (mg_area/he3_area))

## 1.3 Cluster and save<a class="anchor" id="IMPORT_CLUSTER_AND_SAVE"></a>

Import, cluster and save Multi-Grid data. This cell only needs to be run the first time the notebook is used.

In [ ]:
# Declare data sets
data_sets = np.array(['CSPEC-LET_STF_Test_220405_165718_Th6',
                      'CSPEC-LET_ISIS_FirstRun_220422_125443',
                      'CSPEC-LET_ISIS_FirstRun_220422_130155',
                      'CSPEC-LET_ISIS_FirstRun_newDAQ',
                      'CSPEC-LET_ISIS_FirstRun_220422_131159',
                      'CSPEC-LET_ISIS_FirstRun_220422_134929',
                      'CSPEC-LET_ISIS_FirstRun_220422_150521',
                      'CSPEC-LET_ISIS_FirstRun_220422_150624',
                      'CSPEC-LET_ISIS_FirstRun_220422_151925',
                      'CSPEC-LET_ISIS_FirstRun_220422_153133',
                      'CSPEC-ISIS_LongBG_NewDAQ_run009_220422_165729',
                      'CSPEC-ISIS_Test_NewDAQ_run010_220425_155807', 
                      'CSPEC-ISIS_Test_NewDAQ_220425_185547',
                      'CSPEC-ISIS_Test_NewDAQ_220425_190826',
                      'CSPEC-ISIS_Test_NewDAQ_220426_191809',
                      'CSPEC-ISIS_Test_NewDAQ_220427_092523',
                      'MG-Ref_ISIS_NewDAQ_LETTank_NoBeam_220427_121440',
                      'MG-Ref_ISIS_NewDAQ_LETTank_NoBeam_220427_153022',
                      'MG-Ref_ISIS_NewDAQ_LETTank_NoBeam_220428_131354',
                      'MG-Ref_ISIS_NewDAQ_LETTank_NoBeam_220428_152115',
                      'MG-Ref_ISIS_LETTank_NoBeam_220429_170646',
                      'MG-Ref_ISIS_LETTank_NoBeam_220502_145504',
                      'trig1InNIM2_MG-Ref_ISIS_LETTank_NoBeam_220504_094935',
                      'MG-Ref_ISIS_LETTank_C60Sample_WhiteBeam_220504_121017',
                      'MG-Ref_ISIS_LETTank_C60Sample_MONOChrom_220504_142229_R78237',
                      'MG-Ref_ISIS_LETTank_VSampleI_Whitish_220504_174222_R78238',
                      'MG-Ref_ISIS_LETTank_VSampleI_MonoChrom_R78239_220504_184238',
                      'MG-Ref_ISIS_LETTank_VSampleI_MonoChromo_R78239_220505_004304',
                      'MG-Ref_ISIS_LETTank_VSampleI_MonoChromo_R78239_220505_081810',
                      'MG-Ref_ISIS_LETTank_VSampleI_MonoChrom_GasFlowMAX_R78239_220505_133729',
                      'MG-Ref_ISIS_LETTank_VSampleI_MonoChrom_GasFlowMAX_R78239_220505_102636'
                      'MG-Ref_ISIS_Th10_LETTank_SiO2SampleI_MonoChrom_GasFlowMAX_R78260_220505_170125',
                      'MG-Ref_ISIS_Th10_LETTank_SiO2SampleI_MonoChrom_GasFlowMAX_R78260_220505_185715',
                      'MG-Ref_ISIS_Th10_LETTank_SiO2SampleI_MonoChrom_GasFlowMAX_R78260_220505_201533',
                      'MG-Ref_ISIS_Th10_LETTank_SiO2SampleI_MonoChrom_GasFlowMAX_R78260_220506_013007',
                      'DEBUGGING_DAVIDELab_1100HV_wTh8gTh6_220506_132601',
                      'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220506_144328',
                      'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220506_175829',
                      'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220507_171415',
                      'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220508_214118',
                      'DEBUGGING_DAVIDELab_1100HV_wTh8gTh6_220506_132601',
                      'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220506_144328',
                      'DEBUGGING_AmBe_NEWGAS_1100HV_wTh8gTh6_220510_104714',
                      'DEBUGGING_AmBe_GAS_1100HV_wTh8gTh6_220510_115332',
                      'DEBUGGING_AmBe_OLDGAS_1100HV_wTh8gTh6_220510_150015',
                      'DEBUGGING_AmBe_OLDGAS_1100HV_wTh8gTh6_220510_154405',
                      'DEBUGGING_AmBe_OLDGAS_1100HV_wTh8gTh6_220510_221901',
                      'DEBUGGING_AmBe_NewGAS_1100HV_wTh8gTh6_220511_121605'
                     ])
# Cluster and save all data sets (note that this only has to be done the first time the notebook runs)
for data_set in data_sets:
    mg_manage.extract_and_save(data_set, MG_RAW_FOLDER, MG_PROCESSED_FOLDER)

## 1.4 Helper functions<a class="anchor" id="HELPER_FUNCTIONS"></a>

Here we define a few useful helper functions for the analysis.

In [ ]:
def get_peak_energy_high_resolution(Ei_in_meV, energies):
    """ Extracts the center energy in an energy peak.
    
    Args:
        Ei_in_meV (float): Rough estimate of center energy.
        energies (np.array): Array of energies in the peak.
        
    Returns:
        Ei_in_meV_high_resolution (float): The center energy of the peak
                                           with a fine resolution.
    
    """
    lower_limit = Ei_in_meV - (Ei_in_meV * 0.2)
    upper_limit = Ei_in_meV + (Ei_in_meV * 0.2)
    hist, bin_edges = np.histogram(energies, bins=1000, range=[lower_limit, upper_limit])
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    idx_max_value = np.argmax(hist)
    Ei_in_meV_high_resolution = bin_centers[idx_max_value]
    return Ei_in_meV_high_resolution

def get_d_extra(tof_in_us, d, vi, moderator_to_sample_in_m):
    """ Extracts the extra travel distance by neutron when assuming elastic 
        scattering. Does this by first calculating the distance the neutron should
        have travelled with its velocity and tof. After that, the inferred distance is
        removed from that value, which gives the unaccounted distance the neutron travelled.
    
    Args:
        tof_in_us (float or np.array): Full time-of-flight of neutron
        d (float or np.array): Sample-to-detection distance of neutron
        vi (float or np.array): Estimated velocity of neutron based on incident energy
        moderator_to_sample_in_m (float): Moderator-to-sample distance of neutron
        
    Returns:
        Extra travel distance traveled by neutron in m
    
    """
    return vi*tof_in_us*1e-6 - (d+moderator_to_sample_in_m)

def get_shoulder_size(dE, dE_min, dE_max, norm):
    """ Function to calculate statistics in the shoulder region.
    
    Args:
        dE (np.array): Energy transfer values
        dE_min (float): Minimum energy transfer
        dE_max (float): Maximum energy transfer
        norm (float): Normalization constant to apply to the found
                      counts in the shoulder region
        
    Returns:
        shoulder_size (float): Number of counts in shoulder normalized
                               with the norm-constant
        shoulder_size_uncertainty (float): Statistical uncertainty on
                                           counts in shoulder region
    
    """
    counts = len(dE[(dE >= dE_min) & (dE <= dE_max)])
    shoulder_size = counts * norm
    shoulder_size_uncertainty = np.sqrt(counts) * norm
    return shoulder_size, shoulder_size_uncertainty

def get_tof_and_sd_d(Ei_in_meV, tof_range, df, detector, mg_offset, mg_theta, he3_mapping):
    """ Returns the time-of-flight values and sample-to-detection distances for 
        DataFrames containing either Multi-Grid data or Helium-3 data. 
    
    Args:
        Ei_in_meV (float): Incident neutron energy
        tof_range (list): List containing the minimum and maximum tof-value corresponding
                          to the incident energy being considered
        df (pd.DataFrame): DataFrame containing the measurement data, either Multi-Grid
                           or Helium-3 data
        detector (str): Identifier for which detector is being considered, either 'MG' or
                        'He-3'
        mg_offset (dict): Dictionary containing the offsets in (x, y, z) of the external
                          detector reference point in relation to the sample position
                          in the tank
        mg_theta (float): Rotation of the front of the detector vessel in relation to the
                          positive z-axis in the (z, x)-plane in the detector tank
        he3_mapping (dict): Dictionary containing the 'pixel_ID->(x,y,z,thetha,phi,r)'-mapping
                            of the Helium-3 tubes
        
    Returns:
        tof (np.array): Array containing tof-values
        sd_d (np.array): Array containing sample-to-detection distances
        df_peak (pd.DataFrame): DataFrame containing only events within the allowed tof-region
    
    """
    tof_min, tof_max = tof_range[0], tof_range[1]
    if detector == 'MG':
        df_peak = df[((df.tof * 62.5e-9 * 1e6) >= tof_min) & ((df.tof * 62.5e-9 * 1e6) <= tof_max)]
        tof = df_peak['tof'] * 62.5e-9 * 1e6
        sd_d = dclb.get_sample_to_detection_distances(df_peak['wch'], df_peak['gch'], mg_offset, mg_theta)
    else:
        df_peak = df[(df.tof >= tof_min) & (df.tof <= tof_max)]
        tof = df_peak['tof']
        sd_d = he3_mapping['r'][df_peak['pixel_id']]
    return tof, sd_d, df_peak

def plot_hist_normed(array, ax, min_val, max_val, color, label, linestyle, number_bins, include_hist=True):
    """ Function to plot histogram with errorbars normalized to the maximum value in the histogram.
    
    Args:
        array (np.array): Array containing values to be histogrammed
        ax (plt.axis): Matplotlib axis where data should be plotted on
        min_val (float): Minimum value in range of histogram
        max_val (float): Maximum value in range of histogram
        color (str): Color to be used in the plot
        label (str): Label to put on the plot
        linestyle (str): Linestyle to use in the plot
        number_bins (int): Number of bins to divide histogram into
        include_hist (bool): Option to include histogram in the plot instead of just the data points.
                             This option is set to True bu default.
    
    Yields:
        Plot of data in histogram
    
    Returns:
        bin_width (float): The bin-widht used in the histogram
    
    """
    hist, bins = np.histogram(array, range=[min_val, max_val], bins=number_bins)
    idxs_one = np.where(hist == 1)
    norm = 1/max(hist)
    bin_centers = (bins[:-1] + bins[1:]) / 2
    if include_hist:
        ax.hist(array, range=[min_val, max_val], bins=number_bins, weights=norm*np.ones(len(array)),
                color=color, histtype='step', linestyle=linestyle)
    hist, bins = np.histogram(array, range=[min_val, max_val], bins=number_bins, weights=norm*np.ones(len(array)))
    bin_centers = (bins[:-1] + bins[1:]) / 2
    # Extract error bars
    errors_up = np.sqrt(hist/(norm))*(norm)
    errors_down = np.sqrt(hist/(norm))*(norm)
    errors_down[idxs_one] = 0
    ax.errorbar(bin_centers, hist, (errors_down, errors_up), color=color, label=label, linestyle='', #capsize=3,
                marker='.')
    ax.set_ylim(1e-5, 1)
    ax.set_yscale('log')
    ax.set_yticks([1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5])
    ax.set_yticklabels(['1', '1e-1', '1e-2', '1e-3', '1e-4', '1e-5'])
    bin_width = bins[1] - bins[0]
    ax.grid(True, which='major', linestyle='dotted', zorder=0)
    return bin_width

def meV_to_A(energy_in_meV):
    """ Convert neutron energy in millielectronvolts to wavelength in Angstroms.
    
    Args:
        energy_in_meV (float): Neutron energy in [meV]

    Returns:
        wavelength_in_Angstrom (float): Neutron wavelength in [Angstrom]
    
    """
    wavelength_in_Angstrom = np.sqrt(81.81/energy_in_meV)
    return wavelength_in_Angstrom

def A_to_meV(wavelength_in_Angstrom):
    """ Convert neutron wavelength in Angstroms to energy in millielectronvolts.
    
    Args:
        wavelength_in_Angstrom (float): Neutron wavelength in [Angstrom]
        
    Returns:
        energy_in_meV (float): Neutron energy in [meV]
    
    """
    energy_in_meV = (81.81 /(wavelength_in_Angstrom ** 2))
    return energy_in_meV

def set_size(width, fraction=1):
   """Set figure dimensions to avoid scaling in LaTeX.

   Parameters
   ----------
   width: float
           Document textwidth or columnwidth in pts
   fraction: float, optional
           Fraction of the width which you wish the figure to occupy

   Returns
   -------
   fig_dim: tuple
           Dimensions of figure in inches
   """
   # Width of figure (in pts)
   fig_width_pt = width * fraction

   # Convert from pt to inches
   inches_per_pt = 1 / 72.27

   # Golden ratio to set aesthetic figure height
   # https://disq.us/p/2940ij3
   golden_ratio = (5**.5 - 1) / 2

   # Figure width in inches
   fig_width_in = fig_width_pt * inches_per_pt
   # Figure height in inches
   fig_height_in = fig_width_in * golden_ratio

   fig_dim = (fig_width_in, fig_height_in)
   print (fig_dim)
   return fig_dim

def set_style(textwidth=345, columnwidth=0, linewidth=0, paperwidth=0, fraction=1):

    tex_fonts = {
    # Use LaTeX to write all text
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": "Computer Modern Roman",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 10,
    'axes.titlesize': 10,
    "font.size": 10,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "axes.autolimit_mode": "round_numbers",
    "figure.subplot.bottom": 0.17,
    "figure.subplot.right": 0.95,
    "figure.subplot.left": 0.15,
    "figure.figsize": set_size(textwidth) # or above alternatives
    } 
    plt.rcParams.update(tex_fonts)

In [ ]:
# Enable LaTeX font plots with the correct font size
set_style()

# 2. Utgard<a class="anchor" id="UTGARD"></a>

## 2.1 Example<a class="anchor" id="UTGARD_EXAMPLE"></a>

In [ ]:
# Declare file name and area
file_name = 'CSPEC-LET_STF_Test_220405_165718_Th6'
mg_area = 0.025*0.025*6*37
# Cluster and save (only has to be done once)
mg_manage.extract_and_save(file_name, MG_RAW_FOLDER, MG_PROCESSED_FOLDER)
# Import data
clu, ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
# Plot data
mg_basic_plot.mg_plot_basic_bus(file_name,          # Name of file to be visualized
                                9,                  # Bus to plot data from, Bus=9 is used for the MG.REF detector
                                clu,                # DataFrame containing clusters from data file
                                ev,                 # DataFrame containing events from data file
                                mg_filter_standard, # Filter to apply to the data
                                mg_area)            # Active area of detector

## 2.2 With shielding<a class="anchor" id="UTGARD_WITH_SHIELDING"></a>

## 2.1 Without shielding<a class="anchor" id="UTGARD_WITHOUT_SHIELDING"></a>

# 3. STF<a class="anchor" id="STF"></a>

## 3.1 AmBe source<a class="anchor" id="STF_AMBE_SOURCE"></a>

### Plot overviews of datasets

In [ ]:
# Declare file names and areas
file_names = np.array(['CSPEC-LET_STF_Test_220405_165718_Th6'])
mg_areas = 0.025*0.025*6*np.array([37])
# Plot overview for each data set
for file_name, mg_area in zip(file_names, mg_areas):
    clu, ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
    mg_basic_plot.mg_plot_basic_bus(file_name, 9, clu, ev, mg_filter_standard, mg_area, file_name)

## 3.2 $^{60}$Co source<a class="anchor" id="STF_CO_60_SOURCE"></a>

# 4. ISIS<a class="anchor" id="ISIS"></a>

## 4.1 Before beamtime<a class="anchor" id="ISIS_BEFORE_BEAMTIME"></a>

### 4.1.1 AmBe source<a class="anchor" id="ISIS_BEFORE_BEAMTIME_AMBE_SOURCE"></a>

### Plot overviews of datasets

In [ ]:
# Declare file names and areas
file_names = np.array([
                       'CSPEC-LET_ISIS_FirstRun_220422_125443',
                       'CSPEC-LET_ISIS_FirstRun_220422_130155',
                       'CSPEC-LET_ISIS_FirstRun_newDAQ',
                       'CSPEC-LET_ISIS_FirstRun_220422_131159',
                       'CSPEC-LET_ISIS_FirstRun_220422_134929',
                       'CSPEC-LET_ISIS_FirstRun_220422_150521',
                       #'CSPEC-LET_ISIS_FirstRun_220422_150624',  # Ignore this file because the DAQ had an incorrect setting
                       'CSPEC-LET_ISIS_FirstRun_220422_151925',
                       'CSPEC-LET_ISIS_FirstRun_220422_153133'
                      ])
mg_areas = 0.025*0.025*6*np.array([37,
                                   37,
                                   37,
                                   37,
                                   37,
                                   37,
                                   #37,
                                   37,
                                   37])

for file_name, mg_area in zip(file_names, mg_areas):
    # Load data
    clu, ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
    # Plot data
    mg_basic_plot.mg_plot_basic_bus(file_name, 9, clu, ev, mg_filter_standard, mg_area, file_name)

### 4.1.2 Cave background<a class="anchor" id="ISIS_BEFORE_BEAMTIME_CAVE_BACKGROUND"></a>

### Plot overviews of datasets

In [ ]:
# Declare file names and areas
file_name = 'CSPEC-ISIS_LongBG_NewDAQ_run009_220422_165729'
mg_area = 0.025*0.025*6*37
# Extract and save (only has to be done once)
mg_manage.extract_and_save(file_name, MG_RAW_FOLDER, MG_PROCESSED_FOLDER)
# Load data
clu, ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
# Plot data
mg_basic_plot.mg_plot_basic_bus(file_name, 9, clu, ev, mg_filter_standard, mg_area, file_name)

### 4.1.3 Tank background<a class="anchor" id="ISIS_BEFORE_BEAMTIME_TANK_BACKGROUND"></a>

### Plot overviews of datasets

In [ ]:
# Declare file names and areas
file_names = np.array([
                       #'CSPEC-ISIS_Test_NewDAQ_run010_220425_155807', # Ignore because is a file with massive amount of noise in unused channels
                       'CSPEC-ISIS_Test_NewDAQ_220425_185547',
                       'CSPEC-ISIS_Test_NewDAQ_220425_190826',
                       'CSPEC-ISIS_Test_NewDAQ_220426_191809',
                       'CSPEC-ISIS_Test_NewDAQ_220427_092523',
                       'MG-Ref_ISIS_NewDAQ_LETTank_NoBeam_220427_121440',
                       'MG-Ref_ISIS_NewDAQ_LETTank_NoBeam_220427_153022',
                       'MG-Ref_ISIS_NewDAQ_LETTank_NoBeam_220428_131354',
                       'MG-Ref_ISIS_NewDAQ_LETTank_NoBeam_220428_152115',
                       'MG-Ref_ISIS_LETTank_NoBeam_220429_170646',
                       'MG-Ref_ISIS_LETTank_NoBeam_220502_145504',
                       #'trig1InNIM2_MG-Ref_ISIS_LETTank_NoBeam_220504_094935' # Ignore because was only a test for second input trigger
                       ])
mg_areas = 0.025*0.025*6*np.array([
                                  # 37,
                                   37,
                                   37,
                                   32,
                                   32,
                                   32,
                                   32,
                                   32,
                                   32,
                                   32,
                                   32,
                                  # 37
                                  ])

for file_name, mg_area in zip(file_names, mg_areas):
    # Extract and save (only has to be done once)
    #mg_manage.extract_and_save(file_name, MG_RAW_FOLDER, MG_PROCESSED_FOLDER)
    # Load data
    clu, ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
    # Plot data
    mg_basic_plot.mg_plot_basic_bus(file_name, 9, clu, ev, mg_filter_standard, mg_area, file_name)

### 4.1.4 Position reconstruction<a class="anchor" id="ISIS_BEFORE_BEAMTIME_POSITION_RECONSTRUCTION"></a>

In [ ]:
# Define helper function for calculation of center location of Helium-3 tubes
def reject_outliers(data, m = 2.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    return data[s<m]

def get_angles(a, b, c):
    C = np.arccos((a**2 + b**2 - c**2)/(2*a*b))
    A = np.arccos((b**2 + c**2 - a**2)/(2*b*c))
    B = np.arccos((c**2 + a**2 - b**2)/(2*c*a))
    return A, B, C

# Use data set from 2020 to extract pixel locations in the Helium-3 tube
he3_file_name = 'LET00068153'
he3_path = HE3_FOLDER + he3_file_name + '.nxs'
he3_mapping = he3_read.get_pixel_to_xyz_mapping(he3_path)

# Get average pixel locations
zs_first = he3_mapping['z'][0:256]
xs = he3_mapping['x']
ys = he3_mapping['y']
zs = he3_mapping['z']
xs_first = he3_mapping['x'][0:256]
zs_second = he3_mapping['z'][256:256*2]
xs_second = he3_mapping['x'][256:256*2]
zs_chunks = np.array_split(zs, int(len(zs)/256))
ys_chunks = np.array_split(ys, int(len(ys)/256))
xs_chunks = np.array_split(xs, int(len(xs)/256))
zs_av = []
ys_av = []
xs_av = []
for xs_chunk, zs_chunk, ys_chunk in zip(xs_chunks, zs_chunks, ys_chunks):
    xs_chunk_no_outliers = reject_outliers(xs_chunk, m = 2.)
    ys_chunk_no_outliers = reject_outliers(ys_chunk, m = 2.)
    zs_chunk_no_outliers = reject_outliers(zs_chunk, m = 2.)
    zs_av.append(sum(zs_chunk_no_outliers)/len(zs_chunk_no_outliers))
    ys_av.append(sum(ys_chunk_no_outliers)/len(ys_chunk_no_outliers))
    xs_av.append(sum(xs_chunk_no_outliers)/len(xs_chunk_no_outliers))
    
# Extract positions of tubes we want to use as references
tube_1 = [zs_av[8], xs_av[8]]
tube_2 = [zs_av[21], xs_av[21]]
    
# Go from knowledge of distances to find positions in (z, x)
blue_dashed = 0.705 + 0.5*2.54e-2
blue_solid = 0.611 + 0.5*2.54e-2
red_dashed = 0.611 + 0.5*2.54e-2
red_solid = 0.706 + 0.5*2.54e-2
green_solid = np.sqrt( (tube_1[0] - tube_2[0]) ** 2 + (tube_1[1] - tube_2[1]) ** 2 )

# Plot result of where we calculate the corners
fig, ax = plt.subplots()
ax.set_title('Position reconstruction')
circles = []
for x, z in zip(xs_av, zs_av):
    circle = plt.Circle((z, x), 0.5*2.54e-2, color='black', fill=False)
    circles.append(circle)
for circle in circles:
    ax.add_patch(circle)
ax.plot(zs_av, xs_av, marker='.', linestyle='', color='black', markersize=1)
ax.plot([tube_1[0], tube_2[0]], [tube_1[1], tube_2[1]], marker='o',
        linestyle='', color='green', markersize=4)
plt.gca().set_aspect('equal')
ax.set_xlabel('z (m)')
ax.set_ylabel('x (m)')
ax.set_xlim(2.2, 3.3)
ax.set_ylim(-2.3, -1.2)

# Get angles
Ab, Bb, Cb = get_angles(blue_solid, blue_dashed, green_solid)
Ar, Br, Cr = get_angles(red_solid, red_dashed, green_solid)

# To get to the point, we use the angle of the green line
angle_green_line = np.arctan((tube_2[1] - tube_1[1])/(tube_2[0] - tube_1[0]))
angle_b = angle_green_line + Ab
angle_r = angle_green_line + Ar
z_blue = blue_dashed*np.cos(angle_b) + tube_1[0]
x_blue = blue_dashed*np.sin(angle_b) + tube_1[1]
z_red = red_dashed*np.cos(angle_r) + tube_1[0]
x_red = red_dashed*np.sin(angle_r) + tube_1[1]
label_blue = 'z: %.2f m\nx: %.2f m' % (z_blue, x_blue)
label_red = 'z: %.2f m\nx: %.2f m' % (z_red, x_red)
ax.plot([z_blue], [x_blue], color='blue', marker='.', markersize='5', label="r1'", linestyle='', zorder=10)
ax.plot([z_red], [x_red], color='red', marker='.', markersize='5', label="r2'", linestyle='', zorder=10)

# Get necessary angles
front_angle = np.arctan((x_blue - x_red)/(z_blue - z_red))
side_angle = 0 - (np.pi/2 - np.arctan((x_blue - x_red)/(z_blue - z_red)))

# Draw shielding borders as a visual check
z_r3_prime = 0.335*np.cos(side_angle) + z_red
x_r3_prime = 0.335*np.sin(side_angle) + x_red
z_r4_prime = 0.335*np.cos(side_angle) + z_blue
x_r4_prime = 0.335*np.sin(side_angle) + x_blue
ax.plot([z_blue, z_red, z_r3_prime, z_r4_prime, z_blue], [x_blue, x_red, x_r3_prime, x_r4_prime, x_blue], color='black')

# Draw vessel borders as a visual check
z_r1 = z_blue + 0.0325*np.cos(side_angle) + 0.02*np.cos(front_angle + np.pi)
x_r1 = x_blue + 0.0325*np.sin(side_angle) + 0.02*np.sin(front_angle + np.pi)
z_r2 = z_red + 0.03*np.cos(side_angle) + 0.02*np.cos(front_angle)
x_r2 = x_red + 0.03*np.sin(side_angle) + 0.02*np.sin(front_angle)
z_r3 = 0.310*np.cos(side_angle) + z_r2
x_r3 = 0.310*np.sin(side_angle) + x_r2
z_r4 = 0.310*np.cos(side_angle) + z_r1
x_r4 = 0.310*np.sin(side_angle) + x_r1

ax.plot([z_r1], [x_r1], color='blue', marker='x', markersize='5',linestyle='', zorder=10, label='r1')
ax.plot([z_r2], [x_r2], color='red', marker='x', markersize='5', linestyle='', zorder=10, label='r2')

ax.plot([z_r1, z_r2, z_r3, z_r4, z_r1], [x_r1, x_r2, x_r3, x_r4, x_r1], color='black', linestyle='dotted')
ax.plot([z_r4], [x_r4], color='orange', marker='.', markersize=10, label='Reference point', linestyle='')
ax.legend(title='Positions')

# Use data from He-3 histogram to estimate where cadmium shielding begins
r1_height = 1.239 - 0.006
cadmium_height = 0.135
r1_height_above_cadmium = r1_height - cadmium_height
top_of_cadmium_in_data = -1.96
y_r3 = top_of_cadmium_in_data + r1_height_above_cadmium

# Perform cross-check of distances
distance_red_and_blue_circle = np.sqrt((x_red - x_blue)**2 + (z_red - z_blue)**2)
distance_red_and_blue_cross = np.sqrt((x_r1 - x_r2)**2 + (z_r1 - z_r2)**2)
print('Cross-check of values')
print('---------------------')
print('Distance red and blue circles: %.3f m (Measured: 0.35 m)' % distance_red_and_blue_circle)
print('Distance red and blue cross: %.3f m (CAD: 0.31 m)' % distance_red_and_blue_cross)
print()
print('...............')
print('Reference point')
print('x = %.3f m' % x_r4)
print('y = %.3f m' % y_r3)
print('z =  %.3f m' % z_r4)
print('...............')
print('Inclination (in the zx-plane, with respect to the positive z-axis)')
print('angle = %.3f degrees' % (front_angle*180/np.pi))

# Plot positions of voxels from this mapping
offset = {'x': x_r4, 'y': y_r3, 'z': z_r4}
theta = front_angle
for wch in np.arange(0, 96, 1):
    x,y,z = dclb.get_global_xyz(wch, 96, offset, theta)
    ax.plot([z], [x], color='black', marker='x', linestyle='', markersize=0.5)
    
# Plot edges of vessel bottom
bottom_z = np.array([-0.362, -0.362, 0.052, 0.052, -0.362])
bottom_x = np.array([0.358, -0.256, -0.256, 0.358, 0.358])
xs_b = np.empty([len(bottom_x)], dtype='float')
zs_b = np.empty([len(bottom_z)], dtype='float')
for i, (x, z) in enumerate(zip(bottom_x, bottom_z)):
    xs_b[i] = dclb.get_new_x(x, z, theta) + offset['x']
    zs_b[i] = dclb.get_new_z(x, z, theta) + offset['z']
ax.plot(zs_b, xs_b, color='black', linestyle='-.', label='Bottom')


plt.xlim(-4, 4)
plt.ylim(-4, 4)


fig.set_figwidth(7)
fig.set_figheight(7)
    
plt.tight_layout()

## 4.2 Beamtime<a class="anchor" id="ISIS_BEAMTIME"></a>

## 4.2.1 C$_{60}$<a class="anchor" id="ISIS_BEAMTIME_C60"></a>

### 4.2.1.1 White beam<a class="anchor" id="ISIS_BEAMTIME_C60_WHITE_BEAM"></a>

#### Plot overview of dataset

In [ ]:
# Declare file name and area
file_name = 'MG-Ref_ISIS_LETTank_C60Sample_WhiteBeam_220504_121017'
mg_area = 0.025*0.025*6*33
# Extract and save (only has to be done once)
#mg_manage.extract_and_save(file_name, MG_RAW_FOLDER, MG_PROCESSED_FOLDER)
# Load data
clu, ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
# Plot data
mg_basic_plot.mg_plot_basic_bus(file_name, 9, clu, ev, mg_filter_standard, mg_area, file_name)

#### Extracting beam monitor counts

First, we extract the beam monitor data.

In [ ]:
# Get beam monitor data
bm_path = HE3_FOLDER + 'LET00078236' + '.nxs'
bm_ev_C60_white = bm_read.import_data(bm_path)

Then, we plot the data.

In [ ]:
bm_basic_plot.bm_plot_basic(bm_ev_C60_white, 100, 'C-60, white beam')

#### Study background level in Helium-3 tubes in the Multi-Grid detector "shadow"

We start of by visualizing the shadow region and reference region we select for our analysis.

In [ ]:
# Import Helium-3 tubes
he3_file_name = 'LET00078236'
he3_path = HE3_FOLDER + he3_file_name + '.nxs'
he3_df = he3_read.import_histogram_data(he3_path)
he3_mapping = he3_read.get_pixel_to_xyz_mapping(he3_path)
# Import Multi-Grid data
mg_file_name = 'MG-Ref_ISIS_LETTank_C60Sample_WhiteBeam_220504_121017'
mg_clu, mg_ev = mg_manage.load_clusters_and_events(mg_file_name, MG_PROCESSED_FOLDER)
mg_clu_f = mg_manage.filter_data(mg_clu, mg_filter_standard)
# Define a region of the Helium-3 tubes where the shadow is located visually
pixels = []
start_pixel = 2048 + 256*4
height = 58
diff_tubes = 256
number_rows = 6
for i in np.arange(0, number_rows, 1):
    start = start_pixel+diff_tubes*i
    stop = start_pixel+diff_tubes*i + height
    for j in np.arange(start, stop):
        pixels.append(j)
shadow_region= np.array(pixels)
# Define region edges
shadow_region_edges = np.zeros(5, dtype='int')
shadow_region_edges[0] = shadow_region[0]
shadow_region_edges[1] = shadow_region[height-1]
shadow_region_edges[2] = shadow_region[number_rows*height-1]
shadow_region_edges[3] = shadow_region[number_rows*height-height]
shadow_region_edges[4] = shadow_region[0]
# Define define reference region of Helium-3 tubes
pixels = []
start_pixel = 39168
height = 58
diff_tubes = 256
number_rows = 6
for i in np.arange(0, number_rows, 1):
    start = start_pixel+diff_tubes*i
    stop = start_pixel+diff_tubes*i + height
    for j in np.arange(start, stop):
        pixels.append(j)
reference_region = np.array(pixels)
# Define region edges of region of interest
reference_region_edges = np.zeros(5, dtype='int')
reference_region_edges[0] = reference_region[0]
reference_region_edges[1] = reference_region[height-1]
reference_region_edges[2] = reference_region[number_rows*height-1]
reference_region_edges[3] = reference_region[number_rows*height-height]
reference_region_edges[4] = reference_region[0]
# Plot 3D overview to confirm that we have selected the correct area slice of Helium-3 tubes
cmnplt.plot_mg_and_he3_hist_3d(mg_clu_f, he3_df, he3_mapping, MG_OFFSET, MG_THETA, region_edges=shadow_region_edges, cmin=1, cmax=100000)
cmnplt.plot_mg_and_he3_hist_3d(mg_clu_f, he3_df, he3_mapping, MG_OFFSET, MG_THETA, region_edges=reference_region_edges, cmin=1, cmax=100000)

After that, we overlay the rate per unit area for these two regions.

In [ ]:
# Import Helium-3 tubes
he3_file_name = 'LET00078236'
he3_path = HE3_FOLDER + he3_file_name + '.nxs'
he3_df = he3_read.import_histogram_data(he3_path)
he3_mapping = he3_read.get_pixel_to_xyz_mapping(he3_path)
he3_duration = norm.he3_get_duration(he3_path)
he3_area = norm.he3_get_area(len(shadow_region))
column_labels = np.array(he3_df.columns.values.tolist())
# Filter on pixel ID
he3_df_shadow = he3_df.loc[shadow_region]
he3_df_reference = he3_df.loc[reference_region]
# Declare tof-limits
start, stop, step = 2000, 1e5, 500
delimiters = np.arange(start, stop, step)
delimiter_centers = (delimiters[:-1] + delimiters[1:]) / 2
shadow_tof_slice_counts = np.empty([len(delimiters)-1], dtype='int')
shadow_tof_slice_counts_unc = np.empty([len(delimiters)-1], dtype='float')
reference_tof_slice_counts = np.empty([len(delimiters)-1], dtype='int')
reference_tof_slice_counts_unc = np.empty([len(delimiters)-1], dtype='float')
for i, delimiter in enumerate(delimiters[:-1]):
    lower_limit, upper_limit = delimiter, delimiter + step
    column_values = column_labels[np.where((column_labels >= lower_limit) &
                                           (column_labels <= upper_limit))]
    he3_df_shadow_tof = he3_df_shadow.loc[:, he3_df_shadow.columns.isin(column_values)]
    counts_shadow = he3_df_shadow_tof.to_numpy().sum()
    shadow_tof_slice_counts[i] = counts_shadow
    shadow_tof_slice_counts_unc[i] = np.sqrt(counts_shadow) 
    he3_df_reference_tof = he3_df_reference.loc[:, he3_df_reference.columns.isin(column_values)]
    counts_reference = he3_df_reference_tof.to_numpy().sum()
    reference_tof_slice_counts[i] = counts_reference
    reference_tof_slice_counts_unc[i] = np.sqrt(counts_reference) 
# Define normalization constant
tof_fraction = step / (stop - start)
norm_constant = 1 /(he3_area * (he3_duration*tof_fraction))
# Plot results
fig = plt.figure()
plt.errorbar(delimiter_centers,
             shadow_tof_slice_counts*norm_constant,
             shadow_tof_slice_counts_unc*norm_constant,
             linestyle='',
             marker='.',
             color='blue',
             label='Shadow')
plt.errorbar(delimiter_centers,
             reference_tof_slice_counts*norm_constant,
             reference_tof_slice_counts_unc*norm_constant,
             linestyle='',
             marker='.',
             color='red',
             label='Reference')
plt.xlabel('tof (us)')
plt.ylabel('counts/m$^2$/s')
plt.yscale('log')
plt.grid(True, which='major', linestyle='--', zorder=0)
plt.grid(True, which='minor', linestyle='--', zorder=0)
plt.legend(title='Helium-3 array region')
plt.show()

After that, we look at the tof-dependence on the background rate in the shadow region.

In [ ]:
# Import Helium-3 data
he3_file_name = 'LET00078236'
he3_path = HE3_FOLDER + he3_file_name + '.nxs'
he3_df = he3_read.import_histogram_data(he3_path)
he3_mapping = he3_read.get_pixel_to_xyz_mapping(he3_path)
he3_duration = norm.he3_get_duration(he3_path)
he3_solid_angle = norm.he3_get_solid_angle_new(he3_mapping)

# Declare animation folder
animation_folder = nb_path + '/../output/animation/'
shutil.rmtree(animation_folder, ignore_errors=True)
hf.mkdir_p(animation_folder)

# Declare animation limits
start, stop, step = 2000, 50000, 500
delimiters = np.arange(start, stop, step)
for i, delimiter in enumerate(tqdm.tqdm(delimiters)):
    # Filter data on tof
    lower_limit, upper_limit = delimiter, delimiter + step
    column_labels = np.array(he3_df.columns.values.tolist())
    column_values = column_labels[np.where((column_labels >= lower_limit) &
                                           (column_labels <= upper_limit))]
    he3_df_f_tof = he3_df.loc[:, he3_df.columns.isin(column_values)]
    # Histogram 
    thetas = he3_mapping['theta']
    phis = he3_mapping['phi']
    weights = he3_df_f_tof.iloc[:, :].sum(axis=1).to_numpy() * (1/he3_duration)
    # Plot data
    vmin = (1/he3_duration)
    vmax = ((1e5)/he3_duration)*2
    fig = plt.figure()
    fig.suptitle('White neutron beam on C$_{60}$ ("Buckyballs")', fontsize=15)
    gs = GridSpec(2, 2, figure=fig)
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1, 0:2])
    # Ax 1
    __, __, __, im_1 = ax1.hist2d(phis, thetas, weights=weights,
                                  bins=[800, 800],
                                  norm=LogNorm(vmin=vmin, vmax=vmax),
                                  cmap='jet')
    divider = make_axes_locatable(ax1)
    cax_1 = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im_1, orientation='vertical', cax=cax_1, label='Counts/s')
    ax1.set_title('Full', fontsize=12)
    ax1.set_xlabel('$\Phi$ (degrees)')
    ax1.set_ylabel('$\Theta$ (degrees)')
        
    # Ax 2
    __, __, __, im_2 = ax2.hist2d(phis, thetas, weights=weights,
                                  bins=[800, 800],
                                  norm=LogNorm(vmin=vmin, vmax=vmax),
                                  cmap='jet')
    divider = make_axes_locatable(ax2)
    cax_2 = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im_2, orientation='vertical', cax=cax_2, label='Counts/s')
    ax2.set_xlabel('$\Phi$ (degrees)')
    ax2.set_ylabel('$\Theta$ (degrees)')
    ax2.set_xlim(-160, -125)
    ax2.set_ylim(30, 50)
    ax2.set_title('Zoom', fontsize=12)
    ax2.xaxis.set_major_locator(plt.MaxNLocator(4))
    ax2.yaxis.set_major_locator(plt.MaxNLocator(4))
    
    hist_sum = he3_df.iloc[:, :].sum(axis=0).to_numpy() * (1/he3_duration) * (1/he3_solid_angle)

    ax3.step(he3_df.columns.values.tolist(), hist_sum, color='black')
    ax3.set_xlim(0, 1e5)

    ax3.set_yscale('log')
    ax3.set_ylabel('Counts/s/sr')
    ax3.set_xlabel('tof ($\mu$s)')
    ax3.axvline(lower_limit, color='black')
    ax3.set_title('Time-of-flight', fontsize=12)
    ax3.grid(True, which='major', linestyle='--', zorder=0)
    ax3.grid(True, which='minor', linestyle='--', zorder=0)
    fig.set_figwidth(6)
    fig.set_figheight(5)
    plt.tight_layout()
    # Save data
    output_path = animation_folder + str(i) + '.png'
    fig.savefig(output_path, bbox_inches='tight')
    plt.close()

# Animate
output_path = '../output/shadow_tof_sweep.gif'
images = []
files = os.listdir(animation_folder)
files = [file[:-4] for file in files if file[-9:] != '.DS_Store' and file != '.gitignore']
for file in sorted(files, key=int):
    images.append(imageio.imread(animation_folder + file + '.png'))
imageio.mimsave(output_path, images)
shutil.rmtree(animation_folder, ignore_errors=True)

#### Investigate if Debye Scherrer cones are overlapping between Helium-3 tubes and Multi-Grid detector

In [ ]:
# Import Helium-3 data
he3_file_name = 'LET00078236'
he3_path = HE3_FOLDER + he3_file_name + '.nxs'
he3_df = he3_read.import_histogram_data(he3_path)
he3_df_f = he3_df.loc[region_full_except_shadow]
he3_mapping = he3_read.get_pixel_to_xyz_mapping(he3_path)
he3_mapping_df = pd.DataFrame(he3_mapping)
he3_duration = norm.he3_get_duration(he3_path)
he3_solid_angle = norm.he3_get_solid_angle_new(he3_mapping_df.loc[region_full_except_shadow])
# Import Multi-Grid data
mg_file_name = 'MG-Ref_ISIS_LETTank_C60Sample_WhiteBeam_220504_121017'
mg_clu, mg_ev = mg_manage.load_clusters_and_events(mg_file_name, MG_PROCESSED_FOLDER)
mg_clu_f = mg_manage.filter_data(mg_clu, mg_filter_standard)
mg_duration = (mg_clu_f.time.values[-1] - mg_clu_f.time.values[0]) * 62.5e-9
gchs_part_1 = np.arange(96, 114, 1)
gchs_part_2 = np.array([115])
gchs_part_3 = np.arange(116, 131, 1)
gchs_part_4 = np.array([131])
gchs = np.concatenate((gchs_part_1, gchs_part_2, gchs_part_3, gchs_part_4))
mg_solid_angle = norm.mg_get_solid_angle_new(gchs, MG_OFFSET, MG_THETA)
# Declare animation folder
animation_folder = nb_path + '/../output/animation/'
shutil.rmtree(animation_folder, ignore_errors=True)
hf.mkdir_p(animation_folder)
# Declare animation limits
start, stop, step = 2e-3, 40e-3, 0.25e-3 # 23.5e-3, 25e-3, 0.5e-3
delimiters = np.arange(start, stop, step)
#print(delimiters)
for i, delimiter in enumerate(tqdm.tqdm(delimiters)):
    # Filter data on tof
    lower_limit, upper_limit = delimiter, delimiter + step
    mg_clu_f_tof = mg_clu_f[(mg_clu_f.tof >= lower_limit/(62.5e-9)) & (mg_clu_f.tof <= upper_limit/(62.5e-9))]
    column_labels = np.array(he3_df_f.columns.values.tolist())
    column_values = column_labels[np.where((column_labels >= lower_limit*1e6) & (column_labels <= upper_limit*1e6))]
    he3_df_f_tof = he3_df_f.loc[:, he3_df_f.columns.isin(column_values)]
    # Histogram Helium-3 data
    theta_he3 = he3_mapping['theta'][region_full_except_shadow]
    phi_he3 = he3_mapping['phi'][region_full_except_shadow]
    weights_he3 = he3_df_f_tof.iloc[:, :].sum(axis=1).to_numpy()
    # Histogram Multi-Grid data
    H, __ = np.histogramdd(mg_clu_f_tof[['wch', 'gch']].values,
                           bins=(96, 37),
                           range=((0, 96), (96, 133)))
    weights_mg = np.empty([96*37], dtype='float')
    phi_mg = np.empty([96*37], dtype='float')
    theta_mg = np.empty([96*37], dtype='float')
    loc = 0
    for wch in range(0, 96):
        for gch in range(96, 133):
            x, y, z = dclb.get_global_xyz(wch, gch, MG_OFFSET, MG_THETA)
            # Convert to phi and theta
            theta_voxel = np.arctan(np.sqrt(x**2 + y**2)/z)
            if x > 0:
                phi_voxel = np.arctan(y/x)
            elif (x < 0) and (y >= 0):
                phi_voxel = np.arctan(y/x) + np.pi
            elif (x < 0) and (y < 0):
                phi_voxel = np.arctan(y/x) - np.pi
            elif (x == 0) and (y > 0):
                phi_voxel = np.pi/2
            elif (x == 0) and (y < 0):
                phi_voxel = - np.pi/2
            elif (x == 0) and (y == 0):
                phi_voxel = None
            # Insert into array
            phi_mg[loc] = phi_voxel * (180/np.pi)
            theta_mg[loc] = theta_voxel * (180/np.pi)
            weights_mg[loc] = H[wch, gch-96]
            loc += 1

    # Concatenate Multi-Grid and Helium-3 data
    phis = np.concatenate((phi_he3, phi_mg))
    thetas = np.concatenate((theta_he3, theta_mg))
    weights = np.concatenate((weights_he3 * (1/he3_duration), weights_mg * (1/mg_duration)))

    # Plot data
    vmin= 1/3600
    vmax = (1e5)/3600
    fig = plt.figure()
    fig.suptitle('White neutron beam on C$_{60}$ (``Buckyballs")', fontsize=15)
    gs = GridSpec(2, 2, figure=fig)
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1, 0:2])
    # Ax 1
    __, __, __, im_1 = ax1.hist2d(phis, thetas, weights=weights,
                                  bins=[800, 800],
                                  norm=LogNorm(vmin=vmin, vmax=vmax),
                                  cmap='jet')
    divider = make_axes_locatable(ax1)
    cax_1 = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im_1, orientation='vertical', cax=cax_1, label='Counts/s')
    ax1.set_title('Full', fontsize=12)
    ax1.set_xlabel('$\Phi$ (degrees)')
    ax1.set_ylabel('$\Theta$ (degrees)')
    
    ax1.text(-165, 55, 'Multi-Grid\ndetector', fontsize=8)
    ax1.arrow(-100, 52, -22, -7, head_width=5, head_length=5)
    
    ax1.text(75, 73, 'Helium-3\ntubes', fontsize=8)
    ax1.arrow(70, 80, -15, 0, head_width=5, head_length=5)
    
    # Ax 2
    __, __, __, im_2 = ax2.hist2d(phis, thetas, weights=weights,
                                  bins=[800, 800],
                                  norm=LogNorm(vmin=vmin, vmax=vmax),
                                  cmap='jet')
    divider = make_axes_locatable(ax2)
    cax_2 = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im_2, orientation='vertical', cax=cax_2, label='Counts/s')
    ax2.set_xlabel('$\Phi$ (degrees)')
    ax2.set_ylabel('$\Theta$ (degrees)')
    ax2.set_xlim(-160, -125)
    ax2.set_ylim(30, 50)
    ax2.set_title('Zoom', fontsize=12)
    ax2.xaxis.set_major_locator(plt.MaxNLocator(4))
    ax2.yaxis.set_major_locator(plt.MaxNLocator(4))
    
    hist_sum = he3_df_f.iloc[:, :].sum(axis=0).to_numpy() * (1/he3_duration) * (1/he3_solid_angle)
    ax3.hist(mg_clu_f.tof * 62.5e-9 * 1e6, bins=len(hist_sum), color='royalblue', histtype='step', label='Multi-Grid detector',
             weights=np.ones(mg_clu_f.shape[0])*(1/mg_duration)*(1/mg_solid_angle), linestyle='--')
    ax3.step(he3_df_f.columns.values.tolist(), hist_sum, color='indianred', label='Helium-3 tubes')
    ax3.legend()
    #ax3.set_xlim(start*1e6, stop*1e6)
    ax3.set_yscale('log')
    ax3.set_ylabel('Counts/s/sr')
    ax3.set_xlabel('tof ($\mu$s)')
    ax3.set_xlim(0, 1e5)
    ax3.axvline(lower_limit*1e6, color='black')
    ax3.set_title('Time-of-flight', fontsize=12)
    ax3.grid(True, which='major', linestyle='--', zorder=0)
    ax3.grid(True, which='minor', linestyle='--', zorder=0)
    fig.set_figwidth(6)
    fig.set_figheight(5)
    plt.tight_layout()
    # Save data
    output_path = animation_folder + str(i) + '.png'
    fig.savefig(output_path, bbox_inches='tight') #, dpi=600)
    plt.close()

# Animate
output_path = '../output/tof_sweep_mg_and_he3.gif'
images = []
files = os.listdir(animation_folder)
files = [file[:-4] for file in files if file[-9:] != '.DS_Store' and file != '.gitignore']
for file in sorted(files, key=int):
    images.append(imageio.imread(animation_folder + file + '.png'))
imageio.mimsave(output_path, images)
shutil.rmtree(animation_folder, ignore_errors=True)

### 4.2.1.2 Monochromatic<a class="anchor" id="ISIS_BEAMTIME_C60_MONOCHROMATIC"></a>

#### Plot overview of dataset

In [ ]:
# Declare file name and area
file_name = 'MG-Ref_ISIS_LETTank_C60Sample_MONOChrom_220504_142229_R78237'
mg_area = 0.025*0.025*6*33
# Extract and save (only has to be done once)
#mg_manage.extract_and_save(file_name, MG_RAW_FOLDER, MG_PROCESSED_FOLDER)
# Load data
clu, ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
# Plot data
mg_basic_plot.mg_plot_basic_bus(file_name, 9, clu, ev, mg_filter_standard, mg_area, file_name)

#### Extracting beam monitor counts

First, we extract the beam monitor data.

In [ ]:
# Get beam monitor data
bm_path = HE3_FOLDER + 'LET00078237' + '.nxs'
bm_ev_C60_rrm = bm_read.import_data(bm_path)

Then, we plot the data.

In [ ]:
bm_basic_plot.bm_plot_basic(bm_ev_C60_rrm, 100, 'C-60, RRM')

## 4.2.2 V<a class="anchor" id="ISIS_BEAMTIME_V"></a>

### 4.2.2.1 White beam<a class="anchor" id="ISIS_BEAMTIME_V_WHITE_BEAM"></a>

#### Plot overview of dataset

In [ ]:
# Declare file name and area
file_name = 'MG-Ref_ISIS_LETTank_VSampleI_Whitish_220504_174222_R78238'
mg_area = 0.025*0.025*6*32
# Extract and save (only has to be done once)
#mg_manage.extract_and_save(file_name, MG_RAW_FOLDER, MG_PROCESSED_FOLDER)
# Load data
clu, ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
# Plot data
mg_basic_plot.mg_plot_basic_bus(file_name, 9, clu, ev, mg_filter_standard, mg_area, file_name)

#### Extracting beam monitor counts

First, we extract the beam monitor data.

In [ ]:
# Get beam monitor data
bm_path = HE3_FOLDER + 'LET00078238' + '.nxs'
bm_ev_V_white = bm_read.import_data(bm_path)

Then, we plot the data.

In [ ]:
bm_basic_plot.bm_plot_basic(bm_ev_V_white, 100, 'Vanadium, white beam')

### 4.2.2.2 Monochromatic<a class="anchor" id="ISIS_BEAMTIME_C60_MONOCHROMATIC"></a>

#### Plot overviews of datasets

In [ ]:
# Plot with filter
for file_name, mg_area in zip(mg_file_names_V, mg_areas_V):
    clu, ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
    mg_basic_plot.mg_plot_basic_bus(file_name, 9, clu, ev, mg_filter_standard, mg_area, file_name)
# Plot without filter, but remove bus = -1 events, as these are not real clusters
#for file_name, mg_area in zip(mg_file_names_V, mg_areas_V):
#    clu, ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
#    mg_basic_plot.mg_plot_basic_bus(file_name, 9, clu[(clu.bus != -1)], ev, mg_no_filter, mg_area, file_name)

#### Compare time-of-flight and energy spectra between Multi-Grid detector and Helium-3 tubes

First, we append all of the Multi-Grid data sets gathered with the Vanadium sample.

In [ ]:
# Get Multi-Grid data
dfs_clu = np.empty([len(mg_file_names_V)], dtype='object')
dfs_ev = np.empty([len(mg_file_names_V)], dtype='object')
for i, file_name in enumerate(mg_file_names_V):
    df_clu, df_ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
    dfs_clu[i] = df_clu
    dfs_ev[i] = df_ev
mg_clu_V = mg_manage.merge_files(dfs_clu)
mg_ev_V = mg_manage.merge_files(dfs_ev)

Then, we append all of the Helium-3 data sets gathered with the Vanadium sample.

In [ ]:
# Get Helium-3 data
he3_dfs = np.empty([len(he3_file_names_V)], dtype='object')
he3_duration_V = 0
for i, he3_file_name in enumerate(he3_file_names_V):
    he3_path = HE3_FOLDER + he3_file_name + '.nxs'
    he3_ev_temp = he3_read.import_data(he3_path)
    he3_dfs[i] = he3_ev_temp
    he3_duration_V += norm.he3_get_duration(he3_path)
he3_ev_V = pd.concat(he3_dfs)
he3_mapping_V = he3_read.get_pixel_to_xyz_mapping(he3_path)
he3_mapping_df_V = pd.DataFrame(he3_mapping_V)

After that, we filter the data and extract the necessary parameters.

In [ ]:
# Define He-3 filter
he3_pixels = region_semi_full
# Filter data
mg_clu_f_V = mg_manage.filter_data(mg_clu_V, mg_filter_standard)
he3_ev_f_V = he3_ev_V[he3_ev_V['pixel_id'].isin(he3_pixels)]
# Get Multi-Grid detector duration
mg_duration_V = (mg_clu_f_V.time.values[-1] - mg_clu_f_V.time.values[0]) * 62.5e-9
# Get solid angle normalization
he3_solid_angle_V = norm.he3_get_solid_angle_new(he3_mapping_df_V
                                                 [he3_mapping_df_V.index.isin(he3_pixels)]
                                                )
gchs = np.concatenate((np.arange(96, 114, 1), np.array([115]), np.arange(116, 131, 1), np.array([131])))
mg_solid_angle = norm.mg_get_solid_angle_new(gchs, MG_OFFSET, MG_THETA)
# Extract tof and distance for each event
mg_tof_V = mg_clu_f_V['tof'] * 62.5e-9 * 1e6
mg_sd_d_V = dclb.get_sample_to_detection_distances(mg_clu_f_V['wch'], mg_clu_f_V['gch'], MG_OFFSET, MG_THETA)
he3_tof_V = he3_ev_f_V['tof']
he3_sd_d_V = he3_mapping_V['r'][he3_ev_f_V['pixel_id']]
# Extract weights
mg_weights_V = (1/mg_duration_V) * (1/mg_solid_angle) * np.ones(len(mg_tof_V))
he3_weights_V = (1/he3_duration_V) * (1/he3_solid_angle_V) * np.ones(len(he3_tof_V))

Finally, we plot the time-of-flight spectra and energy spectra for both the Multi-Grid detector and the Helium-3 tubes. In the tof-plots, we have also added the delimiters which will be used to cut on time-of-flight when the energy transfer plots will be calculated.

In [ ]:
# Declare number of bin parameters
number_bins_tof = 100
number_bins_E = 1000
# Declare tof ranges colors
colors = cm.rainbow(np.linspace(0, 1, 5))
# Plot time-of-flight
fig = plt.figure()
__, bins_mg = cmnplt.tof_histogram_plot(mg_tof_V, number_bins_tof, label='Multi-Grid detector',
                          color='royalblue', weights=mg_weights_V)
__, bins_he3 = cmnplt.tof_histogram_plot(he3_tof_V, number_bins_tof, label='Helium-3 tubes',
                          color='indianred', linestyle='--', weights=he3_weights_V)

bin_width = bins_he3[1] - bins_he3[0]
delimiter_labels = ['2.4 meV (5.8 Å)', '3.7 meV (4.7 Å)', '6.3 meV (3.6 Å)', '13.4 meV (2.5 Å)', '44.9 meV (1.3 Å)']

for tof_range, color, delimiter_label in zip(TOF_RANGES, colors, delimiter_labels):
    plt.text(tof_range[0], 2e3, delimiter_label, color=color, rotation=30)
    plt.axvline(x=tof_range[0], color=color, linestyle='-')
    plt.axvline(x=tof_range[1], color=color, linestyle='-')

plt.title('')
plt.title('bin width: %.1f $\mu$s' % bin_width, loc='right')
plt.legend()
plt.ylabel('counts/s/sr')
plt.xlim(0, 1e5)
plt.tight_layout()
plt.savefig('%s/../output/v_tof_histogram.png' % nb_path, dpi=600)
plt.show()
# Plot energy
fig = plt.figure()
cmnplt.energy_histogram_plot(mg_tof_V, mg_sd_d_V, number_bins_E,
                             MODERATOR_TO_SAMPLE_IN_M, label='Multi-Grid',
                             color='blue', run='', interval=[0, 50], weights=mg_weights_V)
cmnplt.energy_histogram_plot(he3_tof_V, he3_sd_d_V, number_bins_E,
                             MODERATOR_TO_SAMPLE_IN_M, label='Helium-3 tubes',
                             color='red', run='', interval=[0, 50], linestyle='--', weights=he3_weights_V)
plt.legend(title='Detector')
plt.ylabel('counts/s/sr')
plt.xlim(0, 50)
plt.tight_layout()
plt.savefig('%s/../output/v_energy_histogram.png' % nb_path, dpi=600)
plt.show()

#### Compare energy transfer spectra between Multi-Grid detector and Helium-3 tubes

First, we calculate the energies of the Multi-Grid and Helium-3 events and extract approximate positions of where the peaks are located in energy by using the Helium-3 data.

In [ ]:
# Calculate energies in Helium-3 and Multi-Grid detector data
energies_he3_V = e_calc.get_energy(he3_tof_V, he3_sd_d_V, MODERATOR_TO_SAMPLE_IN_M)
energies_mg_V = e_calc.get_energy(mg_tof_V, mg_sd_d_V, MODERATOR_TO_SAMPLE_IN_M)
# Extract peak positions
prominence = 1e5
hist, bin_edges = np.histogram(energies_he3_V, bins=1000, range=[0, 50])
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
peak_idxs = sci_sig.find_peaks(hist, prominence=prominence)[0]
#Eis_in_meV = bin_centers[peak_idxs]
#print(Eis_in_meV)

Next, we iterate through each peak and compare the Multi-Grid data with the Helium-3 tubes.

In [ ]:
# Declare number of bins for energy transfer plots
number_bins_dE = 50
# Iterate through all peaks
for i, (Ei_in_meV, tof_range) in enumerate(zip(Eis_in_meV, TOF_RANGES)):
    # Extract tof limits
    tof_min, tof_max = tof_range[0], tof_range[1]
    # Get peak energies at high resolution
    Ei_in_meV_he3_V = get_peak_energy_high_resolution(Ei_in_meV, energies_he3_V)
    Ei_in_meV_mg_V = get_peak_energy_high_resolution(Ei_in_meV, energies_mg_V)
    # Filter on tof
    mg_clu_peak_V = mg_clu_f_V[((mg_clu_f_V.tof * 62.5e-9 * 1e6) >= tof_min) &
                               ((mg_clu_f_V.tof * 62.5e-9 * 1e6) <= tof_max)]
    he3_ev_peak_V = he3_ev_f_V[(he3_ev_f_V.tof >= tof_min) &
                               (he3_ev_f_V.tof <= tof_max)]
    # Extract tof and distance
    mg_tof_peak_V = mg_clu_peak_V['tof'] * 62.5e-9 * 1e6
    mg_sd_d_peak_V = dclb.get_sample_to_detection_distances(mg_clu_peak_V['wch'],
                                                            mg_clu_peak_V['gch'],
                                                            MG_OFFSET, MG_THETA)
    he3_tof_peak_V = he3_ev_peak_V['tof']
    he3_sd_d_peak_V = he3_mapping_V['r'][he3_ev_peak_V['pixel_id']]
    # Plot energy transfer
    factor = 1
    fig = plt.figure()
    cmnplt.energy_transfer_plot(Ei_in_meV_mg_V, mg_tof_peak_V, mg_sd_d_peak_V,
                                MODERATOR_TO_SAMPLE_IN_M, number_bins_dE,
                                fig, Ei_in_meV, label='Multi-Grid', color='blue', normalize_by_counts=False,
                                normalize_by_max=True, linestyle='solid', factor=factor, marker='o', markerfacecolor='none', include_hist=True)
    cmnplt.energy_transfer_plot(Ei_in_meV_he3_V, he3_tof_peak_V,
                                he3_sd_d_peak_V, MODERATOR_TO_SAMPLE_IN_M,
                                number_bins_dE, fig, Ei_in_meV, label='Helium-3 tubes', normalize_by_counts=False,
                                color='red', normalize_by_max=True, linestyle='-', factor=factor, markerfacecolor='r', include_hist=True)
    plt.ylabel('Normalized counts (counts*(1/max(hist)))')
    #plt.xlim(-2, 2)
    plt.ylim(1e-5, 1)
    plt.legend(title='Detector')
    plt.show()

After that, we combine all of the individual energy transfer plots into one large plot. We start by applying wide binning.

In [ ]:
def plot_hist_normed(array, ax, min_val, max_val, color, label, linestyle, number_bins, include_hist=True):
    hist, bins = np.histogram(array, range=[min_val, max_val], bins=number_bins)
    idxs_one = np.where(hist == 1)
    norm = 1/max(hist)
    bin_centers = (bins[:-1] + bins[1:]) / 2
    if include_hist:
        ax.hist(array, range=[min_val, max_val], bins=number_bins, weights=norm*np.ones(len(array)),
                color=color, histtype='step', linestyle=linestyle)
    hist, bins = np.histogram(array, range=[min_val, max_val], bins=number_bins, weights=norm*np.ones(len(array)))
    bin_centers = (bins[:-1] + bins[1:]) / 2
    # Extract error bars
    errors_up = np.sqrt(hist/(norm))*(norm)
    errors_down = np.sqrt(hist/(norm))*(norm)
    errors_down[idxs_one] = 0
    ax.errorbar(bin_centers, hist, (errors_down, errors_up), color=color, label=label, linestyle='', #capsize=3,
                marker='.')
    ax.set_ylim(1e-5, 1)
    ax.set_yscale('log')
    ax.set_yticks([1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5])
    ax.set_yticklabels(['1', '1e-1', '1e-2', '1e-3', '1e-4', '1e-5'])
    bin_width = bins[1] - bins[0]
    ax.grid(True, which='major', linestyle='dotted', zorder=0)
    return bin_width

# Declare figure
#((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8), (ax9, ax10))
fig, axes = plt.subplots(nrows=5, ncols=2, sharey=True, sharex=False)
# Declare number of bins for energy transfer plots
number_bins_dE_left = 150
number_bins_dE_right = 30
energy_labels = ['2.4 meV (5.8 Å)', '3.7 meV (4.7 Å)', '6.3 meV (3.6 Å)', '13.4 meV (2.5 Å)', '44.9 meV (1.3 Å)']
# Iterate through all peaks
bin_widths_left = []
bin_widths_right = []
for i, (Ei_in_meV, tof_range, ax_pair, energy_label) in enumerate(zip(Eis_in_meV, TOF_RANGES, axes, energy_labels)):
    # Extract tof limits
    tof_min, tof_max = tof_range[0], tof_range[1]
    # Get peak energies at high resolution
    Ei_in_meV_he3_V = get_peak_energy_high_resolution(Ei_in_meV, energies_he3_V)
    Ei_in_meV_mg_V = get_peak_energy_high_resolution(Ei_in_meV, energies_mg_V)
    # Filter on tof
    mg_clu_peak_V = mg_clu_f_V[((mg_clu_f_V.tof * 62.5e-9 * 1e6) >= tof_min) &
                               ((mg_clu_f_V.tof * 62.5e-9 * 1e6) <= tof_max)]
    he3_ev_peak_V = he3_ev_f_V[(he3_ev_f_V.tof >= tof_min) &
                               (he3_ev_f_V.tof <= tof_max)]
    # Extract tof and distance
    mg_tof_peak_V = mg_clu_peak_V['tof'] * 62.5e-9 * 1e6
    mg_sd_d_peak_V = dclb.get_sample_to_detection_distances(mg_clu_peak_V['wch'],
                                                            mg_clu_peak_V['gch'],
                                                            MG_OFFSET, MG_THETA)
    he3_tof_peak_V = he3_ev_peak_V['tof']
    he3_sd_d_peak_V = he3_mapping_V['r'][he3_ev_peak_V['pixel_id']]
    # Extract dE
    delta_E_mg = e_calc.get_energy_transfer(Ei_in_meV_mg_V, mg_tof_peak_V,
                                            mg_sd_d_peak_V,
                                            MODERATOR_TO_SAMPLE_IN_M)
    delta_E_he3 = e_calc.get_energy_transfer(Ei_in_meV_he3_V, he3_tof_peak_V,
                                             he3_sd_d_peak_V,
                                             MODERATOR_TO_SAMPLE_IN_M)
    # Histogram data
    min_val, max_val = -Ei_in_meV, Ei_in_meV
    bin_width_left = plot_hist_normed(delta_E_mg, ax_pair[0], min_val, max_val, 'royalblue', 'Multi-Grid', '-', number_bins_dE_left)
    plot_hist_normed(delta_E_he3, ax_pair[0], min_val, max_val, 'indianred', '$^3$He-tubes', '--', number_bins_dE_left)
    bin_widths_left.append(bin_width_left)
    ax_pair[0].set_xlim(-0.2*Ei_in_meV, 0.2*Ei_in_meV)
    ax_pair[0].set_ylabel('%s\nIntensity' % energy_label)
    bin_width_right = plot_hist_normed(delta_E_mg, ax_pair[1], min_val, max_val, 'royalblue', 'Multi-Grid', '-', number_bins_dE_right)
    plot_hist_normed(delta_E_he3, ax_pair[1], min_val, max_val, 'indianred', '$^3$He-tubes', '--', number_bins_dE_right)
    ax_pair[1].set_xlim(-Ei_in_meV, Ei_in_meV)
    bin_widths_right.append(bin_width_right)
    
plt.subplots_adjust(wspace=.0)
plt.subplots_adjust(hspace=.2)
# Declare legend
legend_1 = [Line2D([0], [0], color='royalblue', label='Multi-Grid detector', linestyle='-'),
            Line2D([0], [0], color='indianred', label='Helium-3 tubes', linestyle='--')]
fig.legend(bbox_to_anchor=(0.83, 0.91), handles=legend_1, ncol=2)
# Declare energies
#center, start, step = 0.45, 0.86, 0.149
center, start, step = 0.16, 0.870, 0.147
bin_box_offset_x, bin_box_offset_y = 0.04, 0.083
labels_locations = [[center, start], [center, start], [center, 0.6], [center, 0.4], [center, 0.2]]
for i, (energy_label, bin_width_left, bin_width_right) in enumerate(zip(energy_labels, bin_widths_left, bin_widths_right)):
    x_val, y_val = center, start - i*step
    fig.text(x_val, y_val, '\nBin width:\n%.3f meV' % bin_width_left, verticalalignment='center')
    fig.text(x_val+0.4, y_val, '\nBin width:\n%.3f meV' % bin_width_right, verticalalignment='center')
# Set axis labels

axes[-1][0].set_xlabel('E$_i$ - E$_f$ (meV)')
axes[-1][1].set_xlabel('E$_i$ - E$_f$ (meV)')
fig.set_figwidth(5)
fig.set_figheight(10)
fig.suptitle('Vanadium measurements - wide binning', x=0.55, y=0.92)
plt.savefig('%s/../output/vanadium_energy_transfer_wide_binning.png' % nb_path, dpi=800)

After that we do the same plot, but with fine binning instead.

In [ ]:
# Declare figure
fig, axes = plt.subplots(nrows=5, ncols=2, sharey=True, sharex=False)
# Declare number of bins for energy transfer plots
number_bins_dE = 1000
energy_labels = ['2.4 meV (5.8 Å)', '3.7 meV (4.7 Å)', '6.3 meV (3.6 Å)', '13.4 meV (2.5 Å)', '44.9 meV (1.3 Å)']
# Iterate through all peaks
bin_widths = []
for i, (Ei_in_meV, tof_range, ax_pair, energy_label) in enumerate(zip(Eis_in_meV, TOF_RANGES, axes, energy_labels)):
    # Extract tof limits
    tof_min, tof_max = tof_range[0], tof_range[1]
    # Get peak energies at high resolution
    Ei_in_meV_he3_V = get_peak_energy_high_resolution(Ei_in_meV, energies_he3_V)
    Ei_in_meV_mg_V = get_peak_energy_high_resolution(Ei_in_meV, energies_mg_V)
    # Filter on tof
    mg_clu_peak_V = mg_clu_f_V[((mg_clu_f_V.tof * 62.5e-9 * 1e6) >= tof_min) &
                               ((mg_clu_f_V.tof * 62.5e-9 * 1e6) <= tof_max)]
    he3_ev_peak_V = he3_ev_f_V[(he3_ev_f_V.tof >= tof_min) &
                               (he3_ev_f_V.tof <= tof_max)]
    # Extract tof and distance
    mg_tof_peak_V = mg_clu_peak_V['tof'] * 62.5e-9 * 1e6
    mg_sd_d_peak_V = dclb.get_sample_to_detection_distances(mg_clu_peak_V['wch'],
                                                            mg_clu_peak_V['gch'],
                                                            MG_OFFSET, MG_THETA)
    he3_tof_peak_V = he3_ev_peak_V['tof']
    he3_sd_d_peak_V = he3_mapping_V['r'][he3_ev_peak_V['pixel_id']]
    # Extract dE
    delta_E_mg = e_calc.get_energy_transfer(Ei_in_meV_mg_V, mg_tof_peak_V,
                                            mg_sd_d_peak_V,
                                            MODERATOR_TO_SAMPLE_IN_M)
    delta_E_he3 = e_calc.get_energy_transfer(Ei_in_meV_he3_V, he3_tof_peak_V,
                                             he3_sd_d_peak_V,
                                             MODERATOR_TO_SAMPLE_IN_M)
    # Histogram data
    min_val, max_val = -Ei_in_meV, Ei_in_meV
    bin_width = plot_hist_normed(delta_E_mg, ax_pair[0], min_val, max_val, 'royalblue', 'Multi-Grid', '-', number_bins_dE, include_hist=False)
    plot_hist_normed(delta_E_he3, ax_pair[0], min_val, max_val, 'indianred', '$^3$He-tubes', '--', number_bins_dE, include_hist=False)
    ax_pair[0].set_xlim(-0.2*Ei_in_meV, 0.2*Ei_in_meV)
    ax_pair[0].set_ylabel('%s\nIntensity' % energy_label)
    plot_hist_normed(delta_E_mg, ax_pair[1], min_val, max_val, 'royalblue', 'Multi-Grid', '-', number_bins_dE, include_hist=False)
    plot_hist_normed(delta_E_he3, ax_pair[1], min_val, max_val, 'indianred', '$^3$He-tubes', '--', number_bins_dE, include_hist=False)
    ax_pair[1].set_xlim(-Ei_in_meV, Ei_in_meV)
    bin_widths.append(bin_width)
    
plt.subplots_adjust(wspace=.0)
plt.subplots_adjust(hspace=.2)
# Declare legend
legend_1 = [Line2D([0], [0], color='royalblue', label='Multi-Grid detector', linestyle='-'),
            Line2D([0], [0], color='indianred', label='Helium-3 tubes', linestyle='--')]
fig.legend(bbox_to_anchor=(0.83, 0.91), handles=legend_1, ncol=2)
# Declare energies
#center, start, step = 0.45, 0.86, 0.149
center, start, step = 0.16, 0.870, 0.147
bin_box_offset_x, bin_box_offset_y = 0.04, 0.083
labels_locations = [[center, start], [center, start], [center, 0.6], [center, 0.4], [center, 0.2]]
for i, (energy_label, bin_width) in enumerate(zip(energy_labels, bin_widths)):
    x_val, y_val = center, start - i*step
    fig.text(x_val, y_val, '\nBin width:\n%.3f meV' % bin_width, verticalalignment='center')
    fig.text(x_val+0.4, y_val, '\nBin width:\n%.3f meV' % bin_width, verticalalignment='center')
# Set axis labels

axes[-1][0].set_xlabel('E$_i$ - E$_f$ (meV)')
axes[-1][1].set_xlabel('E$_i$ - E$_f$ (meV)')
fig.set_figwidth(5)
fig.set_figheight(10)
fig.suptitle('Vanadium measurements - fine binning', x=0.55, y=0.92)
plt.savefig('%s/../output/vanadium_energy_transfer_fine_binning.png' % nb_path, dpi=800)

Next, we compare the Helium-3 data from three different regions and see if we notice any difference between them.

In [ ]:
# Declare all Helium-3 regions
he3_ev_f_V_semi_full = he3_ev_V[he3_ev_V['pixel_id'].isin(region_semi_full)]
he3_ev_f_V_MG_region = he3_ev_V[he3_ev_V['pixel_id'].isin(region_of_interest)]
he3_ev_f_V_region_around_MG = he3_ev_V[he3_ev_V['pixel_id'].isin(region_around_MG)]
# Declare figure
fig, axes = plt.subplots(nrows=5, ncols=2, sharey=True, sharex=False)
# Declare number of bins for energy transfer plots
number_bins_dE_left = 150
number_bins_dE_right = 30
energy_labels = ['2.4 meV (5.8 Å)', '3.7 meV (4.7 Å)', '6.3 meV (3.6 Å)', '13.4 meV (2.5 Å)', '44.9 meV (1.3 Å)']
# Iterate through all peaks
bin_widths_left = []
bin_widths_right = []
for i, (Ei_in_meV, tof_range, ax_pair, energy_label) in enumerate(zip(Eis_in_meV, TOF_RANGES, axes, energy_labels)):
    # Extract tof limits
    tof_min, tof_max = tof_range[0], tof_range[1]
    # Get peak energies at high resolution
    Ei_in_meV_he3_V = get_peak_energy_high_resolution(Ei_in_meV, energies_he3_V)
    Ei_in_meV_mg_V = get_peak_energy_high_resolution(Ei_in_meV, energies_mg_V)
    # Filter on tof
    mg_clu_peak_V = mg_clu_f_V[((mg_clu_f_V.tof * 62.5e-9 * 1e6) >= tof_min) &
                               ((mg_clu_f_V.tof * 62.5e-9 * 1e6) <= tof_max)]
    he3_ev_peak_V = he3_ev_f_V[(he3_ev_f_V.tof >= tof_min) &
                               (he3_ev_f_V.tof <= tof_max)]
    he3_ev_peak_V_MG_region = he3_ev_f_V_MG_region[(he3_ev_f_V_MG_region.tof >= tof_min) &
                                                   (he3_ev_f_V_MG_region.tof <= tof_max)]
    he3_ev_peak_V_region_around_MG = he3_ev_f_V_region_around_MG[(he3_ev_f_V_region_around_MG.tof >= tof_min) &
                                                                 (he3_ev_f_V_region_around_MG.tof <= tof_max)]
    # Extract tof and distance
    mg_tof_peak_V = mg_clu_peak_V['tof'] * 62.5e-9 * 1e6
    mg_sd_d_peak_V = dclb.get_sample_to_detection_distances(mg_clu_peak_V['wch'],
                                                            mg_clu_peak_V['gch'],
                                                            MG_OFFSET, MG_THETA)
    he3_tof_peak_V = he3_ev_peak_V['tof']
    he3_sd_d_peak_V = he3_mapping_V['r'][he3_ev_peak_V['pixel_id']]
    he3_tof_peak_V_MG_region = he3_ev_peak_V_MG_region['tof']
    he3_sd_d_peak_V_MG_region = he3_mapping_V['r'][he3_ev_peak_V_MG_region['pixel_id']]
    he3_tof_peak_V_region_around_MG = he3_ev_peak_V_region_around_MG['tof']
    he3_sd_d_peak_V_region_around_MG = he3_mapping_V['r'][he3_ev_peak_V_region_around_MG['pixel_id']]
    # Extract dE
    delta_E_mg = e_calc.get_energy_transfer(Ei_in_meV_mg_V, mg_tof_peak_V,
                                            mg_sd_d_peak_V,
                                            MODERATOR_TO_SAMPLE_IN_M)
    delta_E_he3 = e_calc.get_energy_transfer(Ei_in_meV_he3_V, he3_tof_peak_V,
                                             he3_sd_d_peak_V,
                                             MODERATOR_TO_SAMPLE_IN_M)
    delta_E_he3_MG_region = e_calc.get_energy_transfer(Ei_in_meV_he3_V, he3_tof_peak_V_MG_region,
                                                       he3_sd_d_peak_V_MG_region,
                                                       MODERATOR_TO_SAMPLE_IN_M)
    delta_E_he3_region_around_MG = e_calc.get_energy_transfer(Ei_in_meV_he3_V, he3_tof_peak_V_region_around_MG,
                                                              he3_sd_d_peak_V_region_around_MG,
                                                              MODERATOR_TO_SAMPLE_IN_M)
    # Histogram data
    min_val, max_val = -Ei_in_meV, Ei_in_meV
    bin_width_left = plot_hist_normed(delta_E_mg, ax_pair[0], min_val, max_val, 'royalblue', 'Multi-Grid', '-', number_bins_dE_left)
    plot_hist_normed(delta_E_he3, ax_pair[0], min_val, max_val, 'indianred', '$^3$He-tubes', '--', number_bins_dE_left)
    plot_hist_normed(delta_E_he3_MG_region, ax_pair[0], min_val, max_val, 'mediumseagreen', '$^3$He-tubes', 'dotted', number_bins_dE_left)
    plot_hist_normed(delta_E_he3_region_around_MG, ax_pair[0], min_val, max_val, 'orchid', '$^3$He-tubes', '-.', number_bins_dE_left)
    bin_widths_left.append(bin_width_left)
    ax_pair[0].set_xlim(-0.2*Ei_in_meV, 0.2*Ei_in_meV)
    ax_pair[0].set_ylabel('%s\nIntensity' % energy_label)
    bin_width_right = plot_hist_normed(delta_E_mg, ax_pair[1], min_val, max_val, 'royalblue', 'Multi-Grid', '-', number_bins_dE_right)
    plot_hist_normed(delta_E_he3, ax_pair[1], min_val, max_val, 'indianred', '$^3$He-tubes', '--', number_bins_dE_right)
    plot_hist_normed(delta_E_he3_MG_region, ax_pair[1], min_val, max_val, 'mediumseagreen', '$^3$He-tubes', 'dotted', number_bins_dE_right)
    plot_hist_normed(delta_E_he3_region_around_MG, ax_pair[1], min_val, max_val, 'orchid', '$^3$He-tubes', '-.', number_bins_dE_right)
    ax_pair[1].set_xlim(-Ei_in_meV, Ei_in_meV)
    bin_widths_right.append(bin_width_right)
    
plt.subplots_adjust(wspace=.0)
plt.subplots_adjust(hspace=.2)
# Declare legend
legend_1 = [Line2D([0], [0], color='royalblue', label='Multi-Grid detector', linestyle='-')]
legend_2 = [Line2D([0], [0], color='indianred', label='Full array', linestyle='--'),
            Line2D([0], [0], color='mediumseagreen', label='Region 1', linestyle='dotted'),
            Line2D([0], [0], color='orchid', label='Region 2', linestyle='-.')]
fig.legend(bbox_to_anchor=(0.70, 0.97), handles=legend_1, ncol=1)
fig.legend(bbox_to_anchor=(0.85, 0.94), handles=legend_2, ncol=3, title='Helium-3 tubes')
# Declare energies
#center, start, step = 0.45, 0.86, 0.149
center, start, step = 0.16, 0.870, 0.147
bin_box_offset_x, bin_box_offset_y = 0.04, 0.083
labels_locations = [[center, start], [center, start], [center, 0.6], [center, 0.4], [center, 0.2]]
for i, (energy_label, bin_width_left, bin_width_right) in enumerate(zip(energy_labels, bin_widths_left, bin_widths_right)):
    x_val, y_val = center, start - i*step
    fig.text(x_val, y_val, '\nBin width:\n%.3f meV' % bin_width_left, verticalalignment='center')
    fig.text(x_val+0.4, y_val, '\nBin width:\n%.3f meV' % bin_width_right, verticalalignment='center')
# Set axis labels

axes[-1][0].set_xlabel('E$_i$ - E$_f$ (meV)')
axes[-1][1].set_xlabel('E$_i$ - E$_f$ (meV)')
fig.set_figwidth(5)
fig.set_figheight(10)
fig.suptitle('Vanadium measurements - region comparisons', x=0.55, y=0.99)
plt.savefig('%s/../output/vanadium_energy_transfer_comparison_between_regions.png' % nb_path, dpi=800)

#### Extracting beam monitor counts

First, we extract the beam monitor data.

In [ ]:
# Get beam monitor data
bm_path = HE3_FOLDER + he3_file_names_V[0] + '.nxs'
bm_ev_V = bm_read.import_data(he3_path)
for i, he3_file_name in enumerate(he3_file_names_V[1:]):
    bm_path = HE3_FOLDER + he3_file_name + '.nxs'
    bm_ev_temp = bm_read.import_data(he3_path)
    bm_ev_V = bm_ev_V + bm_ev_temp

Then, we plot the data.

In [ ]:
bm_basic_plot.bm_plot_basic(bm_ev_V, 100, 'Vanadium, RRM')

After that, we define delimiters to normalize each peak with. This is based on data from monitor 5.

In [ ]:
# Declare tof ranges colors
colors = ['blue', 'red', 'green', 'purple', 'orange']
fig = plt.figure()
df_bm_5 = bm_ev_V.loc[5]
df_bm_5_tofs = df_bm_5.index.values
weights_bm_5 = df_bm_5.values
hist, bins, __ = plt.hist(df_bm_5.index, weights=weights_bm_5,
                          zorder=5, label='Beam monitor data',
                          histtype='step', range=[0, 100000],
                          bins=500, color='black')
# Extra beam monitor normalization
beam_monitor_norm_V = []
for tof_range, color in zip(TOF_RANGES_BM, colors):
    indices = (df_bm_5_tofs > tof_range[0]) & (df_bm_5_tofs < tof_range[1])
    counts_within_range = sum(weights_bm_5[indices])
    beam_monitor_norm_V.append(counts_within_range)
    plt.axvline(x=tof_range[0], color=color, label='Counts: %d' % counts_within_range)
    plt.axvline(x=tof_range[1], color=color, label=None)
plt.xlim(0, 1e5)
plt.title('Beam monitor data - Vanadium')
plt.legend()
plt.yscale('log')

#### Examining circumference dependence in Helium-3 array

In [ ]:
plt.close()
# Declare parameters
factor = 1
number_bins_dE = 100
colors = ['blue', 'red', 'green', 'orange', 'purple']
animation_folder = nb_path + '/../output/animation/'
shutil.rmtree(animation_folder, ignore_errors=True)
hf.mkdir_p(animation_folder)
# Declare animation steps
start = 0
stop = 384
step = 8
delimiters = np.arange(start, stop, step) 
# Iterate through data
for delimiter in tqdm.tqdm(delimiters):
    # Declare pixels to use
    pixels = np.array([])
    tubes = np.arange(delimiter, delimiter+step, 1)
    for tube in tubes:
        pixels_tube = np.arange(0, 256, 1) + tube*256
        pixels = np.concatenate((pixels, pixels_tube))
    # Extract Helium-3 data from pixels
    he3_ev_f_V_pixels = he3_ev_f_V[he3_ev_f_V['pixel_id'].isin(pixels)]
    # Extract tofs, distances and energies for each event
    he3_tof_V_pixels = he3_ev_f_V_pixels['tof']
    he3_sd_d_V_pixels = he3_mapping_V['r'][he3_ev_f_V_pixels['pixel_id']]
    energies_he3_V_pixels = e_calc.get_energy(he3_tof_V_pixels, he3_sd_d_V_pixels, MODERATOR_TO_SAMPLE_IN_M)
    # Plot energy transfer histograms
    fig = plt.figure()
    for i, (Ei_in_meV, tof_range) in enumerate(zip(Eis_in_meV, TOF_RANGES)):
        # Extract tof limits
        tof_min, tof_max = tof_range[0], tof_range[1]
        # Get peak energies at high resolution
        Ei_in_meV_he3_V_pixels = get_peak_energy_high_resolution(Ei_in_meV, energies_he3_V_pixels)
        # Filter on tof
        he3_ev_peak_V_pixels = he3_ev_f_V_pixels[(he3_ev_f_V_pixels.tof >= tof_min) &
                                                 (he3_ev_f_V_pixels.tof <= tof_max)]
        # Extract tof and distance
        he3_tof_peak_V_pixels = he3_ev_peak_V_pixels['tof']
        he3_sd_d_peak_V_pixels = he3_mapping_V['r'][he3_ev_peak_V_pixels['pixel_id']]
        # Plot energy transfer
        plt.subplot(2, 3, i+2)
        cmnplt.energy_transfer_plot(Ei_in_meV_he3_V_pixels, he3_tof_peak_V_pixels,
                                    he3_sd_d_peak_V_pixels, MODERATOR_TO_SAMPLE_IN_M,
                                    number_bins_dE, fig, Ei_in_meV, color=colors[i], normalize_by_max=True, include_hist=True,
                                    linestyle='-', factor=factor)
        #plt.ylabel('counts*(1/sum(counts))')
        plt.ylabel('counts*(1/max(hist))')
        plt.title('%.2f meV (%.2f Å)' % (Ei_in_meV_he3_V_pixels, e_calc.meV_to_A(Ei_in_meV_he3_V_pixels)))
        #plt.ylim(1e-5, 1)
        plt.ylim(1e-5, 1)
    # Visualize which tubes are being probed
    plt.subplot(2, 3, 1)
    plt.title('Helium-3 array, top view ')
    plt.plot(zs_av, xs_av, marker='.', linestyle='', color='black', markersize=1)
    plt.plot([z_r1, z_r2, z_r3, z_r4, z_r1], [x_r1, x_r2, x_r3, x_r4, x_r1], color='black')
    for tube in tubes:
        plt.plot([0, zs_av[tube]], [0, xs_av[tube]], color='red')
    plt.gca().set_aspect('equal')
    plt.xlim(-4, 4)
    plt.ylim(-4, 4)
    plt.xlabel('z (m)')
    plt.ylabel('x (m)')
    # Save data
    output_path = animation_folder + str(tube) + '.png'
    fig.set_figwidth(14)
    fig.set_figheight(10)
    plt.tight_layout()
    fig.savefig(output_path, bbox_inches='tight')
    plt.close()

# Animate
output_path = '../output/circumference_sweep.gif'
images = []
files = os.listdir(animation_folder)
files = [file[:-4] for file in files if file[-9:] != '.DS_Store' and file != '.gitignore']
for file in sorted(files, key=int):
    images.append(imageio.imread(animation_folder + file + '.png'))
imageio.mimsave(output_path, images)
shutil.rmtree(animation_folder, ignore_errors=True)

#### Examining height dependence in Helium-3 array

In [ ]:
plt.close()
# Declare parameters
factor = 1
number_bins_dE = 100
colors = ['blue', 'red', 'green', 'orange', 'purple']
animation_folder = nb_path + '/../output/animation/'
shutil.rmtree(animation_folder, ignore_errors=True)
hf.mkdir_p(animation_folder)
# Declare animation steps
start = 0
stop = 256
step = 8
delimiters = np.arange(start, stop, step) 
tubes = np.arange(0, 384, 1)
# Iterate through data
for delimiter in tqdm.tqdm(delimiters):
    # Declare pixels to use
    pixels = np.array([])
    for tube in tubes:
        pixels_layer = np.arange(delimiter, delimiter+step, 1) + tube*256
        pixels = np.concatenate((pixels, pixels_layer))
    # Extract Helium-3 data from pixels
    he3_ev_f_V_pixels = he3_ev_f_V[he3_ev_f_V['pixel_id'].isin(pixels)]
    # Extract tofs, distances and energies for each event
    he3_tof_V_pixels = he3_ev_f_V_pixels['tof']
    he3_sd_d_V_pixels = he3_mapping_V['r'][he3_ev_f_V_pixels['pixel_id']]
    energies_he3_V_pixels = e_calc.get_energy(he3_tof_V_pixels, he3_sd_d_V_pixels, MODERATOR_TO_SAMPLE_IN_M)
    # Plot energy transfer histograms
    fig = plt.figure()
    for i, (Ei_in_meV, tof_range) in enumerate(zip(Eis_in_meV, TOF_RANGES)):
        # Extract tof limits
        tof_min, tof_max = tof_range[0], tof_range[1]
        # Get peak energies at high resolution
        Ei_in_meV_he3_V_pixels = get_peak_energy_high_resolution(Ei_in_meV, energies_he3_V_pixels)
        # Filter on tof
        he3_ev_peak_V_pixels = he3_ev_f_V_pixels[(he3_ev_f_V_pixels.tof >= tof_min) &
                                                 (he3_ev_f_V_pixels.tof <= tof_max)]
        # Extract tof and distance
        he3_tof_peak_V_pixels = he3_ev_peak_V_pixels['tof']
        he3_sd_d_peak_V_pixels = he3_mapping_V['r'][he3_ev_peak_V_pixels['pixel_id']]
        # Plot energy transfer
        plt.subplot(2, 3, i+2)
        cmnplt.energy_transfer_plot(Ei_in_meV_he3_V_pixels, he3_tof_peak_V_pixels,
                                    he3_sd_d_peak_V_pixels, MODERATOR_TO_SAMPLE_IN_M,
                                    number_bins_dE, fig, Ei_in_meV, color=colors[i], normalize_by_max=True, include_hist=True,
                                    linestyle='-', factor=factor)
        #plt.ylabel('counts*(1/sum(counts))')
        plt.ylabel('counts*(1/max(hist))')
        plt.title('%.2f meV (%.2f Å)' % (Ei_in_meV_he3_V_pixels, e_calc.meV_to_A(Ei_in_meV_he3_V_pixels)))
        #plt.ylim(1e-5, 1)
        plt.ylim(1e-5, 1)
    # Visualize which tubes are being probed
    plt.subplot(2, 3, 1)
    plt.title('Helium-3 array, side view ')
    plt.plot(zs_chunks[320], ys_chunks[320], marker='.', linestyle='', color='black', markersize=1)
    for pixel in np.arange(delimiter, delimiter+step, 1):
        plt.plot([0, zs_av[320]], [0, ys_chunks[320][pixel]], color='red')
    plt.gca().set_aspect('equal')
    plt.xlim(-3.5, 0)
    plt.ylim(-2.5, 2.5)
    plt.xlabel('distance (m)')
    plt.ylabel('y (m)')
    # Save data
    output_path = animation_folder + str(delimiter) + '.png'
    fig.set_figwidth(14)
    fig.set_figheight(10)
    plt.tight_layout()
    fig.savefig(output_path, bbox_inches='tight')
    plt.close()

# Animate
output_path = '../output/height_sweep.gif'
images = []
files = os.listdir(animation_folder)
files = [file[:-4] for file in files if file[-9:] != '.DS_Store' and file != '.gitignore']
for file in sorted(files, key=int):
    images.append(imageio.imread(animation_folder + file + '.png'))
imageio.mimsave(output_path, images)
shutil.rmtree(animation_folder, ignore_errors=True)

#### Plot 3D hist of vanadium

First, we plot the full 3D histogram collected with Vanadium.

In [ ]:
he3_basic_plot.he3_plot_3D_histogram(he3_ev_V, he3_mapping_V)

After that, we look at the data from all different peaks by using the tof-delimiters.

In [ ]:
# Declare necessary parameters
energy_labels = ['2.4 meV (5.8 Å)', '3.7 meV (4.7 Å)', '6.3 meV (3.6 Å)', '13.4 meV (2.5 Å)', '44.9 meV (1.3 Å)']
for i in np.arange(0, 5, 1):
    tof_range_peak = TOF_RANGES[i]
    Ei_in_meV = Eis_in_meV[i]
    tof_min, tof_max = tof_range_peak[0], tof_range_peak[1]
    he3_ev_peak_V = he3_ev_V[(he3_ev_V.tof >= tof_min) &
                             (he3_ev_V.tof <= tof_max)]
    he3_basic_plot.he3_plot_3D_histogram(he3_ev_peak_V[he3_ev_peak_V['pixel_id'].isin(region_full)], he3_mapping_V, label='Vanadium<br>%s' % energy_labels[i], file_name='Vanadium_%d_meV' % Ei_in_meV)

After that, we create a scan over all peaks to see where the different features are appearing.

In [ ]:
he3_ev_V_semi = he3_ev_V[he3_ev_V['pixel_id'].isin(region_semi_full)]
# Declare number of bins for energy transfer plots
number_bins_dE = 500
factor = 1
number_slices = 20
bins_1D_plots = 30
bins_tof_and_d = 100
dE_percentage = 1
vmin, vmax = 1, 1e4
# Iterate through all peaks
for i, (Ei_in_meV, tof_range) in enumerate(zip(Eis_in_meV, TOF_RANGES)):
    # Extract tof limits
    tof_min, tof_max = tof_range[0], tof_range[1]
    # Get peak energy at high resolution
    Ei_in_meV_he3_V = get_peak_energy_high_resolution(Ei_in_meV, energies_he3_V)
    # Get neutron velocity corresponding to this energy
    vi_in_m_per_s_he3 = np.sqrt((Ei_in_meV_he3_V/5.227)) * 1e3 # Squires (p. 3)
    # Filter on tof
    he3_ev_peak_V = he3_ev_V_semi[(he3_ev_V_semi.tof >= tof_min) &
                                  (he3_ev_V_semi.tof <= tof_max)]
    # Extract tof and distance
    he3_tof_peak_V = he3_ev_peak_V['tof']
    he3_sd_d_peak_V = he3_mapping_V['r'][he3_ev_peak_V['pixel_id']]
    # Get energy transfer
    dE_he3 = e_calc.get_energy_transfer(Ei_in_meV_he3_V, he3_tof_peak_V, he3_sd_d_peak_V, MODERATOR_TO_SAMPLE_IN_M)
    # Go through energy transfer spectra and investigate where the different features are reconstructed
    dE_delimiters = np.linspace(-Ei_in_meV_he3_V*dE_percentage, Ei_in_meV_he3_V*dE_percentage, number_slices)
    step = dE_delimiters[1] - dE_delimiters[0]
    # Declare animation folder
    animation_folder = nb_path + '/../output/animation/'
    shutil.rmtree(animation_folder, ignore_errors=True)
    hf.mkdir_p(animation_folder)
    # Iterate through data
    for j, dE_delimiter in enumerate(tqdm.tqdm(dE_delimiters)):
        # Declare dE range
        dE_min = dE_delimiter
        dE_max = dE_delimiter + step
        # Extract events within dE
        indices_within_dE = (dE_he3 > dE_min) & (dE_he3 < dE_max)
        he3_ev_peak_V_within_dE = he3_ev_peak_V[indices_within_dE]
        # Plot
        fig = plt.figure()
        plt.suptitle('E$_i$=%.2f meV (%.2f Å)' % (Ei_in_meV, e_calc.meV_to_A(Ei_in_meV)))
        plt.subplot(3, 3, 1)
        cmnplt.energy_transfer_plot(Ei_in_meV_he3_V, he3_tof_peak_V,
                                    he3_sd_d_peak_V, MODERATOR_TO_SAMPLE_IN_M,
                                    number_bins_dE, fig, Ei_in_meV,
                                    color='black', linestyle='-', factor=factor)
        plt.ylabel('Counts')
        plt.title('Energy transfer')
        plt.axvspan(dE_min, dE_max, color='red', alpha=0.5)
        plt.subplot(3, 3, 2)
        plt.hist2d(he3_ev_peak_V_within_dE.pixel_id//256,
                   he3_mapping_V['y'][he3_ev_peak_V_within_dE.pixel_id],
                   bins=[bins_1D_plots, bins_1D_plots],
                   range=[[0, 384], [-2.5, 2.5]],
                   norm=LogNorm(vmin, vmax),
                   #vmin=0, vmax=70,
                   cmap='jet')
        plt.ylim(-2.5, 2.5)
        plt.xlim(0, 384)
        plt.title('Tube vs Height histogram')
        cbar = plt.colorbar(extend='max')
        cbar.set_label('Counts')
        plt.xlabel('Tube')
        plt.ylabel('Height (m)')
        plt.subplot(3, 3, 4)
        #hist, __, __  = plt.hist(he3_mapping_V['y'][he3_ev_peak_V.pixel_id], range=[-2.5, 2.5], bins=bins_1D_plots, color='black', histtype='step')
        #plt.ylim(1, max(hist)*2)
        hist, bins, __  = plt.hist(he3_mapping_V['y'][he3_ev_peak_V_within_dE.pixel_id], range=[-2.5, 2.5], bins=bins_1D_plots, color='red', histtype='step')
        bin_centers = (bins[:-1] + bins[1:]) / 2
        plt.errorbar(bin_centers, hist, np.sqrt(hist), color='red', linestyle='')
        plt.xlim(-2.5, 2.5)
        plt.grid(True, which='major', linestyle='--', zorder=0)
        plt.grid(True, which='minor', linestyle='--', zorder=0)
        plt.xlabel('Height (m)')
        plt.ylabel('Counts')
        plt.title('Height histogram')
        #plt.yscale('log')
        plt.yscale('log')
        plt.ylim(1, 1e5)
        plt.subplot(3, 3, 5)
        #hist, __, __  = plt.hist(he3_ev_peak_V.pixel_id//256, range=[0, 384], bins=bins_1D_plots, color='black', histtype='step')
        #plt.ylim(1, max(hist)*2)
        hist, bins, __  = plt.hist(he3_ev_peak_V_within_dE.pixel_id//256, range=[0, 384], bins=bins_1D_plots, color='red', histtype='step')
        bin_centers = (bins[:-1] + bins[1:]) / 2
        plt.errorbar(bin_centers, hist, np.sqrt(hist), color='red', linestyle='')
        plt.xlim(0, 384)
        plt.grid(True, which='major', linestyle='--', zorder=0)
        plt.grid(True, which='minor', linestyle='--', zorder=0)
        #plt.yscale('log')
        plt.xlabel('Tube')
        plt.ylabel('Counts')
        plt.title('Tube histogram')
        plt.yscale('log')
        plt.ylim(1, 1e5)
        plt.subplot(3, 3, 7)
        ds = he3_mapping_V['r'][he3_ev_peak_V.pixel_id]
        ds_within_dE = he3_mapping_V['r'][he3_ev_peak_V_within_dE.pixel_id]
        hist, __, __ = plt.hist(ds, range=[3.4, 4.2], bins=bins_tof_and_d, color='black', histtype='step')
        plt.ylim(1, max(hist)*2)
        hist, bins, __  = plt.hist(ds_within_dE, range=[3.4, 4.2],
                                   bins=bins_tof_and_d, color='red', histtype='step')
        plt.grid(True, which='major', linestyle='--', zorder=0)
        plt.grid(True, which='minor', linestyle='--', zorder=0)
        #plt.xlim(3, 5)
        bin_centers = (bins[:-1] + bins[1:]) / 2
        plt.errorbar(bin_centers, hist, np.sqrt(hist), color='red', linestyle='')
        plt.xlabel('Sample-to-detection (m)')
        plt.title('Sample-to-detection')
        plt.ylabel('Counts')
        plt.yscale('log')
        plt.subplot(3, 3, 8)
        tofs = he3_ev_peak_V.tof
        tofs_within_dE = he3_ev_peak_V_within_dE.tof
        hist, __, __ = plt.hist(tofs, range=[tof_min, tof_max],
                                bins=bins_tof_and_d, color='black', histtype='step')
        plt.ylim(1, max(hist)*2)
        hist, bins, __  = plt.hist(tofs_within_dE, range=[tof_min, tof_max],
                                   bins=bins_tof_and_d, color='red', histtype='step')
        bin_centers = (bins[:-1] + bins[1:]) / 2
        plt.grid(True, which='major', linestyle='--', zorder=0)
        plt.grid(True, which='minor', linestyle='--', zorder=0)
        plt.errorbar(bin_centers, hist, np.sqrt(hist), color='red', linestyle='')
        plt.title('Time-of-flight')
        #plt.xlim(3e4, 5e4)
        plt.xlabel('tof (µs)')
        plt.ylabel('Counts')
        plt.yscale('log')
        plt.subplot(3, 3, 3)
        d_extra = get_d_extra(tofs_within_dE, ds_within_dE, vi_in_m_per_s_he3, MODERATOR_TO_SAMPLE_IN_M)
        if len(d_extra) > 0:
            mu = (sum(d_extra)/len(d_extra))
            sigma = np.sqrt((1/len(d_extra)) * sum((d_extra - mu)**2))
            plt.hist(d_extra, bins=1000, color='red', range=[-2, 5],
                     label='%.2f m ± %.2f' % (mu, sigma), histtype='step')
            plt.grid(True, which='major', linestyle='--', zorder=0)
            plt.grid(True, which='minor', linestyle='--', zorder=0)
            plt.title('d$_{extra}$ = v$_i$ $\cdot$ tof$_{recorded}$ - d$_{recorded}$\nAverage extra distance %.2f m ± %.2f' % (mu, sigma))
            plt.xlabel('d$_{extra}$ (m)')
            plt.ylabel('Counts')
            plt.yscale('log')
        plt.ylim(1, 1e5)
        plt.xlim(-2, 5)
        plt.subplot(3, 3, 6)
        plt.title('tof vs d (full data)')
        plt.hist2d(tofs,
                   ds,
                   norm=LogNorm(1, 1e6),
                   bins=[500, 500],
                   cmap='jet')
        cbar = plt.colorbar()
        cbar.set_label('Counts')
        plt.xlabel('tof (µs)')
        plt.ylabel('Sample-to-detection (m)')
        if len(tofs_within_dE) > 0:
            plt.axvspan(min(tofs_within_dE), max(tofs_within_dE), color='red', alpha=0.5)
        #plt.subplot(3, 3, 9)
        #plt.title('tof vs d (only within dE slice)')
        #plt.hist2d(tofs_within_dE,
        #           ds_within_dE,
        #           bins=[100, 100],
        #           cmap='jet')
        #cbar = plt.colorbar()
        #cbar.set_label('Counts')
        #plt.xlabel('tof (µs)')
        #plt.ylabel('Sample-to-detection (m)')
        # Save data
        output_path = animation_folder + str(j) + '.png'
        fig.set_figwidth(14)
        fig.set_figheight(14)
        plt.tight_layout()
        fig.savefig(output_path, bbox_inches='tight')
        plt.close()
    # Animate
    output_path = '../output/dE_sweep_%.2f_meV.gif' % Ei_in_meV
    images = []
    files = os.listdir(animation_folder)
    files = [file[:-4] for file in files if file[-9:] != '.DS_Store' and file != '.gitignore']
    for file in sorted(files, key=int):
        images.append(imageio.imread(animation_folder + file + '.png'))
    imageio.mimsave(output_path, images)
    shutil.rmtree(animation_folder, ignore_errors=True)

#### Distances corresponding to energy discrepancies

First, we use the Helium-3 data to look at the two side-peaks.

In [ ]:
# Define lists of E values to look closer at
E_values_list = [None, None, [6.34, 6.12, 6.19, 5.8], [13.39, 12.83, 12.97, 12.29], None]
colors = ['red', 'blue', 'green', 'orange']
# Iterate through all peaks
for i, (Ei_in_meV, tof_range, E_values) in enumerate(zip(Eis_in_meV, TOF_RANGES, E_values_list)):
    # Get Ei in high resolution
    Ei_in_meV_he3_V = get_peak_energy_high_resolution(Ei_in_meV, energies_he3_V)
    # Extract velocity vi
    vi_in_m_per_s = np.sqrt((Ei_in_meV_he3_V/5.227)) * 1e3 # Squires (p. 3)
    #print(vi_in_m_per_s)
    # Extract tof limits
    tof_min, tof_max = tof_range[0], tof_range[1]
    # Filter on tof
    he3_ev_peak_V = he3_ev_f_V [(he3_ev_f_V.tof >= tof_min) & (he3_ev_f_V.tof <= tof_max)]
    # Extract tof and distance
    he3_tof_peak_V = he3_ev_peak_V['tof']
    he3_sd_d_peak_V = he3_mapping_V['r'][he3_ev_peak_V['pixel_id']]
    # Plot energy
    energies_he3_V_peak = e_calc.get_energy(he3_tof_peak_V, he3_sd_d_peak_V, MODERATOR_TO_SAMPLE_IN_M)
    sections = []
    d_extras = []
    if E_values is not None:
        for i, E_value in enumerate(E_values):
            # Print distances we get here
            E_min = E_value-0.002*E_value
            E_max = E_value+0.002*E_value
            indices = (energies_he3_V_peak > E_min) & (energies_he3_V_peak < E_max)
            tof_values = he3_tof_peak_V[indices]
            d_values = he3_sd_d_peak_V[indices]
            d_extra = get_d_extra(tof_values, d_values, vi_in_m_per_s, MODERATOR_TO_SAMPLE_IN_M)
            sections.append([E_min, E_max])
            d_extras.append(d_extra)
        fig = plt.figure()
        plt.subplot(1, 2, 1)
        plt.hist(energies_he3_V_peak, color='black', histtype='step', bins=500, label='Helium-3 data')
        for i, (section, E_value) in enumerate(zip(sections, E_values)):
            E_min, E_max = section
            plt.axvspan(E_min, E_max, color=colors[i], alpha=0.5, label='%.2f meV' % E_value)
        plt.grid(True, which='major', zorder=0)
        plt.grid(True, which='minor', linestyle='--', zorder=0)
        plt.yscale('log')
        plt.xlabel('Energy assuming elastic scattering (meV)')
        plt.ylabel('Counts')
        plt.suptitle('E$_i$=%.2f meV (%.2f Å)' % (Ei_in_meV, e_calc.meV_to_A(Ei_in_meV)))
        plt.title('Energy')
        plt.ylim(1, 1e7)
        plt.legend()
        plt.subplot(1, 2, 2)
        plt.xlabel('Extra distance travelled (m)')
        plt.ylabel('Counts')
        for i, (E_value, d_extra) in enumerate(zip(E_values, d_extras)):
            mu = (sum(d_extra)/len(d_extra))
            sigma = np.sqrt((1/len(d_extra)) * sum((d_extra - mu)**2))
            plt.hist(d_extra, bins=200, color=colors[i], range=[-0.5, 1.5],
                     label='%.2f m ± %.2f' % (mu, sigma), histtype='step')
        plt.title('v$_i$ * tof$_{recorded}$ - d$_{recorded}$')
        plt.grid(True, which='major', zorder=0)
        plt.grid(True, which='minor', linestyle='--', zorder=0)
        plt.legend(title='Average extra distance')
        plt.ylim(1, 1e7)
        plt.yscale('log')
        plt.xlim(-0.5, 1.5)
        fig.set_figwidth(10)
        fig.set_figheight(5)
        plt.tight_layout()
        plt.show()

After that, we shift our focus to energy transfer and look at the extra distance travelled by the neutrons in the shoulder seen in the Multi-Grid detector data.

In [ ]:
# Define lists of E values to look closer at
dE_values_list = [None, None, [0.4], [0.65], None]
colors = ['red', 'blue', 'green', 'orange']
# Iterate through all peaks
for i, (Ei_in_meV, tof_range, dE_values) in enumerate(zip(Eis_in_meV, TOF_RANGES, dE_values_list)):
    # Get Ei in high resolution
    Ei_in_meV_mg_V = get_peak_energy_high_resolution(Ei_in_meV, energies_mg_V)
    Ei_in_meV_he3_V = get_peak_energy_high_resolution(Ei_in_meV, energies_he3_V)
    # Extract velocity vi
    vi_in_m_per_s_mg = np.sqrt((Ei_in_meV_mg_V/5.227)) * 1e3 # Squires (p. 3)
    vi_in_m_per_s_he3 = np.sqrt((Ei_in_meV_he3_V/5.227)) * 1e3 # Squires (p. 3)
    # Extract tof and distance
    mg_tof_peak_V, mg_sd_d_peak_V, __ = get_tof_and_sd_d(Ei_in_meV_mg_V, tof_range, mg_clu_f_V, 'MG',
                                                         MG_OFFSET, MG_THETA, he3_mapping_V)
    he3_tof_peak_V, he3_sd_d_peak_V, __ = get_tof_and_sd_d(Ei_in_meV_he3_V, tof_range, he3_ev_f_V, 'He-3',
                                                           MG_OFFSET, MG_THETA, he3_mapping_V)
    # Plot energy
    dE_he3 = e_calc.get_energy_transfer(Ei_in_meV_he3_V, he3_tof_peak_V, he3_sd_d_peak_V, MODERATOR_TO_SAMPLE_IN_M) 
    dE_mg = e_calc.get_energy_transfer(Ei_in_meV_mg_V, mg_tof_peak_V, mg_sd_d_peak_V, MODERATOR_TO_SAMPLE_IN_M)
    sections = []
    d_extras = []
    if dE_values is not None:
        for i, dE_value in enumerate(dE_values):
            # Print distances we get here
            dE_min = dE_value-0.4*dE_value
            dE_max = dE_value+0.4*dE_value
            indices = (dE_mg > dE_min) & (dE_mg < dE_max)
            tof_values = mg_tof_peak_V[indices]
            d_values = mg_sd_d_peak_V[indices]
            d_extra = get_d_extra(tof_values, d_values, vi_in_m_per_s_mg, MODERATOR_TO_SAMPLE_IN_M)
            sections.append([dE_min, dE_max])
            d_extras.append(d_extra)
        fig = plt.figure()
        plt.subplot(1, 2, 1)
        number_bins = 50
        factor = 0.2
        E_start, E_stop = -Ei_in_meV*factor, Ei_in_meV*factor
        hist_mg, __ = np.histogram(dE_mg, range=[E_start, E_stop], bins=number_bins)
        hist_he3, __ = np.histogram(dE_he3, range=[E_start, E_stop], bins=number_bins)
        mg_weights = (1/max(hist_mg)) * np.ones(len(dE_mg))
        he3_weights = (1/max(hist_he3)) * np.ones(len(dE_he3))
        plt.hist(dE_mg, color='black', histtype='step', bins=number_bins, range=[E_start, E_stop],
                 label='Multi-Grid data', weights=mg_weights)
        plt.hist(dE_he3, color='grey', histtype='step', linestyle='--', bins=number_bins, range=[E_start, E_stop],
                 label='Helium-3 data', weights=he3_weights)
        for i, (section, dE_value) in enumerate(zip(sections, dE_values)):
            dE_min, dE_max = section
            plt.axvspan(dE_min, dE_max, color=colors[i], alpha=0.5, label='%.2f meV' % dE_value)
        plt.grid(True, which='major', zorder=0)
        plt.grid(True, which='minor', linestyle='--', zorder=0)
        plt.xlim(E_start, E_stop)
        plt.yscale('log')
        plt.xlabel('E$_i$ - E$_f$')
        plt.ylabel('Normalized counts (1/max(hist))')
        plt.suptitle('E$_i$=%.2f meV (%.2f Å)' % (Ei_in_meV, e_calc.meV_to_A(Ei_in_meV)))
        plt.title('Energy transfer')
        plt.legend()
        plt.subplot(1, 2, 2)
        plt.xlabel('Extra distance travelled (m)')
        plt.ylabel('Counts')
        for i, (dE_value, d_extra) in enumerate(zip(dE_values, d_extras)):
            mu = (sum(d_extra)/len(d_extra))
            sigma = np.sqrt((1/len(d_extra)) * sum((d_extra - mu)**2))
            hist, bins, __ = plt.hist(d_extra, bins=30, color=colors[i], range=[0, 0.3],
                                      label='%.2f m ± %.2f m' % (mu, sigma), histtype='step')
        plt.title('Multi-Grid detector: v$_i$ $\cdot$ tof$_{recorded}$ - d$_{recorded}$')
        plt.grid(True, which='major', zorder=0)
        plt.grid(True, which='minor', linestyle='--', zorder=0)
        plt.legend(title='Average extra distance')
        fig.set_figwidth(10)
        fig.set_figheight(5)
        plt.tight_layout()
        plt.show()

## 4.2.3 SiO$_2$<a class="anchor" id="ISIS_BEAMTIME_SIO2"></a>

#### Plot overviews of datasets

In [ ]:
for file_name, mg_area in zip(mg_file_names_SiO2, mg_areas_SiO2):
    clu, ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
    mg_basic_plot.mg_plot_basic_bus(file_name, 9, clu, ev, mg_filter_standard_no_prompt, mg_area, file_name)

#### Compare time-of-flight and energy spectra between Multi-Grid detector and Helium-3 tubes

First, we append all of the Multi-Grid data sets gathered during both the SiO$_2$ runs.

In [ ]:
# Import data frames and store in lists
dfs_clu = []
dfs_ev = []
for file_name in mg_file_names_SiO2:
    df_clu, df_ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
    dfs_clu.append(df_clu)
    dfs_ev.append(df_ev)
# Merge data frames
mg_clu_SiO2 = mg_manage.merge_files(dfs_clu)
mg_ev_SiO2 = mg_manage.merge_files(dfs_ev)

Then, we append all of the data collected with the Helium-3 tubes from the SiO$_2$ runs.

In [ ]:
# Get Helium-3 data
he3_dfs = []
he3_duration_SiO2 = 0
for he3_file_name in he3_file_names_SiO2:
    he3_path = HE3_FOLDER + he3_file_name + '.nxs'
    he3_ev_temp = he3_read.import_data(he3_path)
    he3_dfs.append(he3_ev_temp)
    he3_duration_SiO2 += norm.he3_get_duration(he3_path)
he3_ev_SiO2 = pd.concat(he3_dfs)
he3_mapping_SiO2 = he3_read.get_pixel_to_xyz_mapping(he3_path)
he3_mapping_df_SiO2 = pd.DataFrame(he3_mapping_SiO2)

After that, we filter the data and extract the necessary parameters.

In [ ]:
# Define He-3 filter
he3_pixels = region_semi_full
# Filter data
mg_clu_f_SiO2 = mg_manage.filter_data(mg_clu_SiO2, mg_filter_standard)
he3_ev_f_SiO2 = he3_ev_SiO2[he3_ev_SiO2['pixel_id'].isin(he3_pixels)]
# Get Multi-Grid detector duration
mg_duration_SiO2 = (mg_clu_f_SiO2.time.values[-1] - mg_clu_f_SiO2.time.values[0]) * 62.5e-9
# Get solid angle normalization
he3_solid_angle_SiO2 = norm.he3_get_solid_angle_new(he3_mapping_df_SiO2[he3_mapping_df_SiO2.index.isin(he3_pixels)])
gchs = np.concatenate((np.arange(96, 114, 1), np.array([115]), np.arange(116, 131, 1), np.array([131])))
mg_solid_angle = norm.mg_get_solid_angle_new(gchs, MG_OFFSET, MG_THETA)
# Extract tof and distance for each event
mg_tof_SiO2 = mg_clu_f_SiO2['tof'] * 62.5e-9 * 1e6
mg_sd_d_SiO2 = dclb.get_sample_to_detection_distances(mg_clu_f_SiO2['wch'], mg_clu_f_SiO2['gch'], MG_OFFSET, MG_THETA)
he3_tof_SiO2 = he3_ev_f_SiO2['tof']
he3_sd_d_SiO2 = he3_mapping_SiO2['r'][he3_ev_f_SiO2['pixel_id']]
# Extract weights
mg_weights_SiO2 = (1/mg_duration_SiO2) * (1/mg_solid_angle) * np.ones(len(mg_tof_SiO2))
he3_weights_SiO2 = (1/he3_duration_SiO2) * (1/he3_solid_angle_SiO2) * np.ones(len(he3_tof_SiO2))

Finally, we plot the time-of-flight spectra and energy spectra for both the Multi-Grid detector and the Helium-3 tubes. In the tof-plots, we have also added the delimiters which will be used to cut on time-of-flight when the energy transfer plots will be calculated.

In [ ]:
# Declare number of bin parameters
number_bins_tof = 1000
number_bins_E = 1000
# Declare tof ranges colors
colors = ['blue', 'red', 'green', 'purple', 'orange']
# Plot time-of-flight
fig = plt.figure()
cmnplt.tof_histogram_plot(mg_tof_SiO2, number_bins_tof, label='Multi-Grid',
                          color='blue', weights=mg_weights_SiO2)
cmnplt.tof_histogram_plot(he3_tof_SiO2, number_bins_tof, label='Helium-3 tubes',
                          color='red', linestyle='--', weights=he3_weights_SiO2)
for tof_range, color in zip(TOF_RANGES, colors):
    plt.axvline(x=tof_range[0], color=color)
    plt.axvline(x=tof_range[1], color=color)
plt.legend(title='Detector')
plt.ylabel('counts/s/sr')
fig.set_figwidth(8)
fig.set_figheight(5)
plt.tight_layout()
plt.show()
# Plot energy
fig = plt.figure()
cmnplt.energy_histogram_plot(mg_tof_SiO2, mg_sd_d_SiO2, number_bins_E,
                             MODERATOR_TO_SAMPLE_IN_M, label='Multi-Grid',
                             color='blue', run='', interval=[0, 50], weights=mg_weights_SiO2)
cmnplt.energy_histogram_plot(he3_tof_SiO2, he3_sd_d_SiO2, number_bins_E,
                             MODERATOR_TO_SAMPLE_IN_M, label='Helium-3 tubes',
                             color='red', run='', interval=[0, 50], linestyle='--', weights=he3_weights_SiO2)
plt.legend(title='Detector')
plt.ylabel('counts/s/sr')
fig.set_figwidth(8)
fig.set_figheight(5)
plt.tight_layout()
plt.show()

#### Compare energy transfer spectra between Multi-Grid detector and Helium-3 tubes

First, we calculate the energies of the Multi-Grid and Helium-3 events.

In [ ]:
# Calculate energies in Helium-3 and Multi-Grid detector data
energies_he3_SiO2 = e_calc.get_energy(he3_tof_SiO2, he3_sd_d_SiO2, MODERATOR_TO_SAMPLE_IN_M)
energies_mg_SiO2 = e_calc.get_energy(mg_tof_SiO2, mg_sd_d_SiO2, MODERATOR_TO_SAMPLE_IN_M)

Next, we iterate through each peak and compare the Multi-Grid data with the Helium-3 tubes.

In [ ]:
# Declare number of bins for energy transfer plots
number_bins_dE = 60
# Iterate through all peaks
for i, (Ei_in_meV, tof_range) in enumerate(zip(Eis_in_meV, TOF_RANGES)):
    # Extract tof limits
    tof_min, tof_max = tof_range[0], tof_range[1]
    # Get peak energies at high resolution
    Ei_in_meV_he3_SiO2 = get_peak_energy_high_resolution(Ei_in_meV, energies_he3_SiO2)
    Ei_in_meV_mg_SiO2 = get_peak_energy_high_resolution(Ei_in_meV, energies_mg_SiO2)
    # Filter on tof
    mg_clu_peak_SiO2 = mg_clu_f_SiO2[((mg_clu_f_SiO2.tof * 62.5e-9 * 1e6) >= tof_min) &
                                     ((mg_clu_f_SiO2.tof * 62.5e-9 * 1e6) <= tof_max)]
    he3_ev_peak_SiO2 = he3_ev_f_SiO2[(he3_ev_f_SiO2.tof >= tof_min) &
                                     (he3_ev_f_SiO2.tof <= tof_max)]
    # Extract tof and distance
    mg_tof_peak_SiO2 = mg_clu_peak_SiO2['tof'] * 62.5e-9 * 1e6
    mg_sd_d_peak_SiO2 = dclb.get_sample_to_detection_distances(mg_clu_peak_SiO2['wch'],
                                                               mg_clu_peak_SiO2['gch'],
                                                               MG_OFFSET, MG_THETA)
    he3_tof_peak_SiO2 = he3_ev_peak_SiO2['tof']
    he3_sd_d_peak_SiO2 = he3_mapping_SiO2['r'][he3_ev_peak_SiO2['pixel_id']]
    # Plot energy transfer
    fig = plt.figure()
    factor = 1
    cmnplt.energy_transfer_plot(Ei_in_meV_mg_SiO2, mg_tof_peak_SiO2, mg_sd_d_peak_SiO2,
                                MODERATOR_TO_SAMPLE_IN_M, number_bins_dE,
                                fig, Ei_in_meV, label='Multi-Grid', color='blue',
                                normalize_by_counts=True, linestyle='solid', factor=factor)
    cmnplt.energy_transfer_plot(Ei_in_meV_he3_SiO2, he3_tof_peak_SiO2,
                                he3_sd_d_peak_SiO2, MODERATOR_TO_SAMPLE_IN_M,
                                number_bins_dE, fig, Ei_in_meV, label='Helium-3 tubes',
                                color='red', normalize_by_counts=True, linestyle='-', factor=factor)
    plt.ylabel('Normalized counts (counts*(1/sum(counts)))')
    plt.legend(title='Detector')
    plt.show()

After that, we look at the energy transfer spectra for different regions in the Helium-3 array and compare it with the data from the Multi-Grid detector.

In [ ]:
# Declare all Helium-3 regions
he3_ev_f_SiO2_semi_full = he3_ev_SiO2[he3_ev_SiO2['pixel_id'].isin(region_semi_full)]
he3_ev_f_SiO2_MG_region = he3_ev_SiO2[he3_ev_SiO2['pixel_id'].isin(region_of_interest)]
he3_ev_f_SiO2_region_around_MG = he3_ev_SiO2[he3_ev_SiO2['pixel_id'].isin(region_around_MG)]
# Declare figure
fig, axes = plt.subplots(nrows=5, ncols=2, sharey=True, sharex=False)
# Declare number of bins for energy transfer plots
number_bins_dE_left = 150
number_bins_dE_right = 30
energy_labels = ['2.4 meV (5.8 Å)', '3.7 meV (4.7 Å)', '6.3 meV (3.6 Å)', '13.4 meV (2.5 Å)', '44.9 meV (1.3 Å)']
# Iterate through all peaks
bin_widths_left = []
bin_widths_right = []
for i, (Ei_in_meV, tof_range, ax_pair, energy_label) in enumerate(zip(Eis_in_meV, TOF_RANGES, axes, energy_labels)):
    # Extract tof limits
    tof_min, tof_max = tof_range[0], tof_range[1]
    # Get peak energies at high resolution
    Ei_in_meV_he3_SiO2 = get_peak_energy_high_resolution(Ei_in_meV, energies_he3_SiO2)
    Ei_in_meV_mg_SiO2 = get_peak_energy_high_resolution(Ei_in_meV, energies_mg_SiO2)
    # Filter on tof
    mg_clu_peak_SiO2 = mg_clu_f_SiO2[((mg_clu_f_SiO2.tof * 62.5e-9 * 1e6) >= tof_min) &
                                     ((mg_clu_f_SiO2.tof * 62.5e-9 * 1e6) <= tof_max)]
    he3_ev_peak_SiO2 = he3_ev_f_SiO2[(he3_ev_f_SiO2.tof >= tof_min) &
                                     (he3_ev_f_SiO2.tof <= tof_max)]
    he3_ev_peak_SiO2_MG_region = he3_ev_f_SiO2_MG_region[(he3_ev_f_SiO2_MG_region.tof >= tof_min) &
                                                         (he3_ev_f_SiO2_MG_region.tof <= tof_max)]
    he3_ev_peak_SiO2_region_around_MG = he3_ev_f_SiO2_region_around_MG[(he3_ev_f_SiO2_region_around_MG.tof >= tof_min) &
                                                                       (he3_ev_f_SiO2_region_around_MG.tof <= tof_max)]
    # Extract tof and distance
    mg_tof_peak_SiO2 = mg_clu_peak_SiO2['tof'] * 62.5e-9 * 1e6
    mg_sd_d_peak_SiO2 = dclb.get_sample_to_detection_distances(mg_clu_peak_SiO2['wch'],
                                                               mg_clu_peak_SiO2['gch'],
                                                               MG_OFFSET, MG_THETA)
    he3_tof_peak_SiO2 = he3_ev_peak_SiO2['tof']
    he3_sd_d_peak_SiO2 = he3_mapping_SiO2['r'][he3_ev_peak_SiO2['pixel_id']]
    he3_tof_peak_SiO2_MG_region = he3_ev_peak_SiO2_MG_region['tof']
    he3_sd_d_peak_SiO2_MG_region = he3_mapping_SiO2['r'][he3_ev_peak_SiO2_MG_region['pixel_id']]
    he3_tof_peak_SiO2_region_around_MG = he3_ev_peak_SiO2_region_around_MG['tof']
    he3_sd_d_peak_SiO2_region_around_MG = he3_mapping_SiO2['r'][he3_ev_peak_SiO2_region_around_MG['pixel_id']]
    # Extract dE
    delta_E_mg = e_calc.get_energy_transfer(Ei_in_meV_mg_SiO2, mg_tof_peak_SiO2,
                                            mg_sd_d_peak_SiO2,
                                            MODERATOR_TO_SAMPLE_IN_M)
    delta_E_he3 = e_calc.get_energy_transfer(Ei_in_meV_he3_SiO2, he3_tof_peak_SiO2,
                                             he3_sd_d_peak_SiO2,
                                             MODERATOR_TO_SAMPLE_IN_M)
    delta_E_he3_MG_region = e_calc.get_energy_transfer(Ei_in_meV_he3_SiO2, he3_tof_peak_SiO2_MG_region,
                                                       he3_sd_d_peak_SiO2_MG_region,
                                                       MODERATOR_TO_SAMPLE_IN_M)
    delta_E_he3_region_around_MG = e_calc.get_energy_transfer(Ei_in_meV_he3_SiO2, he3_tof_peak_SiO2_region_around_MG,
                                                              he3_sd_d_peak_SiO2_region_around_MG,
                                                              MODERATOR_TO_SAMPLE_IN_M)
    # Histogram data
    min_val, max_val = -Ei_in_meV, Ei_in_meV
    bin_width_left = plot_hist_normed(delta_E_mg, ax_pair[0], min_val, max_val, 'royalblue', 'Multi-Grid', '-', number_bins_dE_left)
    plot_hist_normed(delta_E_he3, ax_pair[0], min_val, max_val, 'indianred', '$^3$He-tubes', '--', number_bins_dE_left)
    plot_hist_normed(delta_E_he3_MG_region, ax_pair[0], min_val, max_val, 'mediumseagreen', '$^3$He-tubes', 'dotted', number_bins_dE_left)
    plot_hist_normed(delta_E_he3_region_around_MG, ax_pair[0], min_val, max_val, 'orchid', '$^3$He-tubes', '-.', number_bins_dE_left)
    bin_widths_left.append(bin_width_left)
    ax_pair[0].set_xlim(-0.2*Ei_in_meV, 0.2*Ei_in_meV)
    ax_pair[0].set_ylabel('%s\nIntensity' % energy_label)
    bin_width_right = plot_hist_normed(delta_E_mg, ax_pair[1], min_val, max_val, 'royalblue', 'Multi-Grid', '-', number_bins_dE_right)
    plot_hist_normed(delta_E_he3, ax_pair[1], min_val, max_val, 'indianred', '$^3$He-tubes', '--', number_bins_dE_right)
    plot_hist_normed(delta_E_he3_MG_region, ax_pair[1], min_val, max_val, 'mediumseagreen', '$^3$He-tubes', 'dotted', number_bins_dE_right)
    plot_hist_normed(delta_E_he3_region_around_MG, ax_pair[1], min_val, max_val, 'orchid', '$^3$He-tubes', '-.', number_bins_dE_right)
    ax_pair[1].set_xlim(-Ei_in_meV, Ei_in_meV)
    bin_widths_right.append(bin_width_right)
    
plt.subplots_adjust(wspace=.0)
plt.subplots_adjust(hspace=.2)
# Declare legend
legend_1 = [Line2D([0], [0], color='royalblue', label='Multi-Grid detector', linestyle='-')]
legend_2 = [Line2D([0], [0], color='indianred', label='Full array', linestyle='--'),
            Line2D([0], [0], color='mediumseagreen', label='Region 1', linestyle='dotted'),
            Line2D([0], [0], color='orchid', label='Region 2', linestyle='-.')]
fig.legend(bbox_to_anchor=(0.70, 0.97), handles=legend_1, ncol=1)
fig.legend(bbox_to_anchor=(0.85, 0.94), handles=legend_2, ncol=3, title='Helium-3 tubes')
# Declare energies
#center, start, step = 0.45, 0.86, 0.149
center, start, step = 0.16, 0.870, 0.147
bin_box_offset_x, bin_box_offset_y = 0.04, 0.083
labels_locations = [[center, start], [center, start], [center, 0.6], [center, 0.4], [center, 0.2]]
for i, (energy_label, bin_width_left, bin_width_right) in enumerate(zip(energy_labels, bin_widths_left, bin_widths_right)):
    x_val, y_val = center, start - i*step
    fig.text(x_val, y_val, '\nBin width:\n%.3f meV' % bin_width_left, verticalalignment='center')
    fig.text(x_val+0.4, y_val, '\nBin width:\n%.3f meV' % bin_width_right, verticalalignment='center')
# Set axis labels

axes[-1][0].set_xlabel('E$_i$ - E$_f$ (meV)')
axes[-1][1].set_xlabel('E$_i$ - E$_f$ (meV)')
fig.set_figwidth(5)
fig.set_figheight(10)
fig.suptitle('SiO$_2$ measurements - region comparisons', x=0.55, y=0.99)
plt.savefig('%s/../output/SiO2_energy_transfer_comparison_between_regions.png' % nb_path, dpi=800)

#### Extracting beam monitor counts

First, we extract the beam monitor data.

In [ ]:
# Get beam monitor data
bm_path = HE3_FOLDER + he3_file_names_SiO2[0] + '.nxs'
bm_ev_SiO2 = bm_read.import_data(he3_path)
for i, he3_file_name in enumerate(he3_file_names_SiO2[1:]):
    bm_path = HE3_FOLDER + he3_file_name + '.nxs'
    bm_ev_temp = bm_read.import_data(he3_path)
    bm_ev_SiO2 = bm_ev_SiO2 + bm_ev_temp

Then, we plot the data.

In [ ]:
bm_basic_plot.bm_plot_basic(bm_ev_SiO2, 100, 'SiO$_2$, RRM')

After that, we extract normalization constants based on beam monitor data.

In [ ]:
# Declare tof ranges colors
colors = ['blue', 'red', 'green', 'purple', 'orange']
fig = plt.figure()
df_bm_5 = bm_ev_SiO2.loc[5]
df_bm_5_tofs = df_bm_5.index.values
weights_bm_5 = df_bm_5.values
hist, bins, __ = plt.hist(df_bm_5.index, weights=weights_bm_5,
                          zorder=5, label='Beam monitor data',
                          histtype='step', range=[0, 100000],
                          bins=500, color='black')
# Extra beam monitor normalization
beam_monitor_norm_SiO2 = []
for tof_range, color in zip(TOF_RANGES_BM, colors):
    indices = (df_bm_5_tofs > tof_range[0]) & (df_bm_5_tofs < tof_range[1])
    counts_within_range = sum(weights_bm_5[indices])
    beam_monitor_norm_SiO2.append(counts_within_range)
    plt.axvline(x=tof_range[0], color=color, label='Counts: %d' % counts_within_range)
    plt.axvline(x=tof_range[1], color=color, label=None)
plt.title('Beam monitor data - SiO$_2$')
plt.legend()
plt.yscale('log')

#### Look at 3D histograms for the different energies

In [ ]:
# Declare necessary parameters
energy_labels = ['2.4 meV (5.8 Å)', '3.7 meV (4.7 Å)', '6.3 meV (3.6 Å)', '13.4 meV (2.5 Å)', '44.9 meV (1.3 Å)']
for i in np.arange(0, 5, 1):
    tof_range_peak = TOF_RANGES[i]
    Ei_in_meV = Eis_in_meV[i]
    tof_min, tof_max = tof_range_peak[0], tof_range_peak[1]
    he3_ev_peak_SiO2 = he3_ev_SiO2[(he3_ev_SiO2.tof >= tof_min) &
                                   (he3_ev_SiO2.tof <= tof_max)]
    he3_basic_plot.he3_plot_3D_histogram(he3_ev_peak_SiO2[he3_ev_peak_SiO2['pixel_id'].isin(region_full)], he3_mapping_SiO2, label='SiO<sub>2</sub><br>%s' % energy_labels[i], file_name='SiO2_%d_meV' % Ei_in_meV)

## 4.2.4 V & SiO$_2$<a class="anchor" id="ISIS_BEAMTIME_V_AND_SIO2"></a>

#### Compare time-of-flight and energy spectra between Multi-Grid detector and Helium-3 tubes

First, we append all of the Multi-Grid data sets gathered during both the V and SiO$_2$ runs.

In [ ]:
# Declare file names
mg_file_names_V_and_SiO2 = np.concatenate((mg_file_names_V, mg_file_names_SiO2))
# Import data frames and store in lists
dfs_clu = []
dfs_ev = []
for file_name in mg_file_names_V_and_SiO2:
    df_clu, df_ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
    dfs_clu.append(df_clu)
    dfs_ev.append(df_ev)
# Merge data frames
mg_clu_V_and_SiO2 = mg_manage.merge_files(dfs_clu)
mg_ev_V_and_SiO2 = mg_manage.merge_files(dfs_ev)

After that, we filter the data and extract the necessary parameters.

In [ ]:
# Filter data
mg_clu_f_V_and_SiO2 = mg_manage.filter_data(mg_clu_V_and_SiO2, mg_filter_standard)
# Get Multi-Grid detector duration
mg_duration_V_and_SiO2 = (mg_clu_f_V_and_SiO2.time.values[-1] - mg_clu_f_V_and_SiO2.time.values[0]) * 62.5e-9
# Get solid angle normalization
gchs = np.concatenate((np.arange(96, 114, 1), np.array([115]), np.arange(116, 131, 1), np.array([131])))
mg_solid_angle = norm.mg_get_solid_angle_new(gchs, MG_OFFSET, MG_THETA)
# Extract tof and distance for each event
mg_tof_V_and_SiO2 = mg_clu_f_V_and_SiO2['tof'] * 62.5e-9 * 1e6
mg_sd_d_V_and_SiO2 = dclb.get_sample_to_detection_distances(mg_clu_f_V_and_SiO2['wch'],
                                                            mg_clu_f_V_and_SiO2['gch'],
                                                            MG_OFFSET, MG_THETA)
# Extract weights
mg_weights_V_and_SiO2 = (1/mg_duration_V_and_SiO2) * (1/mg_solid_angle) * np.ones(len(mg_tof_V_and_SiO2))

Finally, we plot the time-of-flight spectra and energy spectra for both the Multi-Grid detector and the Helium-3 tubes. In the tof-plots, we have also added the delimiters which will be used to cut on time-of-flight when the energy transfer plots will be calculated.

In [ ]:
# Declare number of bin parameters
number_bins_tof = 1000
number_bins_E = 1000
# Declare tof ranges
colors = ['blue', 'red', 'green', 'purple', 'orange']
# Plot time-of-flight
fig = plt.figure()
cmnplt.tof_histogram_plot(mg_tof_V_and_SiO2, number_bins_tof, label='Multi-Grid',
                          color='blue', run='', weights=mg_weights_V_and_SiO2)
cmnplt.tof_histogram_plot(he3_tof_V, number_bins_tof, label='Helium-3 tubes',
                          color='red', run='', linestyle='--', weights=he3_weights_V)
for tof_range, color in zip(TOF_RANGES, colors):
    plt.axvline(x=tof_range[0], color=color)
    plt.axvline(x=tof_range[1], color=color)
plt.legend(title='Detector')
plt.ylabel('counts/s/sr')
fig.set_figwidth(8)
fig.set_figheight(5)
plt.tight_layout()
plt.show()
# Plot energy
fig = plt.figure()
cmnplt.energy_histogram_plot(mg_tof_V_and_SiO2, mg_sd_d_V_and_SiO2,
                             number_bins_E, MODERATOR_TO_SAMPLE_IN_M,
                             label='Multi-Grid', color='blue',
                             interval=[0, 50], weights=mg_weights_V_and_SiO2)
cmnplt.energy_histogram_plot(he3_tof_V, he3_sd_d_V,
                             number_bins_E, MODERATOR_TO_SAMPLE_IN_M,
                             label='Helium-3 tubes', color='red',
                             interval=[0, 50], linestyle='--', weights=he3_weights_V)
plt.legend(title='Detector')
plt.ylabel('counts/s/sr')
fig.set_figwidth(8)
fig.set_figheight(5)
plt.tight_layout()
plt.show()

#### Compare energy transfer spectra between Multi-Grid detector and Helium-3 tubes

First, we calculate the energies of the Multi-Grid detector.

In [ ]:
# Calculate energies in Multi-Grid detector data
energies_mg_V_and_SiO2 = e_calc.get_energy(mg_tof_V_and_SiO2, mg_sd_d_V_and_SiO2, MODERATOR_TO_SAMPLE_IN_M)

Then, we iterate through each peak and compare the Multi-Grid data with the Helium-3 tubes.

In [ ]:
# Declare number of bins for energy transfer plots
number_bins_dE = 100
# Iterate through all peaks
for i, (Ei_in_meV, tof_range) in enumerate(zip(Eis_in_meV, TOF_RANGES)):
    # Extract tof limits
    tof_min, tof_max = tof_range[0], tof_range[1]
    # Get peak energies at high resolution
    Ei_in_meV_he3_V = get_peak_energy_high_resolution(Ei_in_meV, energies_he3_V)
    Ei_in_meV_mg_V_and_SiO2 = get_peak_energy_high_resolution(Ei_in_meV, energies_mg_V_and_SiO2)
    # Filter on tof
    mg_clu_peak_V_and_SiO2 = mg_clu_f_V_and_SiO2[((mg_clu_f_V_and_SiO2.tof * 62.5e-9 * 1e6) >= tof_min) &
                                                 ((mg_clu_f_V_and_SiO2.tof * 62.5e-9 * 1e6) <= tof_max)]
    he3_ev_peak_V = he3_ev_f_V[(he3_ev_f_V.tof >= tof_min) &
                               (he3_ev_f_V.tof <= tof_max)]
    # Extract tof and distance
    mg_tof_peak_V_and_SiO2 = mg_clu_peak_V_and_SiO2['tof'] * 62.5e-9 * 1e6
    mg_sd_d_peak_V_and_SiO2 = dclb.get_sample_to_detection_distances(mg_clu_peak_V_and_SiO2['wch'],
                                                                     mg_clu_peak_V_and_SiO2['gch'],
                                                                     MG_OFFSET, MG_THETA)
    he3_tof_peak_V = he3_ev_peak_V['tof']
    he3_sd_d_peak_V = he3_mapping_V['r'][he3_ev_peak_V['pixel_id']]
    # Plot energy transfer
    fig = plt.figure()
    factor = 1
    cmnplt.energy_transfer_plot(Ei_in_meV_mg_V_and_SiO2, mg_tof_peak_V_and_SiO2, mg_sd_d_peak_V_and_SiO2,
                                MODERATOR_TO_SAMPLE_IN_M, number_bins_dE,
                                fig, Ei_in_meV, label='Multi-Grid', color='blue',
                                normalize_by_max=True, include_hist=True, marker='o', markerfacecolor='none',
                                linestyle='solid', factor=factor)
    cmnplt.energy_transfer_plot(Ei_in_meV_he3_V, he3_tof_peak_V,
                                he3_sd_d_peak_V, MODERATOR_TO_SAMPLE_IN_M,
                                number_bins_dE, fig, Ei_in_meV, label='Helium-3 tubes',
                                color='red', normalize_by_max=True, include_hist=True, marker='.', 
                                linestyle='-', factor=factor)
    plt.ylabel('Normalized counts (counts*(1/max(hist)))')
    plt.ylim(1e-5, 1)
    plt.legend(title='Detector')
    plt.show()

#### Investigate shoulder dependence in Multi-Grid detector when different cuts are applied

To investigate any geometrical dependecies we split the detector in three different ways: front and back, left and right, bottom and top.

In [ ]:
# Prepare figure
#fig, axes = plt.subplots(nrows=3, ncols=2, sharey=False, sharex='col')
fig = plt.figure()
axes = [[fig.add_subplot(3, 2, 1), fig.add_subplot(3, 2, 2, projection='3d')],
        [fig.add_subplot(3, 2, 3), fig.add_subplot(3, 2, 4, projection='3d')],
        [fig.add_subplot(3, 2, 5), fig.add_subplot(3, 2, 6, projection='3d')]]
        
# Declare number of bins for energy transfer plots
number_bins_dE = 150
tof_range =  TOF_RANGES[3]
Ei_in_meV = Eis_in_meV[3]
# Get peak energies at high resolution
Ei_in_meV_he3_V = get_peak_energy_high_resolution(Ei_in_meV, energies_he3_V)
Ei_in_meV_mg_V_and_SiO2 = get_peak_energy_high_resolution(Ei_in_meV, energies_mg_V_and_SiO2)
# Extract tof and distance
mg_tof_peak_V_and_SiO2, mg_sd_d_peak_V_and_SiO2, mg_clu_peak_V_and_SiO2 = get_tof_and_sd_d(Ei_in_meV_mg_V_and_SiO2, tof_range, mg_clu_f_V_and_SiO2, 'MG',
                                                                                          MG_OFFSET, MG_THETA, he3_mapping_V)
he3_tof_peak_V, he3_sd_d_peak_V, __ = get_tof_and_sd_d(Ei_in_meV_he3_V, tof_range, he3_ev_f_V, 'He3',
                                                       MG_OFFSET, MG_THETA, he3_mapping_V)
# Plot energy transfer for different geometrical slices
factor = 0.2    
step = 8
layers = np.arange(0, 16, step)
linestyles = ['-', '--']
colors = ['royalblue', 'indianred']
labels_1 = ['Front', 'Back']
# Layers
for layer, linestyle, color, label in zip(layers, linestyles, colors, labels_1):
    front_layer = layer
    back_layer = layer + step - 1
    mg_clu_segment = mg_clu_peak_V_and_SiO2[((mg_clu_peak_V_and_SiO2.wch%16) >= front_layer) &
                                            ((mg_clu_peak_V_and_SiO2.wch%16) <= back_layer)]
    mg_tof_peak_segment_V_and_SiO2, mg_sd_d_peak_segment_V_and_SiO2, __ = get_tof_and_sd_d(Ei_in_meV_mg_V_and_SiO2, tof_range,
                                                                         mg_clu_segment, 'MG',
                                                                         MG_OFFSET, MG_THETA, he3_mapping_V)
    delta_E_mg = e_calc.get_energy_transfer(Ei_in_meV_mg_V_and_SiO2, mg_tof_peak_segment_V_and_SiO2,
                                            mg_sd_d_peak_segment_V_and_SiO2,
                                            MODERATOR_TO_SAMPLE_IN_M)
    min_val, max_val = -Ei_in_meV, Ei_in_meV
    bin_width = plot_hist_normed(delta_E_mg, axes[0][0], min_val, max_val, color, label, linestyle, number_bins_dE)
    axes[0][0].set_xlim(-0.2*Ei_in_meV, 0.2*Ei_in_meV)
    axes[0][0].set_ylabel('%s\nIntensity')
    axes[0][0].legend(loc=1)
    #axes[0][0].set_xlabel('E$_i$-E$_f$ (meV)')
# Rows
step = 3
rows = np.arange(0, 6, step)
labels_2 = ['Left', 'Right']
for row, linestyle, color, label in zip(rows, linestyles, colors, labels_2):
    left_row = row
    right_row = row + step - 1
    mg_clu_segment = mg_clu_peak_V_and_SiO2[((mg_clu_peak_V_and_SiO2.wch//16) >= left_row) &
                                            ((mg_clu_peak_V_and_SiO2.wch//16) <= right_row)]
    mg_tof_peak_segment_V_and_SiO2, mg_sd_d_peak_segment_V_and_SiO2, __ = get_tof_and_sd_d(Ei_in_meV_mg_V_and_SiO2, tof_range,
                                                                         mg_clu_segment, 'MG',
                                                                         MG_OFFSET, MG_THETA, he3_mapping_V)
    delta_E_mg = e_calc.get_energy_transfer(Ei_in_meV_mg_V_and_SiO2, mg_tof_peak_segment_V_and_SiO2,
                                            mg_sd_d_peak_segment_V_and_SiO2,
                                            MODERATOR_TO_SAMPLE_IN_M)
    min_val, max_val = -Ei_in_meV, Ei_in_meV
    bin_width = plot_hist_normed(delta_E_mg, axes[1][0], min_val, max_val, color, label, linestyle, number_bins_dE)
    axes[1][0].set_xlim(-0.2*Ei_in_meV, 0.2*Ei_in_meV)
    axes[1][0].set_ylabel('%s\nIntensity')
    axes[1][0].legend()
    #axes[1][0].set_xlabel('E$_i$-E$_f$ (meV)')
# Top and bottom
step = 18
grids = np.arange(96, 133, step)
labels_3 = ['Bottom', 'Top']
for grid, linestyle, color, label in zip(grids, linestyles, colors, labels_3):
    bottom_grid = grid
    top_grid = grid + step - 1
    mg_clu_segment = mg_clu_peak_V_and_SiO2[((mg_clu_peak_V_and_SiO2.gch) >= bottom_grid) &
                                            ((mg_clu_peak_V_and_SiO2.gch) <= top_grid)]
    mg_tof_peak_segment_V_and_SiO2, mg_sd_d_peak_segment_V_and_SiO2, __ = get_tof_and_sd_d(Ei_in_meV_mg_V_and_SiO2, tof_range,
                                                                                           mg_clu_segment, 'MG',
                                                                                           MG_OFFSET, MG_THETA, he3_mapping_V)
    delta_E_mg = e_calc.get_energy_transfer(Ei_in_meV_mg_V_and_SiO2, mg_tof_peak_segment_V_and_SiO2,
                                            mg_sd_d_peak_segment_V_and_SiO2,
                                            MODERATOR_TO_SAMPLE_IN_M)
    min_val, max_val = -Ei_in_meV, Ei_in_meV
    bin_width = plot_hist_normed(delta_E_mg, axes[2][0], min_val, max_val, color, label, linestyle, number_bins_dE)
    axes[2][0].set_xlim(-0.2*Ei_in_meV, 0.2*Ei_in_meV)
    axes[2][0].set_ylabel('%s\nIntensity')
    axes[2][0].set_xlabel('E$_i$-E$_f$ (meV)')
    axes[2][0].legend(loc=1)
    
# Add 3D representation of detector
xs = []
ys = []
zs = []
cs_1 = []
cs_2 = []
cs_3 = []
for wch in np.arange(0, 96, 1):
    for gch in np.arange(96, 133, 1):
        x,y,z = dclb.get_local_xyz(wch, gch) #, MG_OFFSET, MG_THETA)
        xs.append(x)
        ys.append(y)
        zs.append(z)
        # Front and back
        if wch%16 >= 7:
            cs_1.append('royalblue')
        else:
            cs_1.append('indianred')
        # Left and right
        if wch//16 >= 3:
            cs_2.append('royalblue')
        else:
            cs_2.append('indianred')
        if gch <= 114:
            cs_3.append('royalblue')
        else:
            cs_3.append('indianred')
axes[0][1].scatter(zs, xs, ys, color=cs_1, s=1)
axes[0][1].set_box_aspect((np.ptp(zs), np.ptp(xs), np.ptp(ys)))
axes[0][1]._axis3don = False
axes[0][1].dist = 8.7

axes[1][1].scatter(zs, xs, ys, color=cs_2, s=1)
axes[1][1].set_box_aspect((np.ptp(zs), np.ptp(xs), np.ptp(ys)))
axes[1][1]._axis3don = False
axes[1][1].dist = 8.7

axes[2][1].scatter(zs, xs, ys, color=cs_3, s=1)
axes[2][1].set_box_aspect((np.ptp(zs), np.ptp(xs), np.ptp(ys)))
axes[2][1]._axis3don = False
axes[2][1].dist = 8.7

fig.text(0.45, 0.89, 'Front vs back')
fig.text(0.45, 0.63, 'Left vs right')
fig.text(0.45, 0.37, 'Bottom vs top')
plt.subplots_adjust(wspace=0, hspace=0.4)

fig.set_figheight(7)
fig.set_figwidth(5)
fig.suptitle('Multi-Grid detector, V + SiO$_2$ combined\nEnergy: 13.4 meV (2.5 Å)')
plt.savefig('%s/../output/multi_grid_detector_shoulder_cuts.png' % nb_path, dpi=800)

#### Compare energy transfer spectra between V and SiO$_2$: Multi-Grid detector

In [ ]:
def plot_hist_normed(array, ax, min_val, max_val, color, label, linestyle, number_bins, include_hist=True):
    hist, bins = np.histogram(array, range=[min_val, max_val], bins=number_bins)
    idxs_one = np.where(hist == 1)
    norm = 1/max(hist)
    bin_centers = (bins[:-1] + bins[1:]) / 2
    if include_hist:
        ax.hist(array, range=[min_val, max_val], bins=number_bins, weights=norm*np.ones(len(array)),
                color=color, histtype='step', linestyle=linestyle)
    hist, bins = np.histogram(array, range=[min_val, max_val], bins=number_bins, weights=norm*np.ones(len(array)))
    bin_centers = (bins[:-1] + bins[1:]) / 2
    # Extract error bars
    errors_up = np.sqrt(hist/(norm))*(norm)
    errors_down = np.sqrt(hist/(norm))*(norm)
    errors_down[idxs_one] = 0
    ax.errorbar(bin_centers, hist, (errors_down, errors_up), color=color, label=label, linestyle='', #capsize=3,
                marker='.', markersize=4)
    ax.set_ylim(1e-5, 1)
    ax.set_yscale('log')
    ax.set_yticks([1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5])
    ax.set_yticklabels(['1', '1e-1', '1e-2', '1e-3', '1e-4', '1e-5'])
    bin_width = bins[1] - bins[0]
    ax.grid(True, which='major', linestyle='dotted', zorder=0)
    return bin_width

# Declare figure
#((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8), (ax9, ax10))
fig, axes = plt.subplots(nrows=5, ncols=2, sharey=True, sharex=False)
# Declare number of bins for energy transfer plots
number_bins_dE_left = 150
number_bins_dE_right = 30
energy_labels = ['2.4 meV (5.8 Å)', '3.7 meV (4.7 Å)', '6.3 meV (3.6 Å)', '13.4 meV (2.5 Å)', '44.9 meV (1.3 Å)']
# Iterate through all peaks
bin_widths_left = []
bin_widths_right = []
for i, (Ei_in_meV, tof_range, ax_pair, energy_label) in enumerate(zip(Eis_in_meV, TOF_RANGES, axes, energy_labels)):
    # Extract tof limits
    tof_min, tof_max = tof_range[0], tof_range[1]
    # Get peak energies at high resolution
    Ei_in_meV_mg_V = get_peak_energy_high_resolution(Ei_in_meV, energies_mg_V)
    Ei_in_meV_mg_SiO2 = get_peak_energy_high_resolution(Ei_in_meV, energies_mg_SiO2)
    # Filter on tof
    mg_clu_peak_V = mg_clu_f_V[((mg_clu_f_V.tof * 62.5e-9 * 1e6) >= tof_min) &
                               ((mg_clu_f_V.tof * 62.5e-9 * 1e6) <= tof_max)]
    mg_clu_peak_SiO2 = mg_clu_f_SiO2[((mg_clu_f_SiO2.tof * 62.5e-9 * 1e6) >= tof_min) &
                                     ((mg_clu_f_SiO2.tof * 62.5e-9 * 1e6) <= tof_max)]

    # Extract tof and distance
    mg_tof_peak_V = mg_clu_peak_V['tof'] * 62.5e-9 * 1e6
    mg_sd_d_peak_V = dclb.get_sample_to_detection_distances(mg_clu_peak_V['wch'],
                                                            mg_clu_peak_V['gch'],
                                                            MG_OFFSET, MG_THETA)
    mg_tof_peak_SiO2 = mg_clu_peak_SiO2['tof'] * 62.5e-9 * 1e6
    mg_sd_d_peak_SiO2 = dclb.get_sample_to_detection_distances(mg_clu_peak_SiO2['wch'],
                                                               mg_clu_peak_SiO2['gch'],
                                                               MG_OFFSET, MG_THETA)
    # Extract dE
    delta_E_mg_V = e_calc.get_energy_transfer(Ei_in_meV_mg_V, mg_tof_peak_V,
                                              mg_sd_d_peak_V,
                                              MODERATOR_TO_SAMPLE_IN_M)
    delta_E_mg_SiO2 = e_calc.get_energy_transfer(Ei_in_meV_mg_SiO2, mg_tof_peak_SiO2,
                                                 mg_sd_d_peak_SiO2,
                                                 MODERATOR_TO_SAMPLE_IN_M)
    # Histogram data
    min_val, max_val = -Ei_in_meV, Ei_in_meV
    bin_width_left = plot_hist_normed(delta_E_mg_V, ax_pair[0], min_val, max_val, 'royalblue', 'V', '-', number_bins_dE_left)
    plot_hist_normed(delta_E_mg_SiO2, ax_pair[0], min_val, max_val, 'indianred', 'SiO2', '--', number_bins_dE_left)
    bin_widths_left.append(bin_width_left)
    ax_pair[0].set_xlim(-0.2*Ei_in_meV, 0.2*Ei_in_meV)
    ax_pair[0].set_ylabel('%s\nIntensity' % energy_label)
    bin_width_right = plot_hist_normed(delta_E_mg_V, ax_pair[1], min_val, max_val, 'royalblue', 'V', '-', number_bins_dE_right)
    plot_hist_normed(delta_E_mg_SiO2, ax_pair[1], min_val, max_val, 'indianred', 'SiO2', '--', number_bins_dE_right)
    ax_pair[1].set_xlim(-Ei_in_meV, Ei_in_meV)
    bin_widths_right.append(bin_width_right)
    

plt.subplots_adjust(wspace=.0)
plt.subplots_adjust(hspace=.2)
# Declare legend
legend_1 = [Line2D([0], [0], color='royalblue', label='V', linestyle='-'),
            Line2D([0], [0], color='indianred', label='SiO$_2$', linestyle='--')]
fig.legend(bbox_to_anchor=(0.69, 0.91), handles=legend_1, ncol=2)
# Declare energies
#center, start, step = 0.45, 0.86, 0.149
center, start, step = 0.16, 0.870, 0.147
bin_box_offset_x, bin_box_offset_y = 0.04, 0.083
labels_locations = [[center, start], [center, start], [center, 0.6], [center, 0.4], [center, 0.2]]
for i, (energy_label, bin_width_left, bin_width_right) in enumerate(zip(energy_labels, bin_widths_left, bin_widths_right)):
    x_val, y_val = center, start - i*step
    fig.text(x_val, y_val, '\nBin width:\n%.3f meV' % bin_width_left, verticalalignment='center')
    fig.text(x_val+0.4, y_val, '\nBin width:\n%.3f meV' % bin_width_right, verticalalignment='center')
# Set axis labels

axes[-1][0].set_xlabel('E$_i$ - E$_f$ (meV)')
axes[-1][1].set_xlabel('E$_i$ - E$_f$ (meV)')
fig.set_figwidth(5)
fig.set_figheight(10)
fig.suptitle('Multi-Grid detector, V and SiO$_2$', x=0.55, y=0.925)
plt.savefig('%s/../output/multi_grid_detector_energy_transfer_v_and_SiO2_comparison.png' % nb_path, dpi=800)

#### Compare energy transfer spectra between V and SiO$_2$: Helium-3 tubes

To perform the comparison, we simply iterate through all energy transfer peaks and overlay the results from the Helium-3 tubes collected during the V and SiO$_2$ runs.

In [ ]:
def plot_hist_custom_norm(array, ax, min_val, max_val, color, label, linestyle, number_bins, include_hist=True, norm=1):
    hist, bins = np.histogram(array, range=[min_val, max_val], bins=number_bins)
    idxs_one = np.where(hist == 1)
    bin_centers = (bins[:-1] + bins[1:]) / 2
    if include_hist:
        ax.hist(array, range=[min_val, max_val], bins=number_bins, weights=norm*np.ones(len(array)),
                color=color, histtype='step', linestyle=linestyle)
    hist, bins = np.histogram(array, range=[min_val, max_val], bins=number_bins, weights=norm*np.ones(len(array)))
    bin_centers = (bins[:-1] + bins[1:]) / 2
    # Extract error bars
    errors_up = np.sqrt(hist/(norm))*(norm)
    errors_down = np.sqrt(hist/(norm))*(norm)
    errors_down[idxs_one] = 0
    ax.errorbar(bin_centers, hist, (errors_down, errors_up), color=color, label=label, linestyle='', #capsize=3,
                marker='.', markersize=1)
    ax.set_ylim(1e-5, 1e1)
    ax.set_yscale('log')
    ax.set_yticks([10, 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5])
    ax.set_yticklabels(['1e1', '1', '1e-1', '1e-2', '1e-3', '1e-4', '1e-5'])
    bin_width = bins[1] - bins[0]
    ax.grid(True, which='major', linestyle='dotted', zorder=0)
    return bin_width

# Declare figure
#((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8), (ax9, ax10))
fig, axes = plt.subplots(nrows=6, ncols=1, sharey=True, sharex=False)
# Declare number of bins for energy transfer plots
number_bins_dE_left = 1000
number_bins_dE_right = 1000
energy_labels = ['2.4 meV (5.8 Å)', '3.7 meV (4.7 Å)', '6.3 meV (3.6 Å)', '13.4 meV (2.5 Å)', '44.9 meV (1.3 Å)']
# Iterate through all peaks
bin_widths = []
for i, (Ei_in_meV, tof_range, ax, energy_label) in enumerate(zip(Eis_in_meV, TOF_RANGES, axes[1:], energy_labels)):
    # Extract tof limits
    tof_min, tof_max = tof_range[0], tof_range[1]
    # Get peak energies at high resolution
    Ei_in_meV_he3_V = get_peak_energy_high_resolution(Ei_in_meV, energies_he3_V)
    Ei_in_meV_he3_SiO2 = get_peak_energy_high_resolution(Ei_in_meV, energies_he3_SiO2)
    # Filter on tof
    he3_ev_peak_V = he3_ev_f_V[(he3_ev_f_V.tof >= tof_min) &
                               (he3_ev_f_V.tof <= tof_max)]
    he3_ev_peak_SiO2 = he3_ev_f_SiO2[(he3_ev_f_SiO2.tof >= tof_min) &
                                     (he3_ev_f_SiO2.tof <= tof_max)]
    # Extract tof and distance
    he3_tof_peak_V = he3_ev_peak_V['tof']
    he3_sd_d_peak_V = he3_mapping_V['r'][he3_ev_peak_V['pixel_id']]
    he3_tof_peak_SiO2 = he3_ev_peak_SiO2['tof']
    he3_sd_d_peak_SiO2 = he3_mapping_SiO2['r'][he3_ev_peak_SiO2['pixel_id']]
    # Extract dE
    delta_E_he3_V = e_calc.get_energy_transfer(Ei_in_meV_he3_V, he3_tof_peak_V,
                                               he3_sd_d_peak_V,
                                               MODERATOR_TO_SAMPLE_IN_M)
    delta_E_he3_SiO2 = e_calc.get_energy_transfer(Ei_in_meV_he3_SiO2, he3_tof_peak_SiO2,
                                                  he3_sd_d_peak_SiO2,
                                                  MODERATOR_TO_SAMPLE_IN_M)
    # Histogram data
    min_val, max_val = -Ei_in_meV, Ei_in_meV
    bin_width_left = plot_hist_custom_norm(delta_E_he3_V, ax, min_val, max_val, 'royalblue', 'V', '-', number_bins_dE_left, norm=(1/beam_monitor_norm_V[i]))
    plot_hist_custom_norm(delta_E_he3_SiO2, ax, min_val, max_val, 'indianred', 'SiO$_2$', '--', number_bins_dE_left, norm=(1/beam_monitor_norm_SiO2[i]))
    bin_widths_left.append(bin_width_left)
    ax.set_xlim(-Ei_in_meV, Ei_in_meV)
    ax.set_ylabel('%s\nIntensity' % energy_label)

#plt.tight_layout()
plt.subplots_adjust(wspace=.0)
plt.subplots_adjust(hspace=.2)
# Declare legend
legend_1 = [Line2D([0], [0], color='royalblue', label='V', linestyle='-'),
            Line2D([0], [0], color='indianred', label='SiO$_2$', linestyle='--')]
fig.legend(bbox_to_anchor=(0.7, 0.85), handles=legend_1, ncol=2)
# Declare energies
#center, start, step = 0.45, 0.86, 0.149
center, start, step = 0.5, 0.870, 0.147
bin_box_offset_x, bin_box_offset_y = 0.04, 0.083
labels_locations = [[center, start], [center, start], [center, 0.6], [center, 0.4], [center, 0.2]]
for i, (energy_label, bin_width) in enumerate(zip(energy_labels, bin_widths)):
    x_val, y_val = center, start - i*step
    fig.text(x_val, y_val, '\nBin width:\n%.3f meV' % bin_width, verticalalignment='center')
# Set axis labels
axes[0].axis('off')
axes[-1].set_xlabel('E$_i$ - E$_f$ (meV)')
fig.set_figwidth(3.5) #5
fig.set_figheight(12)
plt.tight_layout()
fig.suptitle('Helium-3 tubes, V vs SiO$_2$\nnormalized by beam monitors', x=0.55, y=0.88)
plt.savefig('%s/../output/vanadium_and_SiO2_he3_tubes.png' % nb_path, dpi=800)

## 4.3 After beamtime<a class="anchor" id="ISIS_AFTER_BEAMTIME"></a>

### 4.3.1 Outside detector testing area<a class="anchor" id="ISIS_AFTER_BEAMTIME_OUTSIDE_DETECTOR_TESTING_AREA"></a>

#### Plot overviews of datasets

In [ ]:
# Declare file names and areas
file_names = np.array([
                       'DEBUGGING_DAVIDELab_1100HV_wTh8gTh6_220506_132601',
                       'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220506_144328',
                       'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220506_175829',
                       'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220507_171415',
                       'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220508_214118',
                       'DEBUGGING_DAVIDELab_1100HV_wTh8gTh6_220506_132601',
                       'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220506_144328'
                       ])
mg_areas = 0.025*0.025*6*np.array([33,
                                   33,
                                   33,
                                   33,
                                   33,
                                   33,
                                   33
                                   ])
for file_name, mg_area in zip(file_names, mg_areas):
    # Extract and save (only has to be done once)
    #mg_manage.extract_and_save(file_name, MG_RAW_FOLDER, MG_PROCESSED_FOLDER)
    # Load data
    clu, ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
    # Plot data
    mg_basic_plot.mg_plot_basic_bus(file_name, 9, clu, ev, mg_filter_standard_no_prompt, mg_area, file_name)

#### Compare PHS between ArCO$_2$ bottle used during measurements and new ArCO$_2$ bottle

In [ ]:
# Declare MG filter
mg_filter_phs_comparison = {'wm': [1, 1, True],                   # Wire multiplicity
                            'gm': [1, 1, True],                   # Grid multiplicity
                            'wadc': [100, np.inf, True],          # Wire charge
                            'gadc': [100, np.inf, True],          # Grid charge
                            'tof': [0, np.inf, False],             # Time-of-flight (TDC channels)
                            'time': [0, np.inf, False],            # Time (TDC channels)
                            'bus': [9, 9, True],                      # Bus
                            'layer': [0, 16, False],              # Layer, front=0 to back=19
                            'row': [0, 11, False],                # Row, right to left (seen from neutrons)
                            'gch': [97, 131, False]}               # Grid channel, bottom=96 to top=132
# Declare file names
file_name_ng = 'DEBUGGING_AmBe_NEWGAS_1100HV_wTh8gTh6_220510_104714'
file_name_og = 'DEBUGGING_AmBe_OLDGAS_1100HV_wTh8gTh6_220510_221901'
# Get data
clu_ng, ev_ng = mg_manage.load_clusters_and_events(file_name_ng, MG_PROCESSED_FOLDER)
clu_og, ev_og = mg_manage.load_clusters_and_events(file_name_og, MG_PROCESSED_FOLDER)
# Filter data
clu_ng_f = mg_manage.filter_data(clu_ng, mg_filter_phs_comparison)
clu_og_f = mg_manage.filter_data(clu_og, mg_filter_phs_comparison)
# Get durations
ng_duration = (clu_ng_f.time.values[-1] - clu_ng_f.time.values[0]) * 62.5e-9
og_duration = (clu_og_f.time.values[-1] - clu_og_f.time.values[0]) * 62.5e-9
# Get scaling factors
scaling_factor_w = ((1/ng_duration)*len(clu_ng_f.wadc)) / ((1/og_duration)*len(clu_og_f.wadc))
scaling_factor_g = ((1/ng_duration)*len(clu_ng_f.gadc)) / ((1/og_duration)*len(clu_og_f.gadc))

print('Scaling factors wires: ', scaling_factor_w)
print('Scaling factors grids: ', scaling_factor_g)


# Comparison between new and old gas
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True, sharex=True)
number_bins = 300
ax1.set_title('Wires')
ax1.hist(clu_ng_f.wadc, bins=number_bins, weights=np.ones(clu_ng_f.shape[0])*(1/ng_duration),
         histtype='step', color='blue', range=[0, 8000], label='New gas')
ax1.hist(clu_og_f.wadc, bins=number_bins, weights=np.ones(clu_og_f.shape[0])*(1/og_duration),
         histtype='step', color='red', range=[0, 8000], label='Old gas', linestyle='--')
ax1.set_xlabel('Charge (ADC)')
ax1.set_ylabel('Counts/s')
#plt.yscale('log')
ax1.set_ylim(1e-4, 2e1)
ax1.grid(True, which='major', zorder=0)
ax1.grid(True, which='minor', linestyle='--', zorder=0)
ax1.set_xlim(0, 8000)

ax2.set_title('Grids')
ax2.hist(clu_ng_f.gadc, bins=number_bins, weights=np.ones(clu_ng_f.shape[0])*(1/ng_duration),
         histtype='step', color='blue', range=[0, 8000], label='New gas')
ax2.hist(clu_og_f.gadc, bins=number_bins, weights=np.ones(clu_og_f.shape[0])*(1/og_duration),
         histtype='step', color='red', range=[0, 8000], label='Old gas', linestyle='--')
ax2.set_xlabel('Charge (ADC)')
#plt.yscale('log')
ax2.set_ylim(1e-4, 2e1)
ax2.grid(True, which='major', zorder=0)
ax2.grid(True, which='minor', linestyle='--', zorder=0)
ax2.set_xlim(0, 8000)
ax2.legend(title='Data set')
#fig.set_figwidth(10)
#fig.set_figheight(5)
plt.subplots_adjust(hspace=.0)
plt.tight_layout()
plt.suptitle('Comparison')
plt.tight_layout()
plt.show()

# Comparison between new and old gas, old gas has been scaled overlap with new gas
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True, sharex=True)
number_bins = 300
ax1.set_title('Wires')
ax1.hist(clu_ng_f.wadc, bins=number_bins, weights=np.ones(clu_ng_f.shape[0])*(1/ng_duration),
         histtype='step', color='blue', range=[0, 8000], label='New gas')
ax1.hist(clu_og_f.wadc, bins=number_bins, weights=np.ones(clu_og_f.shape[0])*(1/og_duration) * scaling_factor_w,
         histtype='step', color='red', range=[0, 8000], label='Old gas', linestyle='--')
ax1.set_xlabel('Charge (ADC)')
ax1.set_ylabel('Counts/s')
#plt.yscale('log')
ax1.set_ylim(1e-4, 2e1)
ax1.grid(True, which='major', zorder=0)
ax1.grid(True, which='minor', linestyle='--', zorder=0)
ax1.set_xlim(0, 8000)

ax2.set_title('Grids')
ax2.hist(clu_ng_f.gadc, bins=number_bins, weights=np.ones(clu_ng_f.shape[0])*(1/ng_duration),
         histtype='step', color='blue', range=[0, 8000], label='New gas')
ax2.hist(clu_og_f.gadc, bins=number_bins, weights=np.ones(clu_og_f.shape[0])*(1/og_duration) * scaling_factor_g,
         histtype='step', color='red', range=[0, 8000], label='Old gas', linestyle='--')
ax2.set_xlabel('Charge (ADC)')
#plt.yscale('log')
ax2.set_ylim(1e-4, 2e1)
ax2.grid(True, which='major', zorder=0)
ax2.grid(True, which='minor', linestyle='--', zorder=0)
ax2.set_xlim(0, 8000)
ax2.legend(title='Data set')
#fig.set_figwidth(10)
#fig.set_figheight(5)
plt.subplots_adjust(hspace=.0)
plt.tight_layout()
plt.suptitle('Comparison (old gas has been scaled to overlap with new gas)')
plt.tight_layout()
plt.show()

#### Show rate vs time

In [ ]:
# Define helper functions
def get_dates_and_hist(times, time_bins, start_time_posix):
    hist, bin_edges = np.histogram(times, bins=time_bins)
    #print(hist)
    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
    delta_t = (bin_centers[1] - bin_centers[0])
    dates_and_times = []
    for i, bin_value in enumerate(bin_centers):
        time_in_posix = start_time_posix + bin_value
        time_in_dt = datetime.datetime.fromtimestamp(time_in_posix)
        dates_and_times.append(time_in_dt)
    #print(dates_and_times)
    return hist, delta_t, dates_and_times

def get_dates_and_hist_interval(times, time_bins, interval):
    hist, bin_edges = np.histogram(times, bins=time_bins, range=[interval[0], interval[1]])
    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
    delta_t = (bin_centers[1] - bin_centers[0])
    dates_and_times = np.empty([len(bin_centers)], dtype='object')
    for i, bin_value in enumerate(bin_centers):
        time_in_dt = datetime.datetime.fromtimestamp(bin_value)
        dates_and_times[i] = time_in_dt
    return hist, delta_t, dates_and_times, bin_edges

def get_mean_and_std(values):
    mean = sum(values)/len(values)
    std = np.sqrt((1/len(values)) * sum(((values - mean) ** 2)))
    return mean, std

In [ ]:
# Define parameters
bin_width = 600 # in seconds

# Define all Multi-Grid detector file names
mg_file_names = ['DEBUGGING_DAVIDELab_1100HV_wTh8gTh6_220506_132601',
                 'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220506_144328',
                 'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220506_175829',
                 'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220507_171415',
                 'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220508_214118'
                 ]

# Define all Helium-3 tube file names
he3_file_names = ['10cm3He1300V_UnShielded_5976s_Thr300mV',
                  '10cm3He1300V_UnShielded_254185s_Thr300mV']

# Get all Multi-Grid detector files
dfs_mg = []
mg_min_time = np.inf
mg_max_time = -np.inf
mg_df_full = pd.DataFrame()
for mg_file_name in mg_file_names:
    # Cluster and save data
    #mg_manage.extract_and_save(mg_file_name, MG_RAW_FOLDER, MG_PROCESSED_FOLDER)
    # Import data
    df_mg, __ = mg_manage.load_clusters_and_events(mg_file_name, MG_PROCESSED_FOLDER)
    start_posix = mg_manage.mg_get_start_time_in_posix(mg_file_name)
    # Filter data
    df_mg_f = mg_manage.filter_data(df_mg, mg_filter_standard)
    dfs_mg.append(df_mg_f)
    # Get min and max time
    smallest_time = df_mg_f.time.values[0] * 62.5e-9 + start_posix
    largest_time = df_mg_f.time.values[-1] * 62.5e-9 + start_posix
    if smallest_time < mg_min_time: mg_min_time = smallest_time
    if largest_time > mg_max_time: mg_max_time = largest_time
    df_mg_f['global_time'] = df_mg_f['time'] * 62.5e-9 + start_posix 
    mg_df_full = mg_df_full.append(df_mg_f, ignore_index=True)
# Get all Helium-3 tube files
dfs_he3 = []
ess_he3_filter = {'adc': [7000, np.inf, True], 'pile_up': [0, 0, True]}
he3_min_time = np.inf
he3_max_time = -np.inf
for he3_file_name in he3_file_names:
    he3_file_path = ESS_HE3_FOLDER + he3_file_name + '.lst'
    start_posix = ess_he3_tube_read.get_start_time_in_posix(he3_file_path)
    df_he3 = ess_he3_tube_read.extract_binary_events(he3_file_path)
    df_he3_f = ess_he3_tube_read.filter_data(df_he3, ess_he3_filter)
    dfs_he3.append(df_he3_f)
    # Get min and max time
    smallest_time = df_he3_f.tof.values[0] + start_posix
    largest_time = df_he3_f.tof.values[-1] + start_posix
    if smallest_time < he3_min_time: he3_min_time = smallest_time
    if largest_time > he3_max_time: he3_max_time = largest_time

overall_start_time_in_posix = min([he3_min_time, mg_min_time])
overall_stop_time_in_posix = max([he3_max_time, mg_max_time])


fig, ax = plt.subplots(1, 1)
fig.autofmt_xdate()
formatter = matplotlib.dates.DateFormatter('%d-%m-%y %H:%M')
ax.xaxis.set_major_formatter(formatter)

# Insert Multi-Grid data
label = None
mg_times = np.array([])
for i, (df_mg_f, mg_file_name) in enumerate(zip(dfs_mg, mg_file_names)):
    start_posix = mg_manage.mg_get_start_time_in_posix(mg_file_name)
    mg_times = np.concatenate((mg_times, start_posix + df_mg_f.time.values * 62.5e-9))
    # Calculate number of time bins
    duration = (df_mg_f.time.values[-1] - df_mg_f.time.values[0]) * 62.5e-9
    time_bins = int(duration/bin_width)
    hist, delta_t, dates_and_times = get_dates_and_hist(df_mg_f.time.values * 62.5e-9, time_bins, start_posix)
    if i == (len(dfs_mg)-1): label = 'Multi-Grid detector'
    ax.errorbar(dates_and_times, (hist/delta_t), (np.sqrt(hist)/delta_t), 
                linestyle='', marker='o', zorder=5, label=label, color='blue')
    
# Insert Helium-3 data
label = None
he3_times = np.array([])
for i, (df_he3_f, he3_file_name) in enumerate(zip(dfs_he3, he3_file_names)):
    he3_file_path = ESS_HE3_FOLDER + he3_file_name + '.lst'
    start_posix = ess_he3_tube_read.get_start_time_in_posix(he3_file_path)
    he3_times = np.concatenate((he3_times, start_posix + df_he3_f.tof.values))
    # Calculate number of time bins
    duration = df_he3_f.tof.values[-1] - df_he3_f.tof.values[0]
    time_bins = int(duration/bin_width)
    hist, delta_t, dates_and_times = get_dates_and_hist(df_he3_f.tof.values, time_bins, start_posix)
    if i == (len(dfs_he3)-1): label = 'Helium-3 tube'
    ax.errorbar(dates_and_times, (hist/delta_t), (np.sqrt(hist)/delta_t), 
                linestyle='', marker='o', zorder=5, label=label, color='red')

ax.set_xlabel('Date and time')
ax.set_ylabel('Rate (Hz)')
#ax.set_yscale('log')
ax.legend(title='Detector')
ax.grid(True, which='major', zorder=0)
ax.grid(True, which='minor', linestyle='--', zorder=0)

fig.set_size_inches(12, 6)


# Plot with common binning
number_bins = 100
interval = [overall_start_time_in_posix, overall_stop_time_in_posix]
fig, ax = plt.subplots(1, 1)
fig.autofmt_xdate()
formatter = matplotlib.dates.DateFormatter('%d-%m-%y %H:%M')
ax.xaxis.set_major_formatter(formatter)

hist_mg, delta_t_mg, dates_and_times_mg, bin_edges_mg = get_dates_and_hist_interval(mg_times, number_bins, interval)
idxs_mg_not_zero = np.where(hist_mg != 0)[0] #[1:-1] # Remove edge data points to reduce binning effect

hist_he3, delta_t_he3, dates_and_times_he3, bin_edges_he3 = get_dates_and_hist_interval(he3_times, number_bins, interval)
idxs_he3_not_zero = np.where(hist_he3 != 0)[0] #[1:-1] # Remove edge data points to reduce binning effect

ax.errorbar(dates_and_times_mg[idxs_mg_not_zero],
            (hist_mg[idxs_mg_not_zero]/delta_t_mg),
            (np.sqrt(hist_mg[idxs_mg_not_zero])/delta_t_mg), 
            linestyle='', marker='o', zorder=5, label='Multi-Grid detector', color='blue')
ax.errorbar(dates_and_times_he3[idxs_he3_not_zero],
            (hist_he3[idxs_he3_not_zero]/delta_t_he3),
            (np.sqrt(hist_he3[idxs_he3_not_zero])/delta_t_he3), 
            linestyle='', marker='o', zorder=5, label='Helium-3 tube', color='red')
idx_before_jump = 29
#ax.plot(dates_and_times_he3[idx_before_jump], hist_he3[idx_before_jump]/delta_t_he3, marker='x', color='black', zorder=50, markersize=10)
print('Ratio MG: %f' % (hist_mg[idx_before_jump+1]/hist_mg[idx_before_jump]))
print('Ratio He-3: %f' % (hist_he3[idx_before_jump+1]/hist_he3[idx_before_jump]))

ax.set_xlabel('Date and time')
ax.set_ylabel('Rate (Hz)')
#ax.set_yscale('log')
ax.legend(title='Detector')
ax.grid(True, which='major', zorder=0)
ax.grid(True, which='minor', linestyle='--', zorder=0)

fig.set_size_inches(12, 6)

# Plot Multi-Grid data normalized by Helium-3 data
fig, ax = plt.subplots(1, 1)
fig.autofmt_xdate()
formatter = matplotlib.dates.DateFormatter('%d-%m-%y %H:%M')
ax.xaxis.set_major_formatter(formatter)

dates_and_times_common = dates_and_times_he3[idxs_he3_not_zero]
mg_values = hist_mg[idxs_he3_not_zero]
he3_values = hist_he3[idxs_he3_not_zero]
ratios = mg_values/he3_values
error = ratios * np.sqrt((np.sqrt(mg_values)/mg_values)**2 + (np.sqrt(he3_values)/he3_values)**2)

ax.errorbar(dates_and_times_common, ratios, error, 
            linestyle='', marker='o', zorder=5, color='black')

ax.grid(True, which='major', zorder=0)
ax.grid(True, which='minor', linestyle='--', zorder=0)

ax.set_xlabel('Date and time')
ax.set_xlim(dates_and_times_mg[0], dates_and_times_mg[-1])
ax.set_ylabel('Multi-Grid detector / Helium-3 tube')

fig.set_size_inches(12, 6)

#### Animate PHS as a function of rate

In [ ]:
# Define parameters
bin_width = 600 # in seconds

# Define all Multi-Grid detector file names
mg_file_names = ['DEBUGGING_DAVIDELab_1100HV_wTh8gTh6_220506_132601',
                 'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220506_144328',
                 'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220506_175829',
                 'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220507_171415',
                 'DEBUGGING_DAVIDELab_NEWGAS_1100HV_wTh8gTh6_220508_214118'
                 ]

# Define all Helium-3 tube file names
he3_file_names = ['10cm3He1300V_UnShielded_5976s_Thr300mV',
                  '10cm3He1300V_UnShielded_254185s_Thr300mV']

# Get all Multi-Grid detector files
dfs_mg = []
mg_min_time = np.inf
mg_max_time = -np.inf
mg_df_full = pd.DataFrame()
for mg_file_name in mg_file_names:
    # Cluster and save data
    #mg_manage.extract_and_save(mg_file_name, MG_RAW_FOLDER, MG_PROCESSED_FOLDER)
    # Import data
    df_mg, __ = mg_manage.load_clusters_and_events(mg_file_name, MG_PROCESSED_FOLDER)
    start_posix = mg_manage.mg_get_start_time_in_posix(mg_file_name)
    # Filter data
    df_mg_f = mg_manage.filter_data(df_mg, mg_filter_standard)
    dfs_mg.append(df_mg_f)
    # Get min and max time
    smallest_time = df_mg_f.time.values[0] * 62.5e-9 + start_posix
    largest_time = df_mg_f.time.values[-1] * 62.5e-9 + start_posix
    if smallest_time < mg_min_time: mg_min_time = smallest_time
    if largest_time > mg_max_time: mg_max_time = largest_time
    df_mg_f['global_time'] = df_mg_f['time'] * 62.5e-9 + start_posix 
    mg_df_full = mg_df_full.append(df_mg_f, ignore_index=True)
print(mg_df_full)
# Get all Helium-3 tube files
dfs_he3 = []
ess_he3_filter = {'adc': [11500, np.inf, True], 'pile_up': [0, 0, True]}
he3_min_time = np.inf
he3_max_time = -np.inf
for he3_file_name in he3_file_names:
    file_path = ESS_HE3_FOLDER + he3_file_name + '.lst'
    start_posix = ess_he3_tube_read.get_start_time_in_posix(he3_file_path)
    df_he3 = ess_he3_tube_read.extract_binary_events(file_path)
    df_he3_f = ess_he3_tube_read.filter_data(df_he3, ess_he3_filter)
    dfs_he3.append(df_he3_f)
    # Get min and max time
    smallest_time = df_he3_f.tof.values[0] + start_posix
    largest_time = df_he3_f.tof.values[-1] + start_posix
    if smallest_time < he3_min_time: he3_min_time = smallest_time
    if largest_time > he3_max_time: he3_max_time = largest_time

overall_start_time_in_posix = mg_min_time #min([he3_min_time, mg_min_time])
overall_stop_time_in_posix = mg_max_time #max([he3_max_time, mg_max_time])


fig, ax = plt.subplots(1, 1)
fig.autofmt_xdate()
formatter = matplotlib.dates.DateFormatter('%d-%m-%y %H:%M')
ax.xaxis.set_major_formatter(formatter)

# Insert Multi-Grid data
label = None
mg_times = np.array([])
for i, (df_mg_f, mg_file_name) in enumerate(zip(dfs_mg, mg_file_names)):
    start_posix = mg_manage.mg_get_start_time_in_posix(mg_file_name)
    mg_times = np.concatenate((mg_times, start_posix + df_mg_f.time.values * 62.5e-9))
    # Calculate number of time bins
    duration = (df_mg_f.time.values[-1] - df_mg_f.time.values[0]) * 62.5e-9
    time_bins = int(duration/bin_width)
    hist, delta_t, dates_and_times = get_dates_and_hist(df_mg_f.time.values * 62.5e-9, time_bins, start_posix)
    if i == (len(dfs_mg)-1): label = 'Multi-Grid detector'
    ax.errorbar(dates_and_times, (hist/delta_t), (np.sqrt(hist)/delta_t), 
                linestyle='', marker='o', zorder=5, label=label, color='blue')
    
# Insert Helium-3 data
label = None
he3_times = np.array([])
for i, (df_he3_f, he3_file_name) in enumerate(zip(dfs_he3, he3_file_names)):
    he3_file_path = ESS_HE3_FOLDER + he3_file_name + '.lst'
    start_posix = ess_he3_tube_read.get_start_time_in_posix(he3_file_path)
    he3_times = np.concatenate((he3_times, start_posix + df_he3_f.tof.values))
    # Calculate number of time bins
    duration = df_he3_f.tof.values[-1] - df_he3_f.tof.values[0]
    time_bins = int(duration/bin_width)
    hist, delta_t, dates_and_times = get_dates_and_hist(df_he3_f.tof.values, time_bins, start_posix)
    if i == (len(dfs_he3)-1): label = 'Helium-3 tube'
    ax.errorbar(dates_and_times, (hist/delta_t), (np.sqrt(hist)/delta_t), 
                linestyle='', marker='o', zorder=5, label=label, color='red')

ax.set_xlabel('Date and time')
ax.set_ylabel('Rate (Hz)')
#ax.set_yscale('log')
ax.legend(title='Detector')
ax.grid(True, which='major', zorder=0)
ax.grid(True, which='minor', linestyle='--', zorder=0)

fig.set_size_inches(12, 6)


# Plot with common binning
number_bins = 50
interval = [overall_start_time_in_posix, overall_stop_time_in_posix]
fig, ax = plt.subplots(1, 1)
fig.autofmt_xdate()
formatter = matplotlib.dates.DateFormatter('%d-%m-%y %H:%M')
ax.xaxis.set_major_formatter(formatter)

hist_mg, delta_t_mg, dates_and_times_mg, bin_edges_mg = get_dates_and_hist_interval(mg_times, number_bins, interval)
idxs_mg_not_zero = np.where(hist_mg != 0)[0] #[1:-1] # Remove edge data points to reduce binning effect

hist_he3, delta_t_he3, dates_and_times_he3, bin_edges_he3 = get_dates_and_hist_interval(he3_times, number_bins, interval)
idxs_he3_not_zero = np.where(hist_he3 != 0)[0] #[1:-1] # Remove edge data points to reduce binning effect

ax.errorbar(dates_and_times_mg[idxs_mg_not_zero],
            (hist_mg[idxs_mg_not_zero]/delta_t_mg),
            (np.sqrt(hist_mg[idxs_mg_not_zero])/delta_t_mg), 
            linestyle='', marker='o', zorder=5, label='Multi-Grid detector', color='blue')
ax.errorbar(dates_and_times_he3[idxs_he3_not_zero],
            (hist_he3[idxs_he3_not_zero]/delta_t_he3),
            (np.sqrt(hist_he3[idxs_he3_not_zero])/delta_t_he3), 
            linestyle='', marker='o', zorder=5, label='Helium-3 tubes', color='red')

ax.set_xlabel('Date and time')
ax.set_ylabel('Rate (Hz)')
#ax.set_yscale('log')
ax.legend(title='Detector')
ax.grid(True, which='major', zorder=0)
ax.grid(True, which='minor', linestyle='--', zorder=0)

fig.set_size_inches(12, 6)

# Plot Multi-Grid data normalized by Helium-3 data
fig, ax = plt.subplots(1, 1)
fig.autofmt_xdate()
formatter = matplotlib.dates.DateFormatter('%d-%m-%y %H:%M')
ax.xaxis.set_major_formatter(formatter)

dates_and_times_common = dates_and_times_he3[idxs_he3_not_zero]
mg_values = hist_mg[idxs_he3_not_zero]
he3_values = hist_he3[idxs_he3_not_zero]
ratios = mg_values/he3_values
error = ratios * np.sqrt((np.sqrt(mg_values)/mg_values)**2 + (np.sqrt(he3_values)/he3_values)**2)

ax.errorbar(dates_and_times_common, ratios, error, 
            linestyle='', marker='o', zorder=5, color='black')

ax.grid(True, which='major', zorder=0)
ax.grid(True, which='minor', linestyle='--', zorder=0)

ax.set_xlabel('Date and time')
ax.set_xlim(dates_and_times_mg[0], dates_and_times_mg[-1])
ax.set_ylabel('Multi-Grid detector / Helium-3 tube')

fig.set_size_inches(12, 6)


# Plot Multi-Grid data rate vs time together with PHS vs time animation


# Declare animation folder
animation_folder = nb_path + '/../output/animation/'
shutil.rmtree(animation_folder, ignore_errors=True)
hf.mkdir_p(animation_folder)

# Iterate through data
number_phs_bins = 100
for i, __ in enumerate(tqdm.tqdm(bin_edges_mg[:-1])):
    start_time, stop_time = bin_edges_mg[i], bin_edges_mg[i+1]
    mg_df_slice = mg_df_full[(mg_df_full['global_time'] >= start_time) &
                             (mg_df_full['global_time'] < stop_time)]
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.autofmt_xdate()
    formatter = matplotlib.dates.DateFormatter('%d-%m-%y %H:%M')
    if len(mg_df_slice.values) > 2:
        mg_slice_duration = mg_df_slice['global_time'].values[-1] - mg_df_slice['global_time'].values[0]
    else:
        mg_slice_duration = -1
    ax2.hist(mg_df_slice.wadc, bins=number_phs_bins, histtype='step',
             zorder=5, range=[0, 8000], label='Wires (filtered)', color='blue',
             weights=(1/mg_slice_duration)*np.ones(len(mg_df_slice.wadc)))
    ax2.hist(mg_df_slice.gadc, bins=number_phs_bins, histtype='step',
             zorder=5, range=[0, 8000], label='Grids (filtered)', color='red',
             weights=(1/mg_slice_duration)*np.ones(len(mg_df_slice.gadc)))
    ax2.grid(True, which='major', zorder=0)
    ax2.grid(True, which='minor', linestyle='--', zorder=0)
    ax2.set_ylabel('counts/s')
    ax2.set_yscale('log')
    ax2.legend()
    ax2.set_ylim(1e-4, 10)
    ax2.set_title('PHS')
    ax2.set_xlabel('Charge (ADC channels)')

    ax1.set_title('Rate')
    ax1.set_ylabel('Rate (Hz)')
    ax1.grid(True, which='major', zorder=0)
    ax1.grid(True, which='minor', linestyle='--', zorder=0)
    ax1.errorbar(dates_and_times_mg[idxs_mg_not_zero],
                 (hist_mg[idxs_mg_not_zero]/delta_t_mg),
                 (np.sqrt(hist_mg[idxs_mg_not_zero])/delta_t_mg), 
                 linestyle='', marker='o', zorder=5, label='Multi-Grid detector', color='black')
    average_posix_time = int((stop_time-start_time)/2 + start_time)
    average_dt_time = datetime.datetime.fromtimestamp(average_posix_time)
    ax1.axvline(average_dt_time, color='red')
    ax1.xaxis.set_major_formatter(formatter)
    fig.set_size_inches(12, 6)

    # Save data
    output_path = animation_folder + str(i) + '.png'
    fig.savefig(output_path, bbox_inches='tight')
    plt.close()

# Animate
output_path = '../output/rate_vs_PHS_sweep.gif'
images = []
files = os.listdir(animation_folder)
files = [file[:-4] for file in files if file[-9:] != '.DS_Store' and file != '.gitignore']
for file in sorted(files, key=int):
    images.append(imageio.imread(animation_folder + file + '.png'))
imageio.mimsave(output_path, images)
shutil.rmtree(animation_folder, ignore_errors=True)

### 4.3.2 AmBe source<a class="anchor" id="ISIS_AFTER_BEAMTIME_AMBE_SOURCE"></a>

#### Plot overviews of datasets

In [ ]:
# Declare file names and areas
file_names = np.array([
                       'DEBUGGING_AmBe_NEWGAS_1100HV_wTh8gTh6_220510_104714',
                       'DEBUGGING_AmBe_GAS_1100HV_wTh8gTh6_220510_115332',
                       'DEBUGGING_AmBe_OLDGAS_1100HV_wTh8gTh6_220510_150015',
                       'DEBUGGING_AmBe_OLDGAS_1100HV_wTh8gTh6_220510_154405',
                       'DEBUGGING_AmBe_OLDGAS_1100HV_wTh8gTh6_220510_221901'
                       ])
mg_areas = 0.025*0.025*6*np.array([33,
                                   33,
                                   33,
                                   33,
                                   33
                                   ])
for file_name, mg_area in zip(file_names, mg_areas):
    # Extract and save (only has to be done once)
    #mg_manage.extract_and_save(file_name, MG_RAW_FOLDER, MG_PROCESSED_FOLDER)
    # Load data
    clu, ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
    # Plot data
    mg_basic_plot.mg_plot_basic_bus(file_name, 9, clu, ev, mg_filter_standard_no_prompt, mg_area, file_name)

#### Show rate vs time

In [ ]:
# Define parameters
bin_width = 600 # in seconds

# Define all Multi-Grid detector file names
mg_file_names = ['DEBUGGING_AmBe_NEWGAS_1100HV_wTh8gTh6_220510_104714',
                 'DEBUGGING_AmBe_GAS_1100HV_wTh8gTh6_220510_115332',
                 'DEBUGGING_AmBe_OLDGAS_1100HV_wTh8gTh6_220510_150015',
                 'DEBUGGING_AmBe_OLDGAS_1100HV_wTh8gTh6_220510_154405',
                 'DEBUGGING_AmBe_OLDGAS_1100HV_wTh8gTh6_220510_221901']

# Define all Helium-3 tube file names
he3_file_names = ['10cm3He1300V_UnShielded_4697s_Thr300mV',
                  '10cm3He1300V_UnShielded_78825s_Thr300mV']

# Get all Multi-Grid detector files
dfs_mg = []
mg_min_time = np.inf
mg_max_time = -np.inf
mg_df_full = pd.DataFrame()
for mg_file_name in mg_file_names:
    # Cluster and save data
    #mg_manage.extract_and_save(mg_file_name, MG_RAW_FOLDER, MG_PROCESSED_FOLDER)
    # Import data
    df_mg, __ = mg_manage.load_clusters_and_events(mg_file_name, MG_PROCESSED_FOLDER)
    start_posix = mg_manage.mg_get_start_time_in_posix(mg_file_name)
    # Filter data
    df_mg_f = mg_manage.filter_data(df_mg, mg_filter_standard)
    dfs_mg.append(df_mg_f)
    # Get min and max time
    smallest_time = df_mg_f.time.values[0] * 62.5e-9 + start_posix
    largest_time = df_mg_f.time.values[-1] * 62.5e-9 + start_posix
    if smallest_time < mg_min_time: mg_min_time = smallest_time
    if largest_time > mg_max_time: mg_max_time = largest_time
    df_mg_f['global_time'] = df_mg_f['time'] * 62.5e-9 + start_posix 
    mg_df_full = mg_df_full.append(df_mg_f, ignore_index=True)
# Get all Helium-3 tube files
dfs_he3 = []
ess_he3_filter = {'adc': [7000, np.inf, True], 'pile_up': [0, 0, True]}
he3_min_time = np.inf
he3_max_time = -np.inf
for he3_file_name in he3_file_names:
    he3_file_path = ESS_HE3_FOLDER + he3_file_name + '.lst'
    start_posix = ess_he3_tube_read.get_start_time_in_posix(he3_file_path)
    df_he3 = ess_he3_tube_read.extract_binary_events(he3_file_path)
    df_he3_f = ess_he3_tube_read.filter_data(df_he3, ess_he3_filter)
    dfs_he3.append(df_he3_f)
    # Get min and max time
    smallest_time = df_he3_f.tof.values[0] + start_posix
    largest_time = df_he3_f.tof.values[-1] + start_posix
    if smallest_time < he3_min_time: he3_min_time = smallest_time
    if largest_time > he3_max_time: he3_max_time = largest_time

overall_start_time_in_posix = min([he3_min_time, mg_min_time])
overall_stop_time_in_posix = max([he3_max_time, mg_max_time])


fig, ax = plt.subplots(1, 1)
fig.autofmt_xdate()
formatter = matplotlib.dates.DateFormatter('%d-%m-%y %H:%M')
ax.xaxis.set_major_formatter(formatter)

# Insert Multi-Grid data
label = None
mg_times = np.array([])
for i, (df_mg_f, mg_file_name) in enumerate(zip(dfs_mg, mg_file_names)):
    start_posix = mg_manage.mg_get_start_time_in_posix(mg_file_name)
    mg_times = np.concatenate((mg_times, start_posix + df_mg_f.time.values * 62.5e-9))
    # Calculate number of time bins
    duration = (df_mg_f.time.values[-1] - df_mg_f.time.values[0]) * 62.5e-9
    time_bins = int(duration/bin_width)
    hist, delta_t, dates_and_times = get_dates_and_hist(df_mg_f.time.values * 62.5e-9, time_bins, start_posix)
    if i == (len(dfs_mg)-1): label = 'Multi-Grid detector'
    ax.errorbar(dates_and_times, (hist/delta_t), (np.sqrt(hist)/delta_t), 
                linestyle='', marker='o', zorder=5, label=label, color='blue')
    
# Insert Helium-3 data
label = None
he3_times = np.array([])
for i, (df_he3_f, he3_file_name) in enumerate(zip(dfs_he3, he3_file_names)):
    he3_file_path = ESS_HE3_FOLDER + he3_file_name + '.lst'
    start_posix = ess_he3_tube_read.get_start_time_in_posix(he3_file_path)
    he3_times = np.concatenate((he3_times, start_posix + df_he3_f.tof.values))
    # Calculate number of time bins
    duration = df_he3_f.tof.values[-1] - df_he3_f.tof.values[0]
    time_bins = int(duration/bin_width)
    hist, delta_t, dates_and_times = get_dates_and_hist(df_he3_f.tof.values, time_bins, start_posix)
    if i == (len(dfs_he3)-1): label = 'Helium-3 tube'
    ax.errorbar(dates_and_times, (hist/delta_t), (np.sqrt(hist)/delta_t), 
                linestyle='', marker='o', zorder=5, label=label, color='red')

ax.set_xlabel('Date and time')
ax.set_ylabel('Rate (Hz)')
#ax.set_yscale('log')
ax.legend(title='Detector')
ax.grid(True, which='major', zorder=0)
ax.grid(True, which='minor', linestyle='--', zorder=0)

fig.set_size_inches(12, 6)


# Plot with common binning
number_bins = 100
interval = [overall_start_time_in_posix, overall_stop_time_in_posix]
fig, ax = plt.subplots(1, 1)
fig.autofmt_xdate()
formatter = matplotlib.dates.DateFormatter('%d-%m-%y %H:%M')
ax.xaxis.set_major_formatter(formatter)

hist_mg, delta_t_mg, dates_and_times_mg, bin_edges_mg = get_dates_and_hist_interval(mg_times, number_bins, interval)
idxs_mg_not_zero = np.where(hist_mg != 0)[0] #[1:-1] # Remove edge data points to reduce binning effect

hist_he3, delta_t_he3, dates_and_times_he3, bin_edges_he3 = get_dates_and_hist_interval(he3_times, number_bins, interval)
idxs_he3_not_zero = np.where(hist_he3 != 0)[0] #[1:-1] # Remove edge data points to reduce binning effect

ax.errorbar(dates_and_times_mg[idxs_mg_not_zero],
            (hist_mg[idxs_mg_not_zero]/delta_t_mg),
            (np.sqrt(hist_mg[idxs_mg_not_zero])/delta_t_mg), 
            linestyle='', marker='o', zorder=5, label='Multi-Grid detector', color='blue')
ax.errorbar(dates_and_times_he3[idxs_he3_not_zero],
            (hist_he3[idxs_he3_not_zero]/delta_t_he3),
            (np.sqrt(hist_he3[idxs_he3_not_zero])/delta_t_he3), 
            linestyle='', marker='o', zorder=5, label='Helium-3 tube', color='red')
idx_before_jump = 29
#ax.plot(dates_and_times_he3[idx_before_jump], hist_he3[idx_before_jump]/delta_t_he3, marker='x', color='black', zorder=50, markersize=10)
print('Ratio MG: %f' % (hist_mg[idx_before_jump+1]/hist_mg[idx_before_jump]))
print('Ratio He-3: %f' % (hist_he3[idx_before_jump+1]/hist_he3[idx_before_jump]))

ax.set_xlabel('Date and time')
ax.set_ylabel('Rate (Hz)')
#ax.set_yscale('log')
ax.legend(title='Detector')
ax.grid(True, which='major', zorder=0)
ax.grid(True, which='minor', linestyle='--', zorder=0)

fig.set_size_inches(12, 6)

# Plot Multi-Grid data normalized by Helium-3 data
fig, ax = plt.subplots(1, 1)
fig.autofmt_xdate()
formatter = matplotlib.dates.DateFormatter('%d-%m-%y %H:%M')
ax.xaxis.set_major_formatter(formatter)

dates_and_times_common = dates_and_times_he3[idxs_he3_not_zero]
mg_values = hist_mg[idxs_he3_not_zero]
he3_values = hist_he3[idxs_he3_not_zero]
ratios = mg_values/he3_values
error = ratios * np.sqrt((np.sqrt(mg_values)/mg_values)**2 + (np.sqrt(he3_values)/he3_values)**2)

ax.errorbar(dates_and_times_common, ratios, error, 
            linestyle='', marker='o', zorder=5, color='black')

ax.grid(True, which='major', zorder=0)
ax.grid(True, which='minor', linestyle='--', zorder=0)

ax.set_xlabel('Date and time')
ax.set_xlim(dates_and_times_mg[0], dates_and_times_mg[-1])
ax.set_ylabel('Multi-Grid detector / Helium-3 tube')

fig.set_size_inches(12, 6)

### 4.3.3 Cave background<a class="anchor" id="ISIS_AFTER_BEAMTIME_CAVE_BACKGROUND"></a>

#### Plot overview of dataset

In [ ]:
# Declare file name and area
file_name = 'DEBUGGING_AmBe_NewGAS_1100HV_wTh8gTh6_220511_121605'
mg_area = 0.025*0.025*6*30
# Extract and save (only has to be done once)
#mg_manage.extract_and_save(file_name, MG_RAW_FOLDER, MG_PROCESSED_FOLDER)
# Load data
clu, ev = mg_manage.load_clusters_and_events(file_name, MG_PROCESSED_FOLDER)
# Plot data
mg_basic_plot.mg_plot_basic_bus(file_name, 9, clu, ev, mg_filter_standard, mg_area, file_name)

# 5. Summary<a class="anchor" id="SUMMARY"></a>